# **0. 데이터 불러오기**

In [ ]:
#나눔폰트 설치 후  런타임 재시작 
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 37 not upgraded.
Need to get 9,604 kB of archives.
After this operation, 29.5 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 fonts-nanum all 20170925-1 [9,604 kB]
Fetched 9,604 kB in 1s (10.1 MB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package fonts-nanum.
(Reading database ... 155225 files and di

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: ignored

In [ ]:
import pandas as pd
import numpy as np
# 경고무시
import warnings
warnings.filterwarnings('ignore')
# 정규표현식
import re

In [ ]:
df_policy = pd.read_excel('/content/drive/MyDrive/Wello+project_신희택,윤다솜/Data/policy_metetable(AIB전달용).xlsx')
df_policy = df_policy.sort_values(by='정책ID').reset_index(drop=True)
# 결론적으로 분석에 큰 지장이 없을 것 같아 소관기관유형.1과 소관기관.1을 삭제하도록함.
df_policy = df_policy.drop(['소관기관유형.1', '소관기관.1'],axis=1)
df_policy.head()

In [ ]:
# wello admit data 가져오기
wello = pd.read_csv('/content/drive/MyDrive/Wello+project_신희택,윤다솜/Data/정책정보_2021-12-28.csv',error_bad_lines=False)
# 2439번째 데이터 컬럼이 하나씩 밀려서 제외하고 가져오기.
wello = wello.sort_values(by='정책ID').reset_index(drop=True)
wello

In [ ]:
wello_compared=wello.iloc[:88486].copy()
wello_compared.head()

# **# IDEA**

## **0. 결측치 확인**

In [ ]:
df_policy.shape

In [ ]:
# 결측치 확인
df_policy.isna().sum() # 0인 값들, 정책에 항상 있는 값이자 중요 변수

In [ ]:
df_policy['서비스명'].unique()

## **1. 변수 연결**

In [ ]:
df_policy.columns

In [ ]:
wello_compared.columns

* 신청절차

In [ ]:
df_policy['신청절차'].unique()

In [ ]:
wello_compared['신청절차'].unique()
# 온라인, 오프라인 => 복합으로 통일

* 자녀

In [ ]:
df_policy['생애주기'].unique()

In [ ]:
wello_compared['자녀'].unique()
# 없음 있음은 대체 무엇 ?

* 가구원

In [ ]:
df_policy['생애주기'].unique()

In [ ]:
wello_compared['가구원'].unique()


* 성별

In [ ]:
df_policy['지원대상'].unique()

In [ ]:
wello_compared['성별'].unique()
# 온라인, 오프라인 => 복합으로 통일

* 학력

In [ ]:
df_policy['선정기준'].unique()

In [ ]:
wello_compared[wello_compared['성별']=='여성']

In [ ]:
wello_compared['학력'].unique()
# 온라인, 오프라인 => 복합으로 통일

* 성별

In [ ]:
df_policy['지원유형'].unique()

In [ ]:
wello_compared['지원유형'].unique()
# 온라인, 오프라인 => 복합으로 통일

* 관심사항특성

In [ ]:
wello_compared['관심상황특성'].unique()
# 온라인, 오프라인 => 복합으로 통일

# **I차. 데이터 Feature Engineering**

## **1. 지역(시도) - sido**

### A. TAG CODE

In [ ]:
# 지역(시도) 변수 생성
# 아래 시군구를 통해 지역(시도)값 더 보충한다.
def sido(df):
  for i in range(len(df)):
    AGENCY_TYPE = df.loc[i,'소관기관유형']
    # 소관기관 유형이 광역시도인 경우 소관기관을 시도값으로 넣는다.
    if AGENCY_TYPE=='광역시도':
      df.loc[i,'sido'] = df.loc[i,'소관기관']
    # 소관기관 유형이 시군구인 경우 띄어쓰기 앞에있는 시도값을 넣는다.
    elif AGENCY_TYPE=='시군구':
      sd = df.loc[i,'소관기관'].split(' ')[0]
      df.loc[i,'sido'] = sd
    # 소관기관 유형이 교육청인 경우 교육청을 제외한 값이 시도값이므로 넣는다.
    elif AGENCY_TYPE == '교육청':
      df.loc[i,'sido'] = df.loc[i,'소관기관'][:-3]
    # 이외의 값은 일단 전국으로 넣어준다.
    else:
      df.loc[i,'sido'] = '전국'
  return df

In [ ]:
# 약 5분 소요 실행
sido(df_policy)

### B. 정리

1. 소관기관유형이 광역시도인 경우 소관기관은 바로 시도값으로
2. 소관기관 유형이 시군구인 경우, 앞의 값 넣기. (강원도 춘천시)
3. 소관기관 유형이 교육청인 경우, 교육청 제외한 앞 값 넣기. (강원도교육청)
4. 기타 전국

### C. TAG CODE 전체

In [ ]:
# df_policy[df_policy['소관기관유형']=='시군구']

In [ ]:
# # 지역(시도) 변수 생성
# # 아래 시군구를 통해 지역(시도)값 더 보충한다.
# def sido(df):
#   for i in range(len(df)):
#     AGENCY_TYPE = df.loc[i,'소관기관유형']
#     # 소관기관 유형이 광역시도인 경우 소관기관을 시도값으로 넣는다.
#     if AGENCY_TYPE=='광역시도':
#       df.loc[i,'sido'] = df.loc[i,'소관기관']
#     # 소관기관 유형이 시군구인 경우 띄어쓰기 앞에있는 시도값을 넣는다.
#     elif AGENCY_TYPE=='시군구':
#       sd = df.loc[i,'소관기관'].split(' ')[0]
#       df.loc[i,'sido'] = sd
#     # 소관기관 유형이 교육청인 경우 교육청을 제외한 값이 시도값이므로 넣는다.
#     elif AGENCY_TYPE == '교육청':
#       df.loc[i,'sido'] = df.loc[i,'소관기관'][:-3]
#     # 이외의 값은 일단 전국으로 넣어준다.
#     else:
#       df.loc[i,'sido'] = '전국'
#   return df
# # 약 5분 소요 실행
# sido(df_policy)

## **2. 지역(시군구) - sigungu**

### A. TAG CODE

*1. 도 - 시군구 테이블 & 광역시도 테이블 생성*

In [ ]:
# 1. 도와 시군구 테이블 생성
# 시군구에 대한 값을 추출해보았다.
DSGG = df_policy[df_policy['소관기관유형']=='시군구']['소관기관'].unique()
df_DSGG = pd.DataFrame(columns=['도','시군구'])

In [ ]:
# 소관기관유형 시군구인 소관기관에서 각 도와 시군구값 뺴기
for i in range(len(DSGG)):
  val = DSGG[i].split(' ')
  df_DSGG.loc[i,'도'] = val[0]
  df_DSGG.loc[i,'시군구'] = val[1]  

In [ ]:
def namesigun(var):
  return var[:2]

In [ ]:
df_DSGG['시군구명'] = df_DSGG['시군구'].apply(namesigun)

In [ ]:
# df_DSGG 는 도와 시군구 시군구명 앞의 두글자를 따서 만든 데이터 프레임이다.
# 이 데이터 프레임의 쓰임은 소관기관유형이 공공기관인 곳에서 '시흥 갯골..' 이런식으로 시작하는 공공기관을 지역(시도) 및 지역(시군구)로 만들기 위함이다.
df_DSGG

In [ ]:
UNIQ = df_policy[df_policy['소관기관유형']=='공공기관 등']['소관기관'].unique()
# 광역시가 들어간 공공기관 뽑기
[i for i in UNIQ if '광역시' in i]
# 광역시 + 도시 or 시설 or 구 포함.
# 지역(시군구) 추출 시 if 문으로 나눌 수 있음.

#### **!. 충남, 충북 & (주) 떼는거 **

In [ ]:
# 2. 광역시도 관련된 광역시도명 테이블 만들기
sido_total = df_policy[df_policy['소관기관유형']=='광역시도']['소관기관'].unique()
df_sido_total = pd.DataFrame(columns=['광역시도','광역시도명'])
for i in range(len(sido_total)):
  val = sido_total[i]
  df_sido_total.loc[i,'광역시도']=val
  # if ('광역' in val) or ('특별' in val):
  #   df_sido_total.loc[i,'광역시도명']=val[:2]
  # else:
  
  df_sido_total.loc[i,'광역시도명']=val[:2]
  if ('북' in val) or ('남' in val):
    sum_val = val[0]+val[2]
    df_sido_total.loc[len(df_sido_total)]=[val,sum_val]

In [ ]:
df_sido_total

In [ ]:
df_sido_total.set_index('광역시도명').T.to_dict('list')

In [ ]:
df_sido_total['광역시도명'].values

*2. 지역(시군구) 생성 및 지역(시도) 보충*

In [ ]:
# 지역(시군구) 생성
def sigungu(df):
  name_gysd = df_sido_total['광역시도명'].values
  name_sgg = df_DSGG['시군구명'].values
  for i in range(len(df)):
    # 시군구인 경우엔 바로 시군구 값을 넣어준다
    if df.loc[i,'소관기관유형']=='시군구':
      sd = df.loc[i,'소관기관'].split(' ')[1]
      df.loc[i,'sigungu'] = sd
    # 공공기관의 경우에 해당하면
    elif df.loc[i,'소관기관유형']=='공공기관 등':
      # 소관기관의 이름과 앞의 두글자를 딴 변수를 구한다.
      val = df.loc[i,'소관기관']
      preval = df.loc[i,'소관기관'][:2]
      
      # 공공기관이면서 광역시 혹은 특별시가 들어간 경우
      if ('광역시' in val) or ('특별시' in val):
        # 00광역시로 시작하므로 시도값은 00광역시까지의 값
        df.loc[i,'sido'] = val[:5]
        # 00광역시00구 까지 써있는 경우엔 구값을 구해 시군구에 넣어준다.
        if '구' in val:
          num = val.find('구')
          SIGUN = val[5:num+1]
          df.loc[i,'sigungu'] = SIGUN
        # 00광역시만 있으면 구는 일단 NAN으로 처리
        else:
          df.loc[i,'sigungu'] = '전체'
      # 광주환경공단은 광역시가 아닌 광주광역시에서 주도하는 단체
      elif val == '광주환경공단':
        df.loc[i,'sido']='광주광역시'
        df.loc[i,'sigungu']='전체'
      # 앞의 두글자를 딴 변수 값이 위에서 만든 df_DSGG 시군구명에 들어간다면
      elif preval in name_sgg:
        # 해당 시군구명과 시도값을 넣어준다.
        val_df = df_DSGG[df_DSGG['시군구명']== preval]
        SIGUN = val_df.iloc[0,1]
        SIDO = val_df.iloc[0,0]
        df.loc[i,'sigungu']=SIGUN
        df.loc[i,'sido']=SIDO
      # 특정 광역시도가 들어간 곳 찾기
      elif preval in name_gysd:
        val_df = df_sido_total[df_sido_total['광역시도명']==preval]
        SIDO = val_df.iloc[0,0]
        df.loc[i,'sigungu']='전체'
        df.loc[i,'sido']=SIDO
      # 특정 광역시도가 들어간 곳 찾기
      elif preval in name_gysd:
        val_df = df_sido_total[df_sido_total['광역시도명']==preval]
        if len(val_df)>1:
          SIDO = val_df.iloc[0,0]
          df.loc[i,'sigungu']='전체'
          df.loc[i,'sido']=SIDO
        else:
          SIDO = val[:4]
          df.loc[i,'sigungu']='전체'
          df.loc[i,'sido']=SIDO
      # 그 외엔 시군구 NAN
      else:
        df.loc[i,'sigungu']='전체'
    else:
      df.loc[i,'sigungu']='전체'
  return df

In [ ]:
sigungu(df_policy)

In [ ]:
# 특정 지역에 해당하는 값을 대부분 제외했지만, 강원랜드와 경남개발공사, 전남개발공사 등 (주)가 포함되었거나 이름을 압축해서 부르는 경우가 있다.
df_policy[df_policy['sido']=='전국']['소관기관'].unique()

### B. 정리

1. 소관기관유형이 광역시도인 경우 소관기관은 바로 시도값으로
2. 소관기관 유형이 시군구인 경우, 앞의 값 넣기. (강원도 춘천시)
3. 소관기관 유형이 교육청인 경우, 교육청 제외한 앞 값 넣기. (강원도교육청)
4. 기타 전국

보완점: 서비스명에 들어가 있는 값 혹은 충남의 값 아직 미해결

### C. TAG CODE 전체

In [ ]:
# # 1. 도와 시군구 테이블 생성
# # 시군구에 대한 값을 추출해보았다.
# DSGG = df_policy[df_policy['소관기관유형']=='시군구']['소관기관'].unique()
# df_DSGG = pd.DataFrame(columns=['도','시군구'])

# # 소관기관유형 시군구인 소관기관에서 각 도와 시군구값 뺴기
# for i in range(len(DSGG)):
#   val = DSGG[i].split(' ')
#   df_DSGG.loc[i,'도'] = val[0]
#   df_DSGG.loc[i,'시군구'] = val[1]

# def namesigun(var):
#   return var[:2]

# df_DSGG['시군구명'] = df_DSGG['시군구'].apply(namesigun)

# UNIQ = df_policy[df_policy['소관기관유형']=='공공기관 등']['소관기관'].unique()

# # 2. 광역시도 관련된 광역시도명 테이블 만들기
# sido_total = df_policy[df_policy['소관기관유형']=='광역시도']['소관기관'].unique()
# df_sido_total = pd.DataFrame(columns=['광역시도','광역시도명'])

# for i in range(len(sido_total)):
#   val = sido_total[i]
#   df_sido_total.loc[i,'광역시도']=val
#   # if ('광역' in val) or ('특별' in val):
#   #   df_sido_total.loc[i,'광역시도명']=val[:2]
#   # else:
#   df_sido_total.loc[i,'광역시도명']=val[:2]
#   if ('북' in val) or ('남' in val):
#     sum_val = val[0]+val[2]
#     df_sido_total.loc[len(df_sido_total)]=[val,sum_val]

# # 지역(시군구) 생성
# def sigungu(df):
#   name_gysd = df_sido_total['광역시도명'].values
#   name_sgg = df_DSGG['시군구명'].values
#   for i in range(len(df)):
#     # 시군구인 경우엔 바로 시군구 값을 넣어준다
#     if df.loc[i,'소관기관유형']=='시군구':
#       sd = df.loc[i,'소관기관'].split(' ')[1]
#       df.loc[i,'sigungu'] = sd
#     # 공공기관의 경우에 해당하면
#     elif df.loc[i,'소관기관유형']=='공공기관 등':
#       # 소관기관의 이름과 앞의 두글자를 딴 변수를 구한다.
#       val = df.loc[i,'소관기관']
#       preval = df.loc[i,'소관기관'][:2]
      
#       # 공공기관이면서 광역시 혹은 특별시가 들어간 경우
#       if ('광역시' in val) or ('특별시' in val):
#         # 00광역시로 시작하므로 시도값은 00광역시까지의 값
#         df.loc[i,'sido'] = val[:5]
#         # 00광역시00구 까지 써있는 경우엔 구값을 구해 시군구에 넣어준다.
#         if '구' in val:
#           num = val.find('구')
#           SIGUN = val[5:num+1]
#           df.loc[i,'sigungu'] = SIGUN
#         # 00광역시만 있으면 구는 일단 NAN으로 처리
#         else:
#           df.loc[i,'sigungu'] = '전체'
#       # 광주환경공단은 광역시가 아닌 광주광역시에서 주도하는 단체
#       elif val == '광주환경공단':
#         df.loc[i,'sido']='광주광역시'
#         df.loc[i,'sigungu']='전체'
#       # 앞의 두글자를 딴 변수 값이 위에서 만든 df_DSGG 시군구명에 들어간다면
#       elif preval in name_sgg:
#         # 해당 시군구명과 시도값을 넣어준다.
#         val_df = df_DSGG[df_DSGG['시군구명']== preval]
#         SIGUN = val_df.iloc[0,1]
#         SIDO = val_df.iloc[0,0]
#         df.loc[i,'sigungu']=SIGUN
#         df.loc[i,'sido']=SIDO
#       # 특정 광역시도가 들어간 곳 찾기
#       elif preval in name_gysd:
#         val_df = df_sido_total[df_sido_total['광역시도명']==preval]
#         SIDO = val_df.iloc[0,0]
#         df.loc[i,'sigungu']='전체'
#         df.loc[i,'sido']=SIDO
#       # 특정 광역시도가 들어간 곳 찾기
#       elif preval in name_gysd:
#         val_df = df_sido_total[df_sido_total['광역시도명']==preval]
#         if len(val_df)>1:
#           SIDO = val_df.iloc[0,0]
#           df.loc[i,'sigungu']='전체'
#           df.loc[i,'sido']=SIDO
#         else:
#           SIDO = val[:4]
#           df.loc[i,'sigungu']='전체'
#           df.loc[i,'sido']=SIDO
#       # 그 외엔 시군구 NAN
#       else:
#         df.loc[i,'sigungu']='전체'
#     else:
#       df.loc[i,'sigungu']='전체'
#   return df

# sigungu(df_policy)

### D. 기타

* 기타 1: 광주환경공단은 광주시가 아님.

In [ ]:
# 지역(시군구) 변수 생성 시, 광주가 들어간 소관기관 중 광주 도시관리공사을 제외한 값들은 전라도 광주광역시이다.
# 따라서 공공기관 앞 두글자를 따서 시군구를 추출하는 과정에서 광주환경공단만 광주광역시로 빼줘야한다.
# 위 변수 생성 시 # 광주환경공단은 광역시가 아닌 광주광역시에서 주도하는 단체 부분 증거
abc = df_policy[df_policy['소관기관유형']=='공공기관 등']['소관기관'].unique()
[i for i in abc if '광주' in i]
# '광주' in abc.values()

* 기타 2: 광역시도인 경우, 시군구의 값 어드민데이터에서 확인

In [ ]:
# 어드민 데이터 광역시도인 값에 지역(시군구) 값 확인
# 각 8도와 광역시의 시군구 값, 다 추출 불가
GYSD_ID = df_policy[df_policy['소관기관유형']=='광역시도']['정책ID'].unique()
wello_compared[wello_compared['정책ID'].apply(lambda x: x in GYSD_ID)]['지역(시군구)'].unique()
t1229_2 = wello_compared[wello_compared['정책ID'].apply(lambda x: x in GYSD_ID)][['정책ID','지역(시도)','지역(시군구)']]
t1229_2
t1229_2['지역(시도)'].unique()

In [ ]:
# 강원도의 경우 시군구는 전체
t1229_2[t1229_2['지역(시도)']=='강원도'][['지역(시도)','지역(시군구)']].value_counts()

In [ ]:
# 경기도의 경우 시군구가 여러값인 걸 확인할 수 있다.
t1229_2[t1229_2['지역(시도)']=='경기도'][['지역(시도)','지역(시군구)']].value_counts()
# 결론: 광역시도로만 되어있는 정책은 모델링을 통해 보완해야 할 듯.

* 기타 3: 어드민 데이터와 시도 및 시군구 비교해보기

In [ ]:
# * 넣은 지역(시도)값과 어드민 데이터와 다른 데이터 값 찾기
SIDO_ID = df_policy[df_policy['sido'].notnull()]['정책ID'].values
SIDO_DF = df_policy[df_policy['정책ID'].apply(lambda x: x in SIDO_ID)]['sido']
SIDO_DF2 = wello_compared[wello_compared['정책ID'].apply(lambda x: x in SIDO_ID)]['지역(시도)']

In [ ]:
# NAN의 경우 전국으로 수정하거나 모델링할 떄 수정하면 될 듯 하다.
# 하지만 17645 인덱스 값은 내가 예측한 경상남도에 더 가까운 것 같지만 어드민데이터엔 왜 경기도라 되어있는지 의문이 남는다.
# 총 82개 다른 것을 확인할 수 있음.
# SIDO_DF: 내가 넣은 시도값
# SIDO_DF2: 어드민 데이터에 있는 시도값
SIDO_DF[(SIDO_DF != SIDO_DF2)&(SIDO_DF!='전국')]

In [ ]:
SIDO_DF2[(SIDO_DF != SIDO_DF2)&(SIDO_DF!='전국')]

In [ ]:
# * 전국으로 치부한 지역(시도)값과 어드민 데이터와 다른 데이터 값 찾기
# 넣은 지역(시도)값과 어드민 데이터와 다른 데이터 값 찾기
# 총 297개 다른 것을 확인할 수 있음.
# SIDO_DF: 내가 전국이라고 넣은 시도값
# SIDO_DF2: 어드민 데이터에 있는 시도값
SIDO_ID = df_policy[df_policy['sido']=='전국']['정책ID'].values
SIDO_DF = df_policy[df_policy['정책ID'].apply(lambda x : x in SIDO_ID)]['sido']
SIDO_DF2 = wello_compared[wello_compared['정책ID'].apply(lambda x : x in SIDO_ID)]['지역(시도)']

In [ ]:
# SIDO_DF
SIDO_DF[SIDO_DF != SIDO_DF2]

In [ ]:
SIDO_DF2[SIDO_DF != SIDO_DF2]

In [ ]:
# 서비스명을 통해 보완 가능할 듯.
print(df_policy.loc[81495]['서비스명'])
print(df_policy.loc[81496]['서비스명'])
print(df_policy.loc[81580]['서비스명'])
print(df_policy.loc[81818]['서비스명'])
print(df_policy.loc[81819]['서비스명'])
# 지원대상을 통해 보완 가능할 듯.
print(df_policy.loc[88296]['지원대상'])

* 기타 4: 넣은 지역(시군구)값과 어드민 데이터와 다른 데이터 값 찾기

In [ ]:

SIGUNGU_ID = df_policy[df_policy['sigungu'].notnull()]['정책ID'].values
SIGUNGU_DF = df_policy[df_policy['정책ID'].apply(lambda x: x in SIGUNGU_ID)]['sigungu']
SIGUNGU_DF2 = wello_compared[wello_compared['정책ID'].apply(lambda x: x in SIGUNGU_ID)]['지역(시군구)']

In [ ]:
# 5개의 값만 다른 것으로 보인다.
# NAN의 경우 전국으로 수정하거나 모델링할 떄 수정하면 될 듯 하다.
# 하지만 17645 인덱스 값은 내가 예측한 경상남도에 더 가까운 것 같지만 어드민데이터엔 왜 경기도라 되어있는지 의문이 남는다.
SIGUNGU_DF[(SIGUNGU_DF != SIGUNGU_DF2)&(SIGUNGU_DF!='전체')]

In [ ]:
SIGUNGU_DF2[(SIGUNGU_DF != SIGUNGU_DF2)&(SIGUNGU_DF!='전체')]
# 총 잘못 분류된 건수 7233 but 지역(시도)에서부터 잘못 되었을 가능성 0
# 어드민 데이터에 양주시로 분류되어있는 건수가 총 326건. => 남양주가 잘못 분류된 것으로 추정

In [ ]:
# 위 주장에 대한 근거
df_policy.loc[8039]

In [ ]:
# 위 주장에 대한 근거2
SIGUNGU_DF2[(SIGUNGU_DF != SIGUNGU_DF2)&(SIGUNGU_DF2=='양주시')]

In [ ]:
# * '전체' 값으로 넣은 지역(시군구)값과 어드민 데이터와 다른 데이터 값 찾기
SIGUNGU_ID = df_policy[df_policy['sigungu']=='전체']['정책ID'].values
SIGUNGU_DF = df_policy[df_policy['정책ID'].apply(lambda x: x in SIGUNGU_ID)]['sigungu'].copy()
SIGUNGU_DF2 = wello_compared[wello_compared['정책ID'].apply(lambda x: x in SIGUNGU_ID)]['지역(시군구)'].copy()

In [ ]:
# 전체로 넣은 값
SIGUNGU_DF[SIGUNGU_DF.isnull() != SIGUNGU_DF2.isnull()]

In [ ]:
wello_compared[wello_compared['지역(시도)']=='전국']['지역(시군구)'].unique()

In [ ]:
check = wello_compared[wello_compared['지역(시군구)'].isnull()]['정책ID'].values
check_df = wello_compared[wello_compared['정책ID'].apply(lambda x: x in check)]
check_df[check_df['지역(시도)']!='전국']

In [ ]:
# 어드민 데이터의 경우 결측값으로 들어간 데이터가 전부..
SIGUNGU_DF2[SIGUNGU_DF.isnull() != SIGUNGU_DF2.isnull()].unique()

In [ ]:
JA_ID=df_policy[df_policy['소관기관유형']=='중앙행정기관']['정책ID'].unique()
wello_compared[wello_compared['정책ID'].apply(lambda x: x in JA_ID)]['지역(시도)'].unique()

In [ ]:
# 중앙행정기관인 경우의 지역(시도)를 확인한 결과 전국이 아닌 곳은 서비스명에서 추출가능해보인다.
# t1229_3=wello_compared[wello_compared['정책ID'].apply(lambda x: x in JA_ID)].copy()
# t1229_3[t1229_3['지역(시도)']!='전국']

In [ ]:
df_policy.loc[17645,'지원대상']

In [ ]:
# df_policy.loc[1070]

In [ ]:
# df_policy.loc[1070]['소관기관']

## **3. 신청절차 (이후 희택님거)**

In [ ]:
t1231 = df_policy.copy()

In [ ]:
t1231.shape

In [ ]:
t1231['신청절차']=t1231['신청절차'].astype(str)

In [ ]:
t1231.dtypes

In [ ]:
# # for i in range(len(df_policy)):
#   if ('인터넷' in df_policy.loc[i,'신청절차'])|('온라인' in df_policy.loc[i,'신청절차'])|('사이트' in df_policy.loc[i,'신청절차']):
#     df_policy.loc[i,'온오프']='온라인'
#   elif ('유선' in df_policy.loc[i,'신청절차'])|('사무소' in df_policy.loc[i,'신청절차'])|('센터' in df_policy.loc[i,'신청절차'])|('방문' in df_policy.loc[i,'신청절차']):
#     df_policy.loc[i,'온오프']='오프라인'
  

In [ ]:
# t1231['온오프'].unique()

In [ ]:
# t1231[t1231['온오프'].isnull()].shape

## **4. 지원유형**

### A. TAG CODE

In [ ]:
# \n과 ||와 ,,를 없애주는 함수
def clean_provision_type(var):
  var = var.replace('\n',',').replace('||',',').replace(',,',',')
  return var

In [ ]:
# 지원유형 dtype이 object인데 중간에 Float이 있어 안 돌아갈 수 있으니 str 변환
df_policy['지원유형']=df_policy['지원유형'].astype(str)

In [ ]:
df_policy['지원유형']=df_policy['지원유형'].apply(clean_provision_type)

In [ ]:
df_policy['지원유형']

In [ ]:
# 확인해보면 어드민데이터 유형이랑 개수 차이가 크지 않아보임
print(len(wello_compared['지원유형'].unique()))
print(len(df_policy['지원유형'].apply(clean_provision_type).unique()))

### B. 정리
보완점 - 지원유형을 단순화할 방법이 필요해보임. 예시) 88485행의 문화/여가지원,감면/면제 의 경우 문화/여가지원으로만 단순화시켜도 될 듯. 혹은 감면/면제를 현금으로 바꿔도 무방할 듯.

### C. TAG CODE 전체

In [ ]:
# # \n과 ||와 ,,를 없애주는 함수
# def clean_provision_type(var):
#   var = var.replace('\n',',').replace('||',',').replace(',,',',')
#   return var
  
# # 지원유형 dtype이 object인데 중간에 Float이 있어 안 돌아갈 수 있으니 str 변환
# df_policy['지원유형']=df_policy['지원유형'].astype(str)
# df_policy['지원유형']=df_policy['지원유형'].apply(clean_provision_type)

## **5. 대상특성**

In [ ]:
# 어떻게 손질해야할지 감이 안 옴.
df_policy['태그-대상특성'].unique()

## **# 지원대상 전처리**

In [ ]:
# 많이 언급되는 지원대상, ○, :, \n, | 등
# 특히 중위 소득=> 중위소득으로 바꿔줘야함.
def clean_support_target(var):
  var = var.lstrip()
  var = var.replace('지원대상','').replace('○','').replace(':','').replace('\n','').replace('O','').replace('|','').replace('  ',' ').replace(' 소득','소득').replace('\xa0','').replace('*','')
  return var
df_policy['지원대상']=df_policy['지원대상'].apply(clean_support_target)

## **6. 중위소득 - START_ICM & END_ICM**

### A. TAG CODE

In [ ]:
# 중위소득 수준 알아보는 반복문
for i in range(len(df_policy)):
  if '중위소득' in df_policy.loc[i,'지원대상']:
    pre_var = df_policy.loc[i,'지원대상'].find('중위')
    if '이하' in df_policy.loc[i,'지원대상'][pre_var:]:
      post_var =df_policy.loc[i,'지원대상'][pre_var:].find('이하')+pre_var+2
      df_policy.loc[i,'MED_ICM'] = df_policy.loc[i,'지원대상'][pre_var:post_var]
    elif '미만' in df_policy.loc[i,'지원대상'][pre_var:]:
      post_var =df_policy.loc[i,'지원대상'][pre_var:].find('미만')+pre_var+2
      df_policy.loc[i,'MED_ICM'] = df_policy.loc[i,'지원대상'][pre_var:post_var]
    elif '이상' in df_policy.loc[i,'지원대상'][pre_var:]:
      post_var =df_policy.loc[i,'지원대상'][pre_var:].find('이상')+pre_var+2
      df_policy.loc[i,'MED_ICM'] = df_policy.loc[i,'지원대상'][pre_var:post_var]
    
    elif '%' in df_policy.loc[i,'지원대상'][pre_var:]:
      post_var =df_policy.loc[i,'지원대상'][pre_var:].find('%')+pre_var
      df_policy.loc[i,'MED_ICM'] = df_policy.loc[i,'지원대상'][pre_var:post_var]
    elif '퍼센트' in df_policy.loc[i,'지원대상'][pre_var:]:
      post_var =df_policy.loc[i,'지원대상'][pre_var:].find('퍼센트')+pre_var
      df_policy.loc[i,'MED_ICM'] = df_policy.loc[i,'지원대상'][pre_var:post_var]
    elif '이하' in df_policy.loc[i,'지원대상'][pre_var:]:
      post_var =df_policy.loc[i,'지원대상'][pre_var:].find('이하')+pre_var
      df_policy.loc[i,'MED_ICM'] = df_policy.loc[i,'지원대상'][pre_var:post_var]
  else:
    df_policy.loc[i,'MED_ICM'] = np.NAN

In [ ]:
# 많이 언급되는 지원대상, ○, :, \n, | 등
# 특히 중위 소득=> 중위소득으로 바꿔줘야함.
def clean_selection_criteria(var):
  var = var.lstrip()
  var = var.replace('기준','').replace('○','').replace(':','').replace('\n','').replace('O','').replace('|','').replace('  ',' ').replace(' 소득','소득').replace('\xa0','').replace('*','')
  return var
df_policy['선정기준']=df_policy['선정기준'].astype(str).apply(clean_selection_criteria)

# 중위소득 수준 알아보는 반복문
for i in range(len(df_policy)):
  if df_policy.loc[i,'MED_ICM']=='nan':
    if '중위소득' in df_policy.loc[i,'선정기준']:
      pre_var = df_policy.loc[i,'선정기준'].find('중위')
      if '이하' in df_policy.loc[i,'선정기준'][pre_var:]:
        post_var =df_policy.loc[i,'선정기준'][pre_var:].find('이하')+pre_var+2
        df_policy.loc[i,'MED_ICM'] = df_policy.loc[i,'선정기준'][pre_var:post_var]
      elif '미만' in df_policy.loc[i,'선정기준'][pre_var:]:
        post_var =df_policy.loc[i,'선정기준'][pre_var:].find('미만')+pre_var+2
        df_policy.loc[i,'MED_ICM'] = df_policy.loc[i,'선정기준'][pre_var:post_var]
      elif '이상' in df_policy.loc[i,'선정기준'][pre_var:]:
        post_var =df_policy.loc[i,'선정기준'][pre_var:].find('이상')+pre_var+2
        df_policy.loc[i,'MED_ICM'] = df_policy.loc[i,'선정기준'][pre_var:post_var]
      
      elif '%' in df_policy.loc[i,'선정기준'][pre_var:]:
        post_var =df_policy.loc[i,'선정기준'][pre_var:].find('%')+pre_var
        df_policy.loc[i,'MED_ICM'] = df_policy.loc[i,'선정기준'][pre_var:post_var]
      elif '퍼센트' in df_policy.loc[i,'선정기준'][pre_var:]:
        post_var =df_policy.loc[i,'선정기준'][pre_var:].find('퍼센트')+pre_var
        df_policy.loc[i,'MED_ICM'] = df_policy.loc[i,'선정기준'][pre_var:post_var]
      elif '이하' in df_policy.loc[i,'선정기준'][pre_var:]:
        post_var =df_policy.loc[i,'선정기준'][pre_var:].find('이하')+pre_var
        df_policy.loc[i,'MED_ICM'] = df_policy.loc[i,'선정기준'][pre_var:post_var]
  else:
    pass

In [ ]:
# 중위소득 값 다듬어주기
def clean_MED_INC(var):
  var = var.replace(',','').replace('100분의','').replace('3분의 2','66%').replace('2/3','66%')
  return var
df_policy['MED_ICM'] = df_policy['MED_ICM'].astype(str).apply(clean_MED_INC)

In [ ]:
# 들어간 중위소득 값 확인
df_policy['MED_ICM'].unique()

In [ ]:
# 문제 있을 때 지우고 다시 시작.
# del df_policy['START_MED_ICM']
# del df_policy['END_MED_ICM']

In [ ]:
# 중위소득 시작과 끝값 넣기
# START_MED_ICM : 중위소득 시작 변수
# END_MED_ICM : 중위소득 끝 변수

for i in range(len(df_policy)):
  # 중위소득 없는 값은 pass
  if df_policy.loc[i,'MED_ICM']=='nan':
    pass
  # 중위소득 있는 값만
  else:
    # 중위소득 변수에서 숫자값만 리스트로 뽑기
    val = re.findall(r"\d+",df_policy.loc[i,'MED_ICM'])
    # 숫자값으로 인식
    val_list = list(map(int,val))
    num = []
    # 10 이상이거나 250 이하의 중위소득에 해당하는 값만 추출
    for j in val_list:
      if (j>=10) and (j<=200):
        num.append(j)
    # 숫자가 2개 이상인 리스트에서
    if len(num)>=2:
      # 두번째 숫자가 첫번째 숫자보다 크면 (ex. 중위소득 40~50 해당)
      if num[1]>num[0]:
        df_policy.loc[i,'START_MED_ICM']=num[0]
        df_policy.loc[i,'END_MED_ICM']=num[1]
      # 마지막 숫자가 첫번째 숫자보다 크면 (ex. 중위소득 40% (1인가구 얼마 2인가구 얼마..) 50% 미만)
      elif num[len(num)-1] > num[0]:
        df_policy.loc[i,'START_MED_ICM']=num[0]
        df_policy.loc[i,'END_MED_ICM']=num[len(num)-1]
      # 그 외
      else:
        df_policy.loc[i,'START_MED_ICM']=0
        df_policy.loc[i,'END_MED_ICM']=num[0]
    # 숫자가 한개인 경우
    elif len(num)==1:
      # 이상이라는 글자가 들어있다면 그 숫자가 시작
      if '이상' in val:
        df_policy.loc[i,'START_MED_ICM']=num[0]
        df_policy.loc[i,'END_MED_ICM']='inf'
      # 없으면 이하나 미만으로 판단
      else:
        df_policy.loc[i,'START_MED_ICM']=0
        df_policy.loc[i,'END_MED_ICM']=num[0]

    # 그 외엔 전부로 취합
    else:
      df_policy.loc[i,'START_MED_ICM']=0
      df_policy.loc[i,'END_MED_ICM']='inf'

In [ ]:
# 중위소득 시작값 유니크
df_policy['START_MED_ICM'].unique()

In [ ]:
# 중위소득 종료값 유니크
df_policy['END_MED_ICM'].unique()

In [ ]:
if df_policy.loc[84625,'MED_ICM']=='nan':
  if '중위소득' in df_policy.loc[84625,'선정기준']:
    pre_var = df_policy.loc[84625,'선정기준'].find('중위')
    if '이하' in df_policy.loc[84625,'선정기준'][pre_var:]:
      post_var =df_policy.loc[84625,'선정기준'][pre_var:].find('이하')+pre_var+2
      df_policy.loc[84625,'MED_ICM'] = df_policy.loc[84625,'선정기준'][pre_var:post_var]
    elif '미만' in df_policy.loc[84625,'선정기준'][pre_var:]:
      post_var =df_policy.loc[84625,'선정기준'][pre_var:].find('미만')+pre_var+2
      df_policy.loc[84625,'MED_ICM'] = df_policy.loc[84625,'선정기준'][pre_var:post_var]
    elif '이상' in df_policy.loc[84625,'선정기준'][pre_var:]:
      post_var =df_policy.loc[84625,'선정기준'][pre_var:].find('이상')+pre_var+2
      df_policy.loc[84625,'MED_ICM'] = df_policy.loc[84625,'선정기준'][pre_var:post_var]
    
    elif '%' in df_policy.loc[84625,'선정기준'][pre_var:]:
      post_var =df_policy.loc[84625,'선정기준'][pre_var:].find('%')+pre_var
      df_policy.loc[84625,'MED_ICM'] = df_policy.loc[84625,'선정기준'][pre_var:post_var]
    elif '퍼센트' in df_policy.loc[84625,'선정기준'][pre_var:]:
      post_var =df_policy.loc[84625,'선정기준'][pre_var:].find('퍼센트')+pre_var
      df_policy.loc[84625,'MED_ICM'] = df_policy.loc[84625,'선정기준'][pre_var:post_var]

else:
  pass


### B. 정리
1. '중위소득' in '지원대상'
  * '이하' or '미만' in 중위소득 이후 글자
  * '이상' or '초과' in 중위소득 이후 글자 
  * '%' Or '퍼센트' in 중위소득 이후 글자 
2. re.findall로 숫자 빼내기 
  * 중위소득 범위인 10에서 200 사이의 값만 추출 (범위 선정 이유: 중위소득 계층은 50% 초과 150% 이하 But 200 최대값 존재) 
  * 값이 하나만 나온 경우
    * ‘이상’이 포함되었다면 값에서 inf까지의 범위
    * 보통은 이하 및 미만이므로 0에서 값까지의 범위
  * 값이 두개 이상 나온 경우 - 범위인 경우 (ex. 중위소득 40~60%)

* 보완점: 값이 잘 들어갔는지 재차 확인해야할 듯. 정확도 아주 높을거라 예상

### C. TAG CODE 전체

In [ ]:
# # 많이 언급되는 지원대상, ○, :, \n, | 등
# # 특히 중위 소득=> 중위소득으로 바꿔줘야함.
# def clean_support_target(var):
#   var = var.lstrip()
#   var = var.replace('지원대상','').replace('○','').replace(':','').replace('\n','').replace('O','').replace('|','').replace('  ',' ').replace(' 소득','소득').replace('\xa0','').replace('*','')
#   return var
# df_policy['지원대상']=df_policy['지원대상'].apply(clean_support_target)

# # 중위소득 수준 알아보는 반복문
# for i in range(len(df_policy)):
#   if '중위소득' in df_policy.loc[i,'지원대상']:
#     pre_var = df_policy.loc[i,'지원대상'].find('중위')
#     if '이하' in df_policy.loc[i,'지원대상'][pre_var:]:
#       post_var =df_policy.loc[i,'지원대상'][pre_var:].find('이하')+pre_var+2
#       df_policy.loc[i,'MED_ICM'] = df_policy.loc[i,'지원대상'][pre_var:post_var]
#     elif '미만' in df_policy.loc[i,'지원대상'][pre_var:]:
#       post_var =df_policy.loc[i,'지원대상'][pre_var:].find('미만')+pre_var+2
#       df_policy.loc[i,'MED_ICM'] = df_policy.loc[i,'지원대상'][pre_var:post_var]
#     elif '이상' in df_policy.loc[i,'지원대상'][pre_var:]:
#       post_var =df_policy.loc[i,'지원대상'][pre_var:].find('이상')+pre_var+2
#       df_policy.loc[i,'MED_ICM'] = df_policy.loc[i,'지원대상'][pre_var:post_var]
    
#     elif '%' in df_policy.loc[i,'지원대상'][pre_var:]:
#       post_var =df_policy.loc[i,'지원대상'][pre_var:].find('%')+pre_var
#       df_policy.loc[i,'MED_ICM'] = df_policy.loc[i,'지원대상'][pre_var:post_var]
#     elif '퍼센트' in df_policy.loc[i,'지원대상'][pre_var:]:
#       post_var =df_policy.loc[i,'지원대상'][pre_var:].find('퍼센트')+pre_var
#       df_policy.loc[i,'MED_ICM'] = df_policy.loc[i,'지원대상'][pre_var:post_var]

#   else:
#     df_policy.loc[i,'MED_ICM'] = np.NAN
# # 많이 언급되는 선정기준, ○, :, \n, | 등
# # 특히 중위 소득=> 중위소득으로 바꿔줘야함.
# def clean_selection_criteria(var):
#   var = var.lstrip()
#   var = var.replace('기준','').replace('○','').replace(':','').replace('\n','').replace('O','').replace('|','').replace('  ',' ').replace(' 소득','소득').replace('\xa0','').replace('*','')
#   return var
# df_policy['선정기준']=df_policy['선정기준'].astype(str).apply(clean_selection_criteria)



In [ ]:
# # 위랑 같이 하면 안 돌아가는 코드가 있어서 따로 분리
# # 중위소득 수준 알아보는 반복문
# for i in range(len(df_policy)):
#   if df_policy.loc[i,'MED_ICM']=='nan':
#     if '중위소득' in df_policy.loc[i,'선정기준']:
#       pre_var = df_policy.loc[i,'선정기준'].find('중위')
#       if '이하' in df_policy.loc[i,'선정기준'][pre_var:]:
#         post_var =df_policy.loc[i,'선정기준'][pre_var:].find('이하')+pre_var+2
#         df_policy.loc[i,'MED_ICM'] = df_policy.loc[i,'선정기준'][pre_var:post_var]
#       elif '미만' in df_policy.loc[i,'선정기준'][pre_var:]:
#         post_var =df_policy.loc[i,'선정기준'][pre_var:].find('미만')+pre_var+2
#         df_policy.loc[i,'MED_ICM'] = df_policy.loc[i,'선정기준'][pre_var:post_var]
#       elif '이상' in df_policy.loc[i,'선정기준'][pre_var:]:
#         post_var =df_policy.loc[i,'선정기준'][pre_var:].find('이상')+pre_var+2
#         df_policy.loc[i,'MED_ICM'] = df_policy.loc[i,'선정기준'][pre_var:post_var]
      
#       elif '%' in df_policy.loc[i,'선정기준'][pre_var:]:
#         post_var =df_policy.loc[i,'선정기준'][pre_var:].find('%')+pre_var
#         df_policy.loc[i,'MED_ICM'] = df_policy.loc[i,'선정기준'][pre_var:post_var]
#       elif '퍼센트' in df_policy.loc[i,'선정기준'][pre_var:]:
#         post_var =df_policy.loc[i,'선정기준'][pre_var:].find('퍼센트')+pre_var
#         df_policy.loc[i,'MED_ICM'] = df_policy.loc[i,'선정기준'][pre_var:post_var]


# # 중위소득 값 다듬어주기
# def clean_MED_INC(var):
#   var = var.replace(',','').replace('100분의','').replace('3분의 2','66%')
#   return var
# df_policy['MED_ICM'] = df_policy['MED_ICM'].astype(str).apply(clean_MED_INC)
# # 중위소득 시작과 끝값 넣기
# # START_MED_ICM : 중위소득 시작 변수
# # END_MED_ICM : 중위소득 끝 변수

# for i in range(len(df_policy)):
#   # 중위소득 없는 값은 pass
#   if df_policy.loc[i,'MED_ICM']=='nan':
#     pass
#   # 중위소득 있는 값만
#   else:
#     # 중위소득 변수에서 숫자값만 리스트로 뽑기
#     val = re.findall(r"\d+",df_policy.loc[i,'MED_ICM'])
#     # 숫자값으로 인식
#     val_list = list(map(int,val))
#     num = []
#     # 10 이상이거나 250 이하의 중위소득에 해당하는 값만 추출
#     for j in val_list:
#       if (j>=10) and (j<=200):
#         num.append(j)
#     # 숫자가 2개 이상인 리스트에서
#     if len(num)>=2:
#       # 두번째 숫자가 첫번째 숫자보다 크면 (ex. 중위소득 40~50 해당)
#       if num[1]>num[0]:
#         df_policy.loc[i,'START_MED_ICM']=num[0]
#         df_policy.loc[i,'END_MED_ICM']=num[1]
#       # 마지막 숫자가 첫번째 숫자보다 크면 (ex. 중위소득 40% (1인가구 얼마 2인가구 얼마..) 50% 미만)
#       elif num[len(num)-1] > num[0]:
#         df_policy.loc[i,'START_MED_ICM']=num[0]
#         df_policy.loc[i,'END_MED_ICM']=num[len(num)-1]
#       # 그 외
#       else:
#         df_policy.loc[i,'START_MED_ICM']=0
#         df_policy.loc[i,'END_MED_ICM']=num[0]
#     # 숫자가 한개인 경우
#     elif len(num)==1:
#       # 이상이라는 글자가 들어있다면 그 숫자가 시작
#       if '이상' in val:
#         df_policy.loc[i,'START_MED_ICM']=num[0]
#         df_policy.loc[i,'END_MED_ICM']='inf'
#       # 없으면 이하나 미만으로 판단
#       else:
#         df_policy.loc[i,'START_MED_ICM']=0
#         df_policy.loc[i,'END_MED_ICM']=num[0]

#     # 그 외엔 전부로 취합
#     else:
#       df_policy.loc[i,'START_MED_ICM']=0
#       df_policy.loc[i,'END_MED_ICM']='inf'

### D. 기타

*0. 원리*

In [ ]:
# 이와 같은 규칙으로 값 뺴내기
# 예시 중위소득 몇퍼 ~~~ 등 숫자값 모두 빼내기
abc = re.findall(r"\d+",'중위소득 30% · 1인 가구 501632원 · 2인 가구 854129원 · 3인 가구 1104945원 · 4인 가구 1355761원 · 5인 가구 1606576원 생계급여액은 생계급여 선정 기준(급여기준)에서 가구의소득 인정액을 차감한 금액 -소득 인정액 =소득평가액 + 재산의소득환산액  ·소득평가액 = 실제소득 - 가구 특성별 지출비용 - 근로소득공제 · 재산의소득환산액 = {(일반/금융 재산의 종류별 가액-기본재산액-부채)+승용차 재산가액} X 재산의 종류별소득환산율 부양의무자 기준 -부양의무자 범위 수급권자의 1촌 직계혈족 및 그 배우자(배우자가 사망한 사위·며느리·계부·계모는 수급권자의 부양의무자가 아님) -부양의무자 적용 기준 · 부양의무자가 없는 경우 · 부양의무자가 있어도 부양능력이 없는 경우  · 부양의무자가 부양능력이 미약한 경우로서 수급권자에 대한 부양비 지원을 전제로 부양능력이 없는 것으로 인정하는 경우 · 부양능력 있는 부양의무자가 있어도 부양을 받을 수 없는 경우 부양능력 판정의 기본 원칙 -부양능력 없음(소득기준과 재산 기준 동시 충족) ·소득기준 부양의무자의 부양능력 판정소득액이 부양의무자 가구 기준 중위소득의 100% 미만',
       )
# 숫자만 뽑았을 때 값을 Int로 변경
abc = list(map(int,abc))
num = []
print(abc)
# 10 < i < 250에 중위소득 값에 해당하는 (중위소득 값의 최대범위) 숫자값만 추출
for i in abc:
  if (i>=10) and (i<=250):
    num.append(i)
num

*1. 값이 잘 들어갔는지 확인*

In [ ]:
# 확인
df_policy[df_policy['START_MED_ICM']==0]

In [ ]:
# 확인
df_policy[df_policy['END_MED_ICM']==60]

## **7. 결혼 - MARRIAGE**

### A. TAG CODE 

In [ ]:
# 지원대상에 이혼, 한부모, 조손가족이 들어갔을 때 이혼으로 취급
for i in range(len(df_policy)):
  if ('이혼' in df_policy.loc[i,'지원대상'])|('한부모' in df_policy.loc[i,'지원대상'])|('조손가족' in df_policy.loc[i,'지원대상']):
      df_policy.loc[i,'DIVORCE']= 1
  else:
    df_policy.loc[i,'DIVORCE']= 0


In [ ]:
# 지원대상에 미혼이 들어갔을 때 미혼으로 취급
for i in range(len(df_policy)):
  if ('미혼' in df_policy.loc[i,'지원대상']):
    df_policy.loc[i,'SINGLE']= 1
  else:
    df_policy.loc[i,'SINGLE']= 0

In [ ]:
# 지원대상에 임산부, 아동, 보육, 양육,자녀, 출산, 임신, 기혼일 때 기혼으로 취급
for i in range(len(df_policy)):
  if ('임산부' in df_policy.loc[i,'지원대상'])|('아동' in df_policy.loc[i,'지원대상'])|('보육' in df_policy.loc[i,'지원대상'])|('양육' in df_policy.loc[i,'지원대상'])|('자녀' in df_policy.loc[i,'지원대상'])|('출산' in df_policy.loc[i,'지원대상'])|('임신' in df_policy.loc[i,'지원대상'])|('기혼' in df_policy.loc[i,'지원대상']):
    df_policy.loc[i,'MARRIED']= 1
  else:
    df_policy.loc[i,'MARRIED']= 0

In [ ]:
# 결혼 값 만들기
df_policy['MARRIAGE']=''

for i in range(len(df_policy)):
  values = df_policy.loc[i,'DIVORCE':'MARRIED'].values
  if (values[0]==0)&(values[1]==0)&(values[2]==0):
    df_policy.loc[i,'MARRIAGE']= '무관'
  else:
    for j in range(len(values)):
      if j == 0:
        if values[j]==1:
          df_policy.loc[i,'MARRIAGE']+='이혼'
      elif j == 1:
        if values[j]==1:
          if df_policy.loc[i,'MARRIAGE']=='':
            df_policy.loc[i,'MARRIAGE']+='미혼'
          else:
            df_policy.loc[i,'MARRIAGE']+=', 미혼'
      elif j == 2:
        if values[j]==1:
          if df_policy.loc[i,'MARRIAGE']=='':
            df_policy.loc[i,'MARRIAGE']+='기혼'
          else:
            df_policy.loc[i,'MARRIAGE']+=', 기혼'

In [ ]:
del df_policy['DIVORCE']
del df_policy['SINGLE']
del df_policy['MARRIED']

### B. 정리 ###
1. 지원대상에 이혼, 한부모, 조손가족이 들어갔을 때 이혼으로 취급
2. 지원대상에 미혼이 들어갔을 때 미혼으로 취급
3. 지원대상에 임산부, 아동, 보육, 양육,자녀, 출산, 임신, 기혼일 때 기혼으로 취급
4. 결혼 값 만들기

=> str으로 in 태그 매칭할 수 있게 해놓음. 이혼기혼의 경우, 이혼한 사람과 기혼인 사람 모두 해당되므로 이혼기혼으로 그냥 둠..
보완점: 무관이 정말 무관인가에 대해 모델링으로 여지를..

### C. TAG CODE 전체

In [ ]:
# # 지원대상에 이혼, 한부모, 조손가족이 들어갔을 때 이혼으로 취급
# for i in range(len(df_policy)):
#   if ('이혼' in df_policy.loc[i,'지원대상'])|('한부모' in df_policy.loc[i,'지원대상'])|('조손가족' in df_policy.loc[i,'지원대상']):
#       df_policy.loc[i,'DIVORCE']= 1
#   else:
#     df_policy.loc[i,'DIVORCE']= 0

# # 지원대상에 미혼이 들어갔을 때 미혼으로 취급
# for i in range(len(df_policy)):
#   if ('미혼' in df_policy.loc[i,'지원대상']):
#     df_policy.loc[i,'SINGLE']= 1
#   else:
#     df_policy.loc[i,'SINGLE']= 0

# # 지원대상에 임산부, 아동, 보육, 양육,자녀, 출산, 임신, 기혼일 때 기혼으로 취급
# for i in range(len(df_policy)):
#   if ('임산부' in df_policy.loc[i,'지원대상'])|('아동' in df_policy.loc[i,'지원대상'])|('보육' in df_policy.loc[i,'지원대상'])|('양육' in df_policy.loc[i,'지원대상'])|('자녀' in df_policy.loc[i,'지원대상'])|('출산' in df_policy.loc[i,'지원대상'])|('임신' in df_policy.loc[i,'지원대상'])|('기혼' in df_policy.loc[i,'지원대상']):
#     df_policy.loc[i,'MARRIED']= 1
#   else:
#     df_policy.loc[i,'MARRIED']= 0
    
# # 결혼 값 만들기
# df_policy['MARRIAGE']=''

# for i in range(len(df_policy)):
#   values = df_policy.loc[i,'DIVORCE':'MARRIED'].values
#   if (values[0]==0)&(values[1]==0)&(values[2]==0):
#     df_policy.loc[i,'MARRIAGE']= '무관'
#   else:
#     for j in range(len(values)):
#       if j == 0:
#         if values[j]==1:
#           df_policy.loc[i,'MARRIAGE']+='이혼'
#       elif j == 1:
#         if values[j]==1:
#           if df_policy.loc[i,'MARRIAGE']=='':
#             df_policy.loc[i,'MARRIAGE']+='미혼'
#           else:
#             df_policy.loc[i,'MARRIAGE']+=', 미혼'
#       elif j == 2:
#         if values[j]==1:
#           if df_policy.loc[i,'MARRIAGE']=='':
#             df_policy.loc[i,'MARRIAGE']+='기혼'
#           else:
#             df_policy.loc[i,'MARRIAGE']+=', 기혼'
# del df_policy['DIVORCE']
# del df_policy['SINGLE']
# del df_policy['MARRIED']

## **8. 학력 - EDUCATION**

### A. TAG CODE 

In [ ]:
# 고졸/대재/대졸/무관

In [ ]:
df_policy.columns

In [ ]:
# 지원대상에 고졸, 고등학교, 고등학교 졸업, 마이스터고 > 고졸로 취급
for i in range(len(df_policy)):
  if ('고졸' in df_policy.loc[i,'지원대상'])|('고등학교' in df_policy.loc[i,'지원대상'])&('졸업' in df_policy.loc[i,'지원대상'])|('마이스터고' in df_policy.loc[i,'지원대상']):
      df_policy.loc[i,'GRAD_HIGH']= 1
  else:
    df_policy.loc[i,'GRAD_HIGH']= 0


In [ ]:
# 지원대상에 전문대, 대학&재학, 대학생, 대학 재학 > 대학으로 취급
for i in range(len(df_policy)):
  if ('전문대' in df_policy.loc[i,'지원대상'])|('대학' in df_policy.loc[i,'지원대상'])&('재학' in df_policy.loc[i,'지원대상'])|('대학생' in df_policy.loc[i,'지원대상']):
    df_policy.loc[i,'ENROLL_UNIV']= 1
  else:
    df_policy.loc[i,'ENROLL_UNIV']= 0

In [ ]:
# 지원대상에 대학&졸업이 같이 들어있을 때 대학 졸업으로 취급
for i in range(len(df_policy)):
  if ('대학' in df_policy.loc[i,'지원대상'])&('졸업' in df_policy.loc[i,'지원대상']):
    df_policy.loc[i,'GRAD_UNIV']= 1
  else:
    df_policy.loc[i,'GRAD_UNIV']= 0

In [ ]:
# 학력 값 만들기
df_policy['EDUCATION']=''

for i in range(len(df_policy)):
  values = df_policy.loc[i,'GRAD_HIGH':'GRAD_UNIV'].values
  if (values[0]==0)&(values[1]==0)&(values[2]==0):
    df_policy.loc[i,'EDUCATION']= '무관'
  else:
    for j in range(len(values)):
      if j == 0:
        if values[j]==1:
          df_policy.loc[i,'EDUCATION']+='고등학교 졸업'
      elif j == 1:
        if values[j]==1:
          if df_policy.loc[i,'EDUCATION']=='':
            df_policy.loc[i,'EDUCATION']+='대학(원) 재학'
          else:
            df_policy.loc[i,'EDUCATION']+=', 대학(원) 재학'
  
      elif j == 2:
        if values[j]==1:
          if df_policy.loc[i,'EDUCATION']=='':
            df_policy.loc[i,'EDUCATION']+='대학(원) 졸업'
          else:
            df_policy.loc[i,'EDUCATION']+=', 대학(원) 졸업'

### B. 정리 ###
1. 지원대상에 고졸, 고등학교&졸업, 고등학생, 마이스터고이 들어갔을 때 고등학교 졸업으로 취급
2. 지원대상에 전문대, 대학&재학, 대학생, 대학 재학이 들어갔을 때 대학 재학으로 취급
3. 지원대상에 대학&졸업이 같이 들어있을 때 대학 졸업으로 취급
4. 셋에도 해당 안되면 무관으로 취급

=> str으로 취급, 고졸/대학재학/대졸 세개의 경우의 수를 따져서 총 8개의 조합이 나왔다.

### C. TAG CODE 전체

In [ ]:
# # 지원대상에 고졸, 고등학교, 고등학교 졸업, 마이스터고 > 고졸로 취급
# for i in range(len(df_policy)):
#   if ('고졸' in df_policy.loc[i,'지원대상'])|('고등학교' in df_policy.loc[i,'지원대상'])&('졸업' in df_policy.loc[i,'지원대상'])|('마이스터고' in df_policy.loc[i,'지원대상']):
#       df_policy.loc[i,'GRAD_HIGH']= 1
#   else:
#     df_policy.loc[i,'GRAD_HIGH']= 0

# # 지원대상에 전문대, 대학&재학, 대학생, 대학 재학 > 대학으로 취급
# for i in range(len(df_policy)):
#   if ('전문대' in df_policy.loc[i,'지원대상'])|('대학' in df_policy.loc[i,'지원대상'])&('재학' in df_policy.loc[i,'지원대상'])|('대학생' in df_policy.loc[i,'지원대상']):
#     df_policy.loc[i,'ENROLL_UNIV']= 1
#   else:
#     df_policy.loc[i,'ENROLL_UNIV']= 0
# # 지원대상에 대학&졸업이 같이 들어있을 때 대학 졸업으로 취급
# for i in range(len(df_policy)):
#   if ('대학' in df_policy.loc[i,'지원대상'])&('졸업' in df_policy.loc[i,'지원대상']):
#     df_policy.loc[i,'GRAD_UNIV']= 1
#   else:
#     df_policy.loc[i,'GRAD_UNIV']= 0

# # 학력 값 만들기
# df_policy['EDUCATION']=''

# for i in range(len(df_policy)):
#   values = df_policy.loc[i,'GRAD_HIGH':'GRAD_UNIV'].values
#   if (values[0]==0)&(values[1]==0)&(values[2]==0):
#     df_policy.loc[i,'EDUCATION']= '무관'
#   else:
#     for j in range(len(values)):
#       if j == 0:
#         if values[j]==1:
#           df_policy.loc[i,'EDUCATION']+='고등학교 졸업'
#       elif j == 1:
#         if values[j]==1:
#           if df_policy.loc[i,'EDUCATION']=='':
#             df_policy.loc[i,'EDUCATION']+='대학(원) 재학'
#           else:
#             df_policy.loc[i,'EDUCATION']+=', 대학(원) 재학'
  
#       elif j == 2:
#         if values[j]==1:
#           if df_policy.loc[i,'EDUCATION']=='':
#             df_policy.loc[i,'EDUCATION']+='대학(원) 졸업'
#           else:
#             df_policy.loc[i,'EDUCATION']+=', 대학(원) 졸업'
# del df_policy['GRAD_HIGH']
# del df_policy['ENROLL_UNIV']
# del df_policy['GRAD_UNIV']

## **9. 직장**

In [ ]:
wello_compared['직장'].unique()

## 10.신청절차 ##

### A. TAG Code ###

In [ ]:
# 신청절차 TAG 추출을 위해 필요한 열(['정책ID','정책서비스ID','서비스명','소관기관','온라인신청사이트URL','신청절차','태그-신청절차']) 사용
# df_policy_apply_type으로 선언
df_policy_apply_type = df_policy[['정책ID','정책서비스ID','서비스명','소관기관','온라인신청사이트URL','신청절차','태그-신청절차']]
df_policy_apply_type.head()

,정책ID,정책서비스ID,서비스명,소관기관,온라인신청사이트URL,신청절차,태그-신청절차
0,1,000000465790,만 3~5세 유치원 유아 학비 지원(3~5세 누리과정 운영),교육부,http://online.bokjiro.go.kr/,신청ㆍ접수(읍면동사무소) - 보장 결정(시군구 사업팀),NaN
1,2,105100000001,근로·자녀장려금,국세청,https://www.hometax.go.kr,"○ 개벌 신청 안내받은 경우에는 ARS, 모바일 국세청 홈택스 앱, 인터넷 홈택스 ...",NaN
2,3,116010000001,월세 자금보증,금융위원회,-,"신청 방식 : 은행 방문\n신청 절차\n1. 은행 방문 및 상담 신청\n2. 보증,...",오프라인
3,4,116010000027,‘힘내라 대한민국’특별운영자금 (산업은행),금융위원회,http://www.kdb.co.kr,○ 3.26일부터 기업의 신청접수ㆍ지원 중\n\n ➡ 산업은행 전국 영업점을 방문하...,오프라인
4,5,119200000001,친환경 에너지 절감 장비 보급,해양수산부,-,해당 지자체(시군구청)에 방문하여 신청,오프라인


* 온라인, 오프라인 열을 생성
* '온라인신청사이트URL'의 값을 확인하여 '온라인'열에 해당하는 값 추가
* '신청절차'의 값을 확인하여 '오프라인'열에 해당하는 값 추가
* '온라인'열과 '오프라인'열을 비교하여 '태그-신청절차'에 값 반영

In [ ]:
# 온라인, 오프라인 여부를 따로 측정을 위한 열 생성
df_policy_apply_type.loc[:,'온라인'] = np.NaN
df_policy_apply_type.loc[:,'오프라인'] = np.NaN
df_policy_apply_type.loc[:,'자동'] = np.NaN
df_policy_apply_type.loc[:,'종합'] = np.NaN

In [ ]:
df_policy_apply_type.head()

,정책ID,정책서비스ID,서비스명,소관기관,온라인신청사이트URL,신청절차,태그-신청절차,온라인,오프라인,자동,종합
0,1,000000465790,만 3~5세 유치원 유아 학비 지원(3~5세 누리과정 운영),교육부,http://online.bokjiro.go.kr/,신청ㆍ접수(읍면동사무소) - 보장 결정(시군구 사업팀),NaN,NaN,NaN,NaN,NaN
1,2,105100000001,근로·자녀장려금,국세청,https://www.hometax.go.kr,"○ 개벌 신청 안내받은 경우에는 ARS, 모바일 국세청 홈택스 앱, 인터넷 홈택스 ...",NaN,NaN,NaN,NaN,NaN
2,3,116010000001,월세 자금보증,금융위원회,-,"신청 방식 : 은행 방문\n신청 절차\n1. 은행 방문 및 상담 신청\n2. 보증,...",오프라인,NaN,NaN,NaN,NaN
3,4,116010000027,‘힘내라 대한민국’특별운영자금 (산업은행),금융위원회,http://www.kdb.co.kr,○ 3.26일부터 기업의 신청접수ㆍ지원 중\n\n ➡ 산업은행 전국 영업점을 방문하...,오프라인,NaN,NaN,NaN,NaN
4,5,119200000001,친환경 에너지 절감 장비 보급,해양수산부,-,해당 지자체(시군구청)에 방문하여 신청,오프라인,NaN,NaN,NaN,NaN


In [ ]:
# '태그-신청절차' 값 분포 확인
df_policy_apply_type['태그-신청절차'].value_counts()

오프라인        31217
온라인,오프라인     3427
온라인          2373
Name: 태그-신청절차, dtype: int64

In [ ]:
# '태그-신청절차' 결측치 확인
df_policy_apply_type['태그-신청절차'].isna().sum()

51469

* '태그-신청절차' 값 분포
    * 오프라인 31,217
    * 온라인 2,373
    * 온라인, 오프라인 3,427
    * 결측치 51,469

In [ ]:
# 일부 값은 float형이 있어, '신청절차'열을 문자열 형태로 변형
df_policy_apply_type['신청절차'] = df_policy_apply_type['신청절차'].astype(str)
df_policy_apply_type['온라인신청사이트URL'] = df_policy_apply_type['온라인신청사이트URL'].astype(str)

In [ ]:
# 태그 보완 작업을 위한 결측치 0으로 대체
df_policy_apply_type['종합'] = df_policy_apply_type['종합'].fillna(0)

In [ ]:
# 온/오프라인/자동 분류
# 온라인
start = time.process_time()

for i in range(len(df_policy_apply_type)):
    if (('페이지' in df_policy_apply_type.loc[i,'신청절차'])|('사이트' in df_policy_apply_type.loc[i,'신청절차'])
    |('이메일' in df_policy_apply_type.loc[i,'신청절차'])|('인터넷' in df_policy_apply_type.loc[i,'신청절차'])
    |('온라인' in df_policy_apply_type.loc[i,'신청절차'])|('누리집' in df_policy_apply_type.loc[i,'신청절차'])
    |('복지로' in df_policy_apply_type.loc[i,'신청절차'])|('전산' in df_policy_apply_type.loc[i,'신청절차'])
    |('E-mail' in df_policy_apply_type.loc[i,'신청절차'])|('모바일' in df_policy_apply_type.loc[i,'신청절차'])
    |('시스템' in df_policy_apply_type.loc[i,'신청절차'])):
        df_policy_apply_type.loc[i,'온라인']='온라인'

end = time.process_time()
print("Time elapsed: ", timedelta(seconds=end-start))

Time elapsed:  0:03:09.127912


In [ ]:
# 온/오프라인/자동 분류
# 오프라인
start = time.process_time()

for i in range(len(df_policy_apply_type)):
    if (('방문' in df_policy_apply_type.loc[i,'신청절차'])|('우편' in df_policy_apply_type.loc[i,'신청절차'])
    |('오프라인' in df_policy_apply_type.loc[i,'신청절차'])|('시·군·구' in df_policy_apply_type.loc[i,'신청절차'])
    |('영업점' in df_policy_apply_type.loc[i,'신청절차'])|('은행' in df_policy_apply_type.loc[i,'신청절차'])
    |('사무소' in df_policy_apply_type.loc[i,'신청절차'])|('기관' in df_policy_apply_type.loc[i,'신청절차'])
    |('지자체' in df_policy_apply_type.loc[i,'신청절차'])|('시․군․구' in df_policy_apply_type.loc[i,'신청절차'])
    |('시‧군‧구' in df_policy_apply_type.loc[i,'신청절차'])|('수협' in df_policy_apply_type.loc[i,'신청절차'])
    |('위원회' in df_policy_apply_type.loc[i,'신청절차'])|('센터' in df_policy_apply_type.loc[i,'신청절차'])
    |('시군' in df_policy_apply_type.loc[i,'신청절차'])|('검사' in df_policy_apply_type.loc[i,'신청절차'])
    |('경찰' in df_policy_apply_type.loc[i,'신청절차'])|('현장' in df_policy_apply_type.loc[i,'신청절차'])
    |('시도' in df_policy_apply_type.loc[i,'신청절차'])|('보건소' in df_policy_apply_type.loc[i,'신청절차'])
    |('민간보조' in df_policy_apply_type.loc[i,'신청절차'])|('유선' in df_policy_apply_type.loc[i,'신청절차'])):
        df_policy_apply_type.loc[i,'오프라인']='오프라인'

end = time.process_time()
print("Time elapsed: ", timedelta(seconds=end-start))

Time elapsed:  0:01:58.535099


In [ ]:
# 온/오프라인/자동 분류
# 자동
start = time.process_time()

for i in range(len(df_policy_apply_type)):
    if (('통일부' in df_policy_apply_type.loc[i,'신청절차'])|('하나원' in df_policy_apply_type.loc[i,'신청절차'])
    |('통계교육원' in df_policy_apply_type.loc[i,'신청절차'])|('대입원서' in df_policy_apply_type.loc[i,'신청절차'])):
        df_policy_apply_type.loc[i,'자동']='자동'

end = time.process_time()
print("Time elapsed: ", timedelta(seconds=end-start))

Time elapsed:  0:00:03.021338


In [ ]:
# 온/오프라인/자동 분류
# '온라인신청사이트URL'의 값에 따른 온라인 분류
start = time.process_time()

for i in range(len(df_policy_apply_type)):
    if (('http' in df_policy_apply_type.loc[i,'온라인신청사이트URL'])|('go' in df_policy_apply_type.loc[i,'온라인신청사이트URL'])
    |('kr' in df_policy_apply_type.loc[i,'온라인신청사이트URL'])|('@' in df_policy_apply_type.loc[i,'온라인신청사이트URL'])
    |('인터넷' in df_policy_apply_type.loc[i,'온라인신청사이트URL'])|('www' in df_policy_apply_type.loc[i,'온라인신청사이트URL'])
    |('WWW' in df_policy_apply_type.loc[i,'온라인신청사이트URL'])|('net' in df_policy_apply_type.loc[i,'온라인신청사이트URL'])):
        df_policy_apply_type.loc[i,'온라인']='온라인'

end = time.process_time()
print("Time elapsed: ", timedelta(seconds=end-start))

Time elapsed:  0:00:36.391940


In [ ]:
# 온/오프라인/자동 분류
# '온라인신청사이트URL'의 값에 따른 오프라인 분류
start = time.process_time()

for i in range(len(df_policy_apply_type)):
    if (('방문' in df_policy_apply_type.loc[i,'온라인신청사이트URL'])|('불가능' in df_policy_apply_type.loc[i,'온라인신청사이트URL'])):
        df_policy_apply_type.loc[i,'오프라인']='오프라인'        
    elif ((df_policy_apply_type.loc[i,'온라인신청사이트URL'] == 'nan') or (df_policy_apply_type.loc[i,'온라인신청사이트URL'] =='-')
    or (df_policy_apply_type.loc[i,'온라인신청사이트URL'] == '정상섭')):
        df_policy_apply_type.loc[i,'오프라인']='오프라인'

end = time.process_time()
print("Time elapsed: ", timedelta(seconds=end-start))

Time elapsed:  0:02:06.763746


In [ ]:
# 온라인 열에 '온라인'값과 오프라인 열에 '오프라인'값이 모두 반영되어 있는 경우: 온/오프라인 복합
# 온라인 열에만 '온라인'값이 반영된 경우: 온라인
# 오프라인 열에만 '오프라인'값이 반영된 경우: 오프라인
# 자동 열에만 '자동'값이 반영된 경우: 자동
start = time.process_time()
for i in range(len(df_policy_apply_type)):
    if df_policy_apply_type['온라인'][i] == '온라인' and df_policy_apply_type['오프라인'][i] == '오프라인':
        df_policy_apply_type['종합'][i] = '온/오프라인 복합'
    elif df_policy_apply_type['온라인'][i] != '온라인' and df_policy_apply_type['오프라인'][i] == '오프라인':
        df_policy_apply_type['종합'][i] = '오프라인'
    elif df_policy_apply_type['온라인'][i] == '온라인' and df_policy_apply_type['오프라인'][i] != '오프라인':
        df_policy_apply_type['종합'][i] = '온라인'
    elif df_policy_apply_type['자동'][i] == '자동':
        df_policy_apply_type['종합'][i] = '자동'
end = time.process_time()
print("Time elapsed: ", timedelta(seconds=end-start))   

Time elapsed:  0:00:33.891642


In [ ]:
# '종합'열의 값 분포 확인
df_policy_apply_type['종합'].value_counts()

오프라인         71781
온/오프라인 복합    12014
온라인           4690
0.0              1
Name: 종합, dtype: int64

In [ ]:
con = (df_policy_apply_type.종합 ==  0)
df_policy_apply_type[con]

,정책ID,정책서비스ID,서비스명,소관기관,온라인신청사이트URL,신청절차,태그-신청절차,온라인,오프라인,자동,종합
67544,67545,479000000497,농기계임대사업지원,전라북도 부안군,정상섭,농기계임대사업지원,NaN,NaN,NaN,NaN,0


### B. 정리 ###

* 1단계: TAG 추출을 위한 열 추출 및 생성
* 2단계: '신청절차'에서 온라인 키워드 분류
* 3단계: '신청절차'에서 오프라인 키워드 분류
* 4단계: '신청절차'에서 자동 키워드 분류
* 5단계: '온라인신청사이트URL'에서 온라인 키워드 분류
* 6단계: '온라인신청사이트URL'에서 오프라인 키워드 분류
* 7단계: 온라인, 오프라인, 자동 키워드 분포를 통해 '온라인', '오프라인', '온/오프라인 복합', '자동' 태그 최종 추출

특성공학 코드 실행시 소요시간 10분

### C. TAG Code 전체 ###

In [ ]:
# 신청절차 TAG 추출을 위해 필요한 열(['정책ID','정책서비스ID','서비스명','소관기관','온라인신청사이트URL','신청절차','태그-신청절차']) 사용
# df_policy_apply_type으로 선언
df_policy_apply_type = df_policy[['정책ID','정책서비스ID','서비스명','소관기관','온라인신청사이트URL','신청절차']]
df_policy_apply_type.head()

# 온라인, 오프라인 여부를 따로 측정을 위한 열 생성
df_policy_apply_type.loc[:,'온라인'] = np.NaN
df_policy_apply_type.loc[:,'오프라인'] = np.NaN
df_policy_apply_type.loc[:,'자동'] = np.NaN
df_policy_apply_type.loc[:,'종합'] = np.NaN

# 일부 값은 float형이 있어, '신청절차'열을 문자열 형태로 변형
df_policy_apply_type['신청절차'] = df_policy_apply_type['신청절차'].astype(str)
df_policy_apply_type['온라인신청사이트URL'] = df_policy_apply_type['온라인신청사이트URL'].astype(str)

# 온/오프라인/자동 분류
## 온라인
for i in range(len(df_policy_apply_type)):
    if (('페이지' in df_policy_apply_type.loc[i,'신청절차'])|('사이트' in df_policy_apply_type.loc[i,'신청절차'])
    |('이메일' in df_policy_apply_type.loc[i,'신청절차'])|('인터넷' in df_policy_apply_type.loc[i,'신청절차'])
    |('온라인' in df_policy_apply_type.loc[i,'신청절차'])|('누리집' in df_policy_apply_type.loc[i,'신청절차'])
    |('복지로' in df_policy_apply_type.loc[i,'신청절차'])|('전산' in df_policy_apply_type.loc[i,'신청절차'])
    |('E-mail' in df_policy_apply_type.loc[i,'신청절차'])|('모바일' in df_policy_apply_type.loc[i,'신청절차'])
    |('시스템' in df_policy_apply_type.loc[i,'신청절차'])):
        df_policy_apply_type.loc[i,'온라인']='온라인'
## 오프라인
for i in range(len(df_policy_apply_type)):
    if (('방문' in df_policy_apply_type.loc[i,'신청절차'])|('우편' in df_policy_apply_type.loc[i,'신청절차'])
    |('오프라인' in df_policy_apply_type.loc[i,'신청절차'])|('시·군·구' in df_policy_apply_type.loc[i,'신청절차'])
    |('영업점' in df_policy_apply_type.loc[i,'신청절차'])|('은행' in df_policy_apply_type.loc[i,'신청절차'])
    |('사무소' in df_policy_apply_type.loc[i,'신청절차'])|('기관' in df_policy_apply_type.loc[i,'신청절차'])
    |('지자체' in df_policy_apply_type.loc[i,'신청절차'])|('시․군․구' in df_policy_apply_type.loc[i,'신청절차'])
    |('시‧군‧구' in df_policy_apply_type.loc[i,'신청절차'])|('수협' in df_policy_apply_type.loc[i,'신청절차'])
    |('위원회' in df_policy_apply_type.loc[i,'신청절차'])|('센터' in df_policy_apply_type.loc[i,'신청절차'])
    |('시군' in df_policy_apply_type.loc[i,'신청절차'])|('검사' in df_policy_apply_type.loc[i,'신청절차'])
    |('경찰' in df_policy_apply_type.loc[i,'신청절차'])|('현장' in df_policy_apply_type.loc[i,'신청절차'])
    |('시도' in df_policy_apply_type.loc[i,'신청절차'])|('보건소' in df_policy_apply_type.loc[i,'신청절차'])
    |('민간보조' in df_policy_apply_type.loc[i,'신청절차'])|('유선' in df_policy_apply_type.loc[i,'신청절차'])):
        df_policy_apply_type.loc[i,'오프라인']='오프라인'
## 자동
for i in range(len(df_policy_apply_type)):
    if (('통일부' in df_policy_apply_type.loc[i,'신청절차'])|('하나원' in df_policy_apply_type.loc[i,'신청절차'])
    |('통계교육원' in df_policy_apply_type.loc[i,'신청절차'])|('대입원서' in df_policy_apply_type.loc[i,'신청절차'])):
        df_policy_apply_type.loc[i,'자동']='자동'
## '온라인신청사이트URL'의 값에 따른 온라인 분류
for i in range(len(df_policy_apply_type)):
    if (('http' in df_policy_apply_type.loc[i,'온라인신청사이트URL'])|('go' in df_policy_apply_type.loc[i,'온라인신청사이트URL'])
    |('kr' in df_policy_apply_type.loc[i,'온라인신청사이트URL'])|('@' in df_policy_apply_type.loc[i,'온라인신청사이트URL'])
    |('인터넷' in df_policy_apply_type.loc[i,'온라인신청사이트URL'])|('www' in df_policy_apply_type.loc[i,'온라인신청사이트URL'])
    |('WWW' in df_policy_apply_type.loc[i,'온라인신청사이트URL'])|('net' in df_policy_apply_type.loc[i,'온라인신청사이트URL'])):
        df_policy_apply_type.loc[i,'온라인']='온라인'
## '온라인신청사이트URL'의 값에 따른 오프라인 분류
for i in range(len(df_policy_apply_type)):
    if (('방문' in df_policy_apply_type.loc[i,'온라인신청사이트URL'])|('불가능' in df_policy_apply_type.loc[i,'온라인신청사이트URL'])):
        df_policy_apply_type.loc[i,'오프라인']='오프라인'        
    elif ((df_policy_apply_type.loc[i,'온라인신청사이트URL'] == 'nan') or (df_policy_apply_type.loc[i,'온라인신청사이트URL'] =='-')
    or (df_policy_apply_type.loc[i,'온라인신청사이트URL'] == '정상섭')):
        df_policy_apply_type.loc[i,'오프라인']='오프라인'

# 온라인 열에 '온라인'값과 오프라인 열에 '오프라인'값이 모두 반영되어 있는 경우: 온/오프라인 복합
# 온라인 열에만 '온라인'값이 반영된 경우: 온라인
# 오프라인 열에만 '오프라인'값이 반영된 경우: 오프라인
# 자동 열에만 '자동'값이 반영된 경우: 자동
for i in range(len(df_policy_apply_type)):
    if df_policy_apply_type['온라인'][i] == '온라인' and df_policy_apply_type['오프라인'][i] == '오프라인':
        df_policy_apply_type['종합'][i] = '온/오프라인 복합'
    elif df_policy_apply_type['온라인'][i] != '온라인' and df_policy_apply_type['오프라인'][i] == '오프라인':
        df_policy_apply_type['종합'][i] = '오프라인'
    elif df_policy_apply_type['온라인'][i] == '온라인' and df_policy_apply_type['오프라인'][i] != '오프라인':
        df_policy_apply_type['종합'][i] = '온라인'
    elif df_policy_apply_type['자동'][i] == '자동':
        df_policy_apply_type['종합'][i] = '자동'

# df_policy_apply_type['종합']을 df_policy['태그-신청절차']로 사용
# df_policy['태그-신청절차'] = df_policy_apply_type['종합']

In [ ]:
df_policy_apply_type['종합'].value_counts()

오프라인         71782
온/오프라인 복합    11955
온라인           4749
Name: 종합, dtype: int64

In [ ]:
df_policy.shape

(88486, 38)

In [ ]:
71782+11955+4749

88486

## 11.연령 ##

생애주기
* 영, 유아: 0 ~ 5세
* 아동: 6 ~ 12세
* 청소년: 13 ~ 18세
* 청년: 19 ~ 29세
* 중년: 30 ~ 49세
* 장년: 50 ~ 65세
* 노년: 65세 이상

기존 데이터의 태그는 생애주기의 값을 추출하여 추출된 값에 따라 대상연령을 지정한 것으로 보인다.

적용방법: 신청대상에서 나이를 우선 추출하여 TAG값에 반영한다. 나이 추출로 반영이 안 되는 것에 대하여 생애주기에 라 연령TAG를 부여한다.

### A. TAG Code ###

In [ ]:
# 연령 태그 추출을 위해 '정책ID', '정책서비스ID', '서비스명', '선정기준', '지원내용', '지원대상', '태그-대상연령시작', '태그-대상연령끝'
# df_policy_age 선언
df_policy_age = df_policy[['정책ID', '정책서비스ID', '서비스명', '선정기준', '지원내용', '지원대상', '태그-대상연령시작', '태그-대상연령끝']]
df_policy_age.head()

,정책ID,정책서비스ID,서비스명,선정기준,지원내용,지원대상,태그-대상연령시작,태그-대상연령끝
0,1,000000465790,만 3~5세 유치원 유아 학비 지원(3~5세 누리과정 운영),1) 지원대상 : 어린이집을 이용하는 만 3~5세 아동\n2) 신청인 : 아동의 보...,"만 3~5세 누리과정 운영에 필요한 비용 지원 \n - 보육료, 담당교사 처우 개선...",지원대상 : 어린이집을 이용하는 만 3~5세 아동\n- 취학유예 : 만 6세아 초등...,6,12
1,2,105100000001,근로·자녀장려금,"○ 근로소득, 종교인소득 또는 종교인소득이 있고 아래 요건을 모두 충족하는 자\n\...",○ 전년도 연간 부부합산 총 급여액 등(사업소득과 근로소득의 합계)에 따라\n - ...,"○ 전년도에 정해진 범위의 근로소득, 사업소득 또는 종교인소득이 있고 근로장려금과 ...",0,99
2,3,116010000001,월세 자금보증,○ (주택도시기금 주거안정 월세대출 대상자 요건) 무주택 세대주(세대원 전원 포함)...,○ 주택도시기금 주거안정 월세대출 요건을 충족하는 대상자에 대해 월세자금보증 지원\...,○ 보증금 1억 원 이하 및 월세금 60만 원 이하인 월세 계약을 체결한 주택도시 ...,0,99
3,4,116010000027,‘힘내라 대한민국’특별운영자금 (산업은행),○ 산업은행 심사기준 충족 업체,○ 자금용도 : 운영자금\n\n○ 한도 : 기존 대출한도 外 일정범위내 특별한도 부...,"○ 지원대상 : 코로나19 등 질병, 자연재해 등 국가 재난 발생으로 피해를 입은 ...",0,99
4,5,119200000001,친환경 에너지 절감 장비 보급,"○ 기관ㆍ장비ㆍ설비사업의 사업자 선정은 다음의 순위로 하되, 동일 순위자의 세부 우...",○ 유류비 절감을 위한 유류절감장치 설치\n○ 저효율(육상기관 포함)기관 및 사...,"○ 어선 노후기관, 장비를 교체하고자 하는 연근해 어업인",0,99


In [ ]:
df_policy_age['지원대상'] = df_policy_age['지원대상'].astype(str)

In [ ]:
# 특성 추출을 위한 '지원대상' 결측치 확인
df_policy_age['지원대상'].isna().sum()

0

In [ ]:
df_policy_age['num_list'] = 0
df_policy_age['new_num_list'] = 0
df_policy_age['tag_min'] = 0
df_policy_age['tag_max'] = 0

In [ ]:
df_policy_age.head()

,정책ID,정책서비스ID,서비스명,선정기준,지원내용,지원대상,태그-대상연령시작,태그-대상연령끝,num_list,new_num_list,tag_min,tag_max
0,1,000000465790,만 3~5세 유치원 유아 학비 지원(3~5세 누리과정 운영),1) 지원대상 : 어린이집을 이용하는 만 3~5세 아동\n2) 신청인 : 아동의 보...,"만 3~5세 누리과정 운영에 필요한 비용 지원 \n - 보육료, 담당교사 처우 개선...",지원대상 : 어린이집을 이용하는 만 3~5세 아동\n- 취학유예 : 만 6세아 초등...,6,12,0,0,0,0
1,2,105100000001,근로·자녀장려금,"○ 근로소득, 종교인소득 또는 종교인소득이 있고 아래 요건을 모두 충족하는 자\n\...",○ 전년도 연간 부부합산 총 급여액 등(사업소득과 근로소득의 합계)에 따라\n - ...,"○ 전년도에 정해진 범위의 근로소득, 사업소득 또는 종교인소득이 있고 근로장려금과 ...",0,99,0,0,0,0
2,3,116010000001,월세 자금보증,○ (주택도시기금 주거안정 월세대출 대상자 요건) 무주택 세대주(세대원 전원 포함)...,○ 주택도시기금 주거안정 월세대출 요건을 충족하는 대상자에 대해 월세자금보증 지원\...,○ 보증금 1억 원 이하 및 월세금 60만 원 이하인 월세 계약을 체결한 주택도시 ...,0,99,0,0,0,0
3,4,116010000027,‘힘내라 대한민국’특별운영자금 (산업은행),○ 산업은행 심사기준 충족 업체,○ 자금용도 : 운영자금\n\n○ 한도 : 기존 대출한도 外 일정범위내 특별한도 부...,"○ 지원대상 : 코로나19 등 질병, 자연재해 등 국가 재난 발생으로 피해를 입은 ...",0,99,0,0,0,0
4,5,119200000001,친환경 에너지 절감 장비 보급,"○ 기관ㆍ장비ㆍ설비사업의 사업자 선정은 다음의 순위로 하되, 동일 순위자의 세부 우...",○ 유류비 절감을 위한 유류절감장치 설치\n○ 저효율(육상기관 포함)기관 및 사...,"○ 어선 노후기관, 장비를 교체하고자 하는 연근해 어업인",0,99,0,0,0,0


In [ ]:
# 방법 1. 지원 내용의 숫자를 추출
for i in range(len(df_policy_age)):
    df_policy_age['num_list'][i] = re.findall("\d+", df_policy_age['지원대상'][i])

In [ ]:
df_policy_age['new_num_list'] = df_policy_age['num_list']

In [ ]:
df_policy_age.head()

,정책ID,정책서비스ID,서비스명,선정기준,지원내용,지원대상,태그-대상연령시작,태그-대상연령끝,num_list,new_num_list,tag_min,tag_max
0,1,000000465790,만 3~5세 유치원 유아 학비 지원(3~5세 누리과정 운영),1) 지원대상 : 어린이집을 이용하는 만 3~5세 아동\n2) 신청인 : 아동의 보...,"만 3~5세 누리과정 운영에 필요한 비용 지원 \n - 보육료, 담당교사 처우 개선...",지원대상 : 어린이집을 이용하는 만 3~5세 아동\n- 취학유예 : 만 6세아 초등...,6,12,"[3, 5, 6, 1]","[3, 5, 6, 1]",0,0
1,2,105100000001,근로·자녀장려금,"○ 근로소득, 종교인소득 또는 종교인소득이 있고 아래 요건을 모두 충족하는 자\n\...",○ 전년도 연간 부부합산 총 급여액 등(사업소득과 근로소득의 합계)에 따라\n - ...,"○ 전년도에 정해진 범위의 근로소득, 사업소득 또는 종교인소득이 있고 근로장려금과 ...",0,99,[5],[5],0,0
2,3,116010000001,월세 자금보증,○ (주택도시기금 주거안정 월세대출 대상자 요건) 무주택 세대주(세대원 전원 포함)...,○ 주택도시기금 주거안정 월세대출 요건을 충족하는 대상자에 대해 월세자금보증 지원\...,○ 보증금 1억 원 이하 및 월세금 60만 원 이하인 월세 계약을 체결한 주택도시 ...,0,99,"[1, 60]","[1, 60]",0,0
3,4,116010000027,‘힘내라 대한민국’특별운영자금 (산업은행),○ 산업은행 심사기준 충족 업체,○ 자금용도 : 운영자금\n\n○ 한도 : 기존 대출한도 外 일정범위내 특별한도 부...,"○ 지원대상 : 코로나19 등 질병, 자연재해 등 국가 재난 발생으로 피해를 입은 ...",0,99,[19],[19],0,0
4,5,119200000001,친환경 에너지 절감 장비 보급,"○ 기관ㆍ장비ㆍ설비사업의 사업자 선정은 다음의 순위로 하되, 동일 순위자의 세부 우...",○ 유류비 절감을 위한 유류절감장치 설치\n○ 저효율(육상기관 포함)기관 및 사...,"○ 어선 노후기관, 장비를 교체하고자 하는 연근해 어업인",0,99,[],[],0,0


In [ ]:
for i in range(len(df_policy_age)):
    for j in df_policy_age['num_list'][i]:
        if len(df_policy_age['num_list']) != 0:
            if (((str(j)+'~') in df_policy_age.loc[i, '지원대상'])|((str(j)+'세~') in df_policy_age.loc[i, '지원대상'])|((str(j)+'세 ~') in df_policy_age.loc[i, '지원대상'])
            |((str(j)+'이상') in df_policy_age.loc[i, '지원대상'])|((str(j)+'세 이상') in df_policy_age.loc[i, '지원대상'])|((str(j)+'대 이상') in df_policy_age.loc[i, '지원대상'])
            |((str(j)+' 대 이상') in df_policy_age.loc[i, '지원대상'])):
                df_policy_age['tag_min'][i] = j
for i in range(len(df_policy_age)):
    for j in df_policy_age['num_list'][i]:
        if len(df_policy_age['num_list']) != 0:
            if ((('~'+str(j)) in df_policy_age.loc[i, '지원대상'])|(('~ '+ str(j)) in df_policy_age.loc[i, '지원대상'])|((str(j)+'세 이하') in df_policy_age.loc[i, '지원대상'])
            |((str(j)+'세 미만') in df_policy_age.loc[i, '지원대상'])|((str(j)+'세 까지') in df_policy_age.loc[i, '지원대상'])|(('~만 '+ str(j)) in df_policy_age.loc[i, '지원대상'])
            |(('~ 만'+ str(j)) in df_policy_age.loc[i, '지원대상'])|(('~만'+ str(j)) in df_policy_age.loc[i, '지원대상'])):
                df_policy_age['tag_max'][i] = j

In [ ]:
# 전액 감면, 장애인 등 특정 대상이 있는 것에 대해 연령 tag_min, max는 0으로 조정
for i in range(len(df_policy_age)):
    if ((('전액감면' in df_policy_age.loc[i, '지원대상'])|('장애인' in df_policy_age.loc[i, '지원대상']))&('비장애' not in df_policy_age.loc[i, '지원대상'])):
        df_policy_age['tag_min'][i] = 0
        df_policy_age['tag_max'][i] = 0

In [ ]:
df_policy_age

,정책ID,정책서비스ID,서비스명,선정기준,지원내용,지원대상,태그-대상연령시작,태그-대상연령끝,num_list,new_num_list,tag_min,tag_max
0,1,000000465790,만 3~5세 유치원 유아 학비 지원(3~5세 누리과정 운영),1) 지원대상 : 어린이집을 이용하는 만 3~5세 아동\n2) 신청인 : 아동의 보...,"만 3~5세 누리과정 운영에 필요한 비용 지원 \n - 보육료, 담당교사 처우 개선...",지원대상 : 어린이집을 이용하는 만 3~5세 아동\n- 취학유예 : 만 6세아 초등...,6,12,"[3, 5, 6, 1]","[3, 5, 6, 1]",3,5
1,2,105100000001,근로·자녀장려금,"○ 근로소득, 종교인소득 또는 종교인소득이 있고 아래 요건을 모두 충족하는 자\n\...",○ 전년도 연간 부부합산 총 급여액 등(사업소득과 근로소득의 합계)에 따라\n - ...,"○ 전년도에 정해진 범위의 근로소득, 사업소득 또는 종교인소득이 있고 근로장려금과 ...",0,99,[5],[5],0,0
2,3,116010000001,월세 자금보증,○ (주택도시기금 주거안정 월세대출 대상자 요건) 무주택 세대주(세대원 전원 포함)...,○ 주택도시기금 주거안정 월세대출 요건을 충족하는 대상자에 대해 월세자금보증 지원\...,○ 보증금 1억 원 이하 및 월세금 60만 원 이하인 월세 계약을 체결한 주택도시 ...,0,99,"[1, 60]","[1, 60]",0,0
3,4,116010000027,‘힘내라 대한민국’특별운영자금 (산업은행),○ 산업은행 심사기준 충족 업체,○ 자금용도 : 운영자금\n\n○ 한도 : 기존 대출한도 外 일정범위내 특별한도 부...,"○ 지원대상 : 코로나19 등 질병, 자연재해 등 국가 재난 발생으로 피해를 입은 ...",0,99,[19],[19],0,0
4,5,119200000001,친환경 에너지 절감 장비 보급,"○ 기관ㆍ장비ㆍ설비사업의 사업자 선정은 다음의 순위로 하되, 동일 순위자의 세부 우...",○ 유류비 절감을 위한 유류절감장치 설치\n○ 저효율(육상기관 포함)기관 및 사...,"○ 어선 노후기관, 장비를 교체하고자 하는 연근해 어업인",0,99,[],[],0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
88481,88482,645000000579,FTA 피해 보전 직불금 지원,○ 다음 각 요건을 모두 충족한 자\n -농업인 등에 해당하는 자로 ｢농어업경영체 ...,"FTA 협정 이행으로 인한 수입 증가로 가격 하락의 피해가 발생할 경우, 발생된 피...",○ FTA 피해 보전직불금 지원 대상 품목을 해당 협정의 발효일 이전부터 생산한 농...,0,99,"[4, 1]","[4, 1]",0,0
88482,88483,650000000822,이혼위기가족지원,협의 및 재판 이혼 신청자 등 이혼을 준비 중인 부부와 아동,"이혼 위기 상담, 교육, 문화서비스\n\n\n\n금융·세금·법률>상담/법률구제>고충...",협의 및 재판 이혼 신청자 등 이혼을 준비 중인 부부와 아동\n\n일반,6,12,[],[],0,0
88483,88484,451000000421,여성 새로 일하기 센터 운영 지원,○ 경력단절 여성 등의 경제활동 촉진법 제3조\n - 정의 : 경력단절 여성 등 이...,여성 새로 일하기 센터에서 구직자 1:1 상담 후 적합한 취업 지원 서비스 연계\n...,"임신, 출산, 육아와 가족 구성원의 돌봄 등을 이유로 경제활동을 중단하였거나 경제활...",0,99,[],[],0,0
88484,88485,440000000167,취학 전 아동 실명예방,- 국민기초생활 수급과 및 차상위 계층\n- 저소득 가정의 어린이(최저생계비 200...,- 안과 사전검사 및 수술비 본인 부담금 지원\n- 실명예방재단 등과 같은 기관에 ...,"만3세~6세 어린이\n\n저소득계층(최저생계비 200% 이내), 취학 전 아동\n\...",6,12,"[3, 6, 200]","[3, 6, 200]",3,6


In [ ]:
df_policy_age.loc[88484]

정책ID                                                        88485
정책서비스ID                                              440000000167
서비스명                                                 취학 전 아동 실명예방
선정기준            - 국민기초생활 수급과 및 차상위 계층\n- 저소득 가정의 어린이(최저생계비 200...
지원내용            - 안과 사전검사 및 수술비 본인 부담금 지원\n- 실명예방재단 등과 같은 기관에 ...
지원대상            만3세~6세 어린이\n\n저소득계층(최저생계비 200% 이내), 취학 전 아동\n\...
태그-대상연령시작                                                       6
태그-대상연령끝                                                       12
num_list                                              [3, 6, 200]
new_num_list                                          [3, 6, 200]
tag_min                                                         3
tag_max                                                         6
Name: 88484, dtype: object

### B. 정리 ###

* '지원대상'에서 숫자만 추출
* 추출된 숫자 리스트 내부 값과 최소, 최대 연령을 파악할 수 있는 단어를 포함하여 탐색
* 탐색 결과에 따라 tag_min, max 값을 지정
* man, max값이 모두 0인 경우 연령 무관

### C. TAG Code 전체 ###

In [ ]:
# 연령 태그 추출을 위해 '정책ID', '정책서비스ID', '서비스명', '선정기준', '지원내용', '지원대상', '태그-대상연령시작', '태그-대상연령끝'
# df_policy_age 선언
df_policy_age = df_policy[['정책ID', '정책서비스ID', '서비스명', '선정기준', '지원내용', '지원대상', '태그-대상연령시작', '태그-대상연령끝']]
df_policy_age.head()
# '지원대상' 문자열 변환
df_policy_age['지원대상'] = df_policy_age['지원대상'].astype(str)
# 태그 지정을 위한 열 추가
df_policy_age['num_list'] = 0
df_policy_age['new_num_list'] = 0
df_policy_age['tag_min'] = 0
df_policy_age['tag_max'] = 0
# 방법 1. 지원 내용의 숫자를 추출
for i in range(len(df_policy_age)):
    df_policy_age['num_list'][i] = re.findall("\d+", df_policy_age['지원대상'][i])
df_policy_age['new_num_list'] = df_policy_age['num_list']

# 연령추출
# 최소 연령 추출
for i in range(len(df_policy_age)):
    for j in df_policy_age['num_list'][i]:
        if len(df_policy_age['num_list']) != 0:
            if (((str(j)+'~') in df_policy_age.loc[i, '지원대상'])|((str(j)+'세~') in df_policy_age.loc[i, '지원대상'])|((str(j)+'세 ~') in df_policy_age.loc[i, '지원대상'])
            |((str(j)+'이상') in df_policy_age.loc[i, '지원대상'])|((str(j)+'세 이상') in df_policy_age.loc[i, '지원대상'])|((str(j)+'대 이상') in df_policy_age.loc[i, '지원대상'])
            |((str(j)+' 대 이상') in df_policy_age.loc[i, '지원대상'])):
                df_policy_age['tag_min'][i] = j
# 최대 연령 추출
for i in range(len(df_policy_age)):
    for j in df_policy_age['num_list'][i]:
        if len(df_policy_age['num_list']) != 0:
            if ((('~'+str(j)) in df_policy_age.loc[i, '지원대상'])|(('~ '+ str(j)) in df_policy_age.loc[i, '지원대상'])|((str(j)+'세 이하') in df_policy_age.loc[i, '지원대상'])
            |((str(j)+'세 미만') in df_policy_age.loc[i, '지원대상'])|((str(j)+'세 까지') in df_policy_age.loc[i, '지원대상'])|(('~만 '+ str(j)) in df_policy_age.loc[i, '지원대상'])
            |(('~ 만'+ str(j)) in df_policy_age.loc[i, '지원대상'])|(('~만'+ str(j)) in df_policy_age.loc[i, '지원대상'])):
                df_policy_age['tag_max'][i] = j

# 전액 감면, 장애인 등 특정 대상이 있는 것에 대해 연령 tag_min, max는 0으로 조정
for i in range(len(df_policy_age)):
    if ((('전액감면' in df_policy_age.loc[i, '지원대상'])|('장애인' in df_policy_age.loc[i, '지원대상']))&('비장애' not in df_policy_age.loc[i, '지원대상'])):
        df_policy_age['tag_min'][i] = 0
        df_policy_age['tag_max'][i] = 0

## 13.성별 ##

* 성별 분류(키워드): 지원대상
    * 남성: 남성 
        * 농촌에 거주하면서 실제 영농에 종사하고 있는 만 50세 미만 미혼남성 농업인으로서 외국인 여성과 국제결혼을 희망하는 자: **여성이 제외되어야 함**
        * 노인, 장애, 가족 등의 키워드가 있는 값은 무관으로 조정
    * 여성: 여성, 임신
        * 노인, 장애, 가족 등의 키워드가 있는 값은 무관으로 조정
    * 무관: 위의 두 조건에 충족되지 않는 값

### A. TAG Code ###

In [ ]:
# 연령 태그 추출을 위해 '정책ID', '정책서비스ID', '서비스명', '선정기준', '지원내용', '지원대상', '태그-성별'
# df_policy_sex 선언
df_policy_sex = df_policy[['정책ID', '정책서비스ID', '서비스명', '선정기준', '지원내용', '지원대상', '태그-성별']]
df_policy_sex.head()

,정책ID,정책서비스ID,서비스명,선정기준,지원내용,지원대상,태그-성별
0,1,000000465790,만 3~5세 유치원 유아 학비 지원(3~5세 누리과정 운영),1) 지원대상 : 어린이집을 이용하는 만 3~5세 아동\n2) 신청인 : 아동의 보...,"만 3~5세 누리과정 운영에 필요한 비용 지원 \n - 보육료, 담당교사 처우 개선...",지원대상 : 어린이집을 이용하는 만 3~5세 아동\n- 취학유예 : 만 6세아 초등...,NaN
1,2,105100000001,근로·자녀장려금,"○ 근로소득, 종교인소득 또는 종교인소득이 있고 아래 요건을 모두 충족하는 자\n\...",○ 전년도 연간 부부합산 총 급여액 등(사업소득과 근로소득의 합계)에 따라\n - ...,"○ 전년도에 정해진 범위의 근로소득, 사업소득 또는 종교인소득이 있고 근로장려금과 ...",NaN
2,3,116010000001,월세 자금보증,○ (주택도시기금 주거안정 월세대출 대상자 요건) 무주택 세대주(세대원 전원 포함)...,○ 주택도시기금 주거안정 월세대출 요건을 충족하는 대상자에 대해 월세자금보증 지원\...,○ 보증금 1억 원 이하 및 월세금 60만 원 이하인 월세 계약을 체결한 주택도시 ...,NaN
3,4,116010000027,‘힘내라 대한민국’특별운영자금 (산업은행),○ 산업은행 심사기준 충족 업체,○ 자금용도 : 운영자금\n\n○ 한도 : 기존 대출한도 外 일정범위내 특별한도 부...,"○ 지원대상 : 코로나19 등 질병, 자연재해 등 국가 재난 발생으로 피해를 입은 ...",NaN
4,5,119200000001,친환경 에너지 절감 장비 보급,"○ 기관ㆍ장비ㆍ설비사업의 사업자 선정은 다음의 순위로 하되, 동일 순위자의 세부 우...",○ 유류비 절감을 위한 유류절감장치 설치\n○ 저효율(육상기관 포함)기관 및 사...,"○ 어선 노후기관, 장비를 교체하고자 하는 연근해 어업인",NaN


In [ ]:
# 새 태그 지정을 위한 열 생성
df_policy_sex.loc[:,'tag_sex'] = '0'
df_policy_sex.head()

,정책ID,정책서비스ID,서비스명,선정기준,지원내용,지원대상,태그-성별,tag_sex
0,1,000000465790,만 3~5세 유치원 유아 학비 지원(3~5세 누리과정 운영),1) 지원대상 : 어린이집을 이용하는 만 3~5세 아동\n2) 신청인 : 아동의 보...,"만 3~5세 누리과정 운영에 필요한 비용 지원 \n - 보육료, 담당교사 처우 개선...",지원대상 : 어린이집을 이용하는 만 3~5세 아동\n- 취학유예 : 만 6세아 초등...,NaN,0
1,2,105100000001,근로·자녀장려금,"○ 근로소득, 종교인소득 또는 종교인소득이 있고 아래 요건을 모두 충족하는 자\n\...",○ 전년도 연간 부부합산 총 급여액 등(사업소득과 근로소득의 합계)에 따라\n - ...,"○ 전년도에 정해진 범위의 근로소득, 사업소득 또는 종교인소득이 있고 근로장려금과 ...",NaN,0
2,3,116010000001,월세 자금보증,○ (주택도시기금 주거안정 월세대출 대상자 요건) 무주택 세대주(세대원 전원 포함)...,○ 주택도시기금 주거안정 월세대출 요건을 충족하는 대상자에 대해 월세자금보증 지원\...,○ 보증금 1억 원 이하 및 월세금 60만 원 이하인 월세 계약을 체결한 주택도시 ...,NaN,0
3,4,116010000027,‘힘내라 대한민국’특별운영자금 (산업은행),○ 산업은행 심사기준 충족 업체,○ 자금용도 : 운영자금\n\n○ 한도 : 기존 대출한도 外 일정범위내 특별한도 부...,"○ 지원대상 : 코로나19 등 질병, 자연재해 등 국가 재난 발생으로 피해를 입은 ...",NaN,0
4,5,119200000001,친환경 에너지 절감 장비 보급,"○ 기관ㆍ장비ㆍ설비사업의 사업자 선정은 다음의 순위로 하되, 동일 순위자의 세부 우...",○ 유류비 절감을 위한 유류절감장치 설치\n○ 저효율(육상기관 포함)기관 및 사...,"○ 어선 노후기관, 장비를 교체하고자 하는 연근해 어업인",NaN,0


In [ ]:
# 일부 값은 float형이 있어, '신청대상'열을 문자열 형태로 변형
df_policy_sex['지원대상'] = df_policy_sex['지원대상'].astype(str)

In [ ]:
# 남성 분류
for i in range(len(df_policy_sex)):
    if ((('남성' in df_policy_sex.loc[i, '지원대상'])|('전립선' in df_policy_sex.loc[i, '지원대상']))&(('노인' not in df_policy_sex.loc[i, '지원대상'])
    |('장애' not in df_policy_sex.loc[i, '지원대상'])|('가족' not in df_policy_sex.loc[i, '지원대상']))):
        df_policy_sex.loc[i,'tag_sex']='남성'

In [ ]:
# 여성 분류
for i in range(len(df_policy_sex)):
    if ((('여성' in df_policy_sex.loc[i, '지원대상'])|('임신' in df_policy_sex.loc[i, '지원대상'])|('출산' in df_policy_sex.loc[i, '지원대상']))
    &('남성' not in df_policy_sex.loc[i, '지원대상'])&(('아동' not in df_policy_sex.loc[i, '지원대상'])
    |('노인' not in df_policy_sex.loc[i, '지원대상'])|('가족' not in df_policy_sex.loc[i, '지원대상']))):
        df_policy_sex.loc[i,'tag_sex']='여성'

In [ ]:
# 무관
for i in range(len(df_policy_sex)):
    if ('0' in df_policy_sex.loc[i, 'tag_sex']):
        df_policy_sex.loc[i,'tag_sex']='무관'

In [ ]:
df_policy_sex['tag_sex'].value_counts()

무관    78278
여성    10108
남성      100
Name: tag_sex, dtype: int64

In [ ]:
wello_compared['성별'].value_counts()

무관    67702
여성     7594
남성      521
Name: 성별, dtype: int64

In [ ]:
cnt_mil = 0
for i in range(len(df_policy_sex)):
    if (('군인' in df_policy_sex.loc[i,'서비스명'])|('보훈' in df_policy_sex.loc[i,'서비스명'])|('재향' in df_policy_sex.loc[i,'서비스명'])):
        cnt_mil += 1
cnt_mil

546

### B. 정리 ###

* 태그 착안사항
    * 군인(재향 또는 보훈) 키워드가 있는 경우 admin데이터에서는 남성으로 분류된 경향이 있었다.군인의 경우 자격사항으로 성별을 분류하는 기준으로 선정하기는 어렵다.
    * 대상에서 남성을 추출한 결과, 국제결혼(또는 외국인 여성)키워드가 함께 있어 무관으로 분류될 가능성이 있었다.
* 1단계: '남성' 분류
* 2단계: '여성' 분류
* 3단계: 미분류 값에 대한 '무관' 분류

### C. TAG Code 전체 ###

In [ ]:
# 연령 태그 추출을 위해 '정책ID', '정책서비스ID', '서비스명', '선정기준', '지원내용', '지원대상', '태그-성별'
# df_policy_sex 선언
df_policy_sex = df_policy[['정책ID', '정책서비스ID', '서비스명', '선정기준', '지원내용', '지원대상', '태그-성별']]

# 새 태그 지정을 위한 열 생성
df_policy_sex.loc[:,'tag_sex'] = '0'

# 남성 분류
for i in range(len(df_policy_sex)):
    if ((('남성' in df_policy_sex.loc[i, '지원대상'])|('전립선' in df_policy_sex.loc[i, '지원대상']))&(('노인' not in df_policy_sex.loc[i, '지원대상'])
    |('장애' not in df_policy_sex.loc[i, '지원대상'])|('가족' not in df_policy_sex.loc[i, '지원대상']))):
        df_policy_sex.loc[i,'tag_sex']='남성'
        
# 여성 분류
for i in range(len(df_policy_sex)):
    if ((('여성' in df_policy_sex.loc[i, '지원대상'])|('임신' in df_policy_sex.loc[i, '지원대상'])|('출산' in df_policy_sex.loc[i, '지원대상']))
    &('남성' not in df_policy_sex.loc[i, '지원대상'])&(('아동' not in df_policy_sex.loc[i, '지원대상'])
    |('노인' not in df_policy_sex.loc[i, '지원대상'])|('가족' not in df_policy_sex.loc[i, '지원대상']))):
        df_policy_sex.loc[i,'tag_sex']='여성'

# 무관
for i in range(len(df_policy_sex)):
    if ('0' in df_policy_sex.loc[i, 'tag_sex']):
        df_policy_sex.loc[i,'tag_sex']='무관'

# 'tag_sex'를 '태그-성별'로 지정
# df_policy['태그-성별'] = df_policy_sex['tag_sex']

## 14.가구원 ##

### A. TAG Code

In [ ]:
# 어드민 데이터의 '가구원'데이터 분포 확인
wello_compared['가구원'].value_counts()

무관                                  72607
주택 세대주, 주택 세대원                        903
무주택 세대주, 무주택 세대원, 주택 세대주, 주택 세대원      295
미혼                                    146
무주택 세대주, 무주택 세대원                      127
무주택 세대주, 주택 세대주                       123
무주택 세대원, 주택 세대원                        80
재혼                                     33
주택 세대주                                 28
무주택 세대주                                25
이혼                                     17
무주택 세대원                                14
무주택 세대주, 주택 세대주, 주택 세대원                11
무주택 세대주, 무주택 세대원, 주택 세대주                7
주택 세대원, 주택 세대주                          1
무관, 주택 세대주, 주택 세대원                      1
Name: 가구원, dtype: int64

In [ ]:
# 가구원 태그 추출을 위해 '정책ID', '정책서비스ID', '서비스명', '선정기준', '지원내용', '지원대상', '태그-가구원'
# df_policy_house 선언
df_policy_house = df_policy[['정책ID', '정책서비스ID', '서비스명', '선정기준', '지원내용', '지원대상', '태그-가구원']]
df_policy_house.head()

,정책ID,정책서비스ID,서비스명,선정기준,지원내용,지원대상,태그-가구원
0,1,000000465790,만 3~5세 유치원 유아 학비 지원(3~5세 누리과정 운영),1) 지원대상 : 어린이집을 이용하는 만 3~5세 아동\n2) 신청인 : 아동의 보...,"만 3~5세 누리과정 운영에 필요한 비용 지원 \n - 보육료, 담당교사 처우 개선...",지원대상 : 어린이집을 이용하는 만 3~5세 아동\n- 취학유예 : 만 6세아 초등...,NaN
1,2,105100000001,근로·자녀장려금,"○ 근로소득, 종교인소득 또는 종교인소득이 있고 아래 요건을 모두 충족하는 자\n\...",○ 전년도 연간 부부합산 총 급여액 등(사업소득과 근로소득의 합계)에 따라\n - ...,"○ 전년도에 정해진 범위의 근로소득, 사업소득 또는 종교인소득이 있고 근로장려금과 ...",NaN
2,3,116010000001,월세 자금보증,○ (주택도시기금 주거안정 월세대출 대상자 요건) 무주택 세대주(세대원 전원 포함)...,○ 주택도시기금 주거안정 월세대출 요건을 충족하는 대상자에 대해 월세자금보증 지원\...,○ 보증금 1억 원 이하 및 월세금 60만 원 이하인 월세 계약을 체결한 주택도시 ...,"주택 세대주,주택 세대원"
3,4,116010000027,‘힘내라 대한민국’특별운영자금 (산업은행),○ 산업은행 심사기준 충족 업체,○ 자금용도 : 운영자금\n\n○ 한도 : 기존 대출한도 外 일정범위내 특별한도 부...,"○ 지원대상 : 코로나19 등 질병, 자연재해 등 국가 재난 발생으로 피해를 입은 ...",NaN
4,5,119200000001,친환경 에너지 절감 장비 보급,"○ 기관ㆍ장비ㆍ설비사업의 사업자 선정은 다음의 순위로 하되, 동일 순위자의 세부 우...",○ 유류비 절감을 위한 유류절감장치 설치\n○ 저효율(육상기관 포함)기관 및 사...,"○ 어선 노후기관, 장비를 교체하고자 하는 연근해 어업인",NaN


In [ ]:
df_policy_house['tag_house'] = np.NaN
df_policy_house.head()

,정책ID,정책서비스ID,서비스명,선정기준,지원내용,지원대상,태그-가구원,tag_house
0,1,000000465790,만 3~5세 유치원 유아 학비 지원(3~5세 누리과정 운영),1) 지원대상 : 어린이집을 이용하는 만 3~5세 아동\n2) 신청인 : 아동의 보...,"만 3~5세 누리과정 운영에 필요한 비용 지원 \n - 보육료, 담당교사 처우 개선...",지원대상 : 어린이집을 이용하는 만 3~5세 아동\n- 취학유예 : 만 6세아 초등...,NaN,NaN
1,2,105100000001,근로·자녀장려금,"○ 근로소득, 종교인소득 또는 종교인소득이 있고 아래 요건을 모두 충족하는 자\n\...",○ 전년도 연간 부부합산 총 급여액 등(사업소득과 근로소득의 합계)에 따라\n - ...,"○ 전년도에 정해진 범위의 근로소득, 사업소득 또는 종교인소득이 있고 근로장려금과 ...",NaN,NaN
2,3,116010000001,월세 자금보증,○ (주택도시기금 주거안정 월세대출 대상자 요건) 무주택 세대주(세대원 전원 포함)...,○ 주택도시기금 주거안정 월세대출 요건을 충족하는 대상자에 대해 월세자금보증 지원\...,○ 보증금 1억 원 이하 및 월세금 60만 원 이하인 월세 계약을 체결한 주택도시 ...,"주택 세대주,주택 세대원",NaN
3,4,116010000027,‘힘내라 대한민국’특별운영자금 (산업은행),○ 산업은행 심사기준 충족 업체,○ 자금용도 : 운영자금\n\n○ 한도 : 기존 대출한도 外 일정범위내 특별한도 부...,"○ 지원대상 : 코로나19 등 질병, 자연재해 등 국가 재난 발생으로 피해를 입은 ...",NaN,NaN
4,5,119200000001,친환경 에너지 절감 장비 보급,"○ 기관ㆍ장비ㆍ설비사업의 사업자 선정은 다음의 순위로 하되, 동일 순위자의 세부 우...",○ 유류비 절감을 위한 유류절감장치 설치\n○ 저효율(육상기관 포함)기관 및 사...,"○ 어선 노후기관, 장비를 교체하고자 하는 연근해 어업인",NaN,NaN


In [ ]:
# 무주택 분류
for i in range(len(df_policy_house)):
    if ('무주택' in df_policy_house.loc[i, '지원대상']):
        df_policy_house.loc[i,'tag_house']='무주택'
    else:
        df_policy_house.loc[i,'tag_house']='무관'

In [ ]:
df_policy_house.head()

,정책ID,정책서비스ID,서비스명,선정기준,지원내용,지원대상,태그-가구원,tag_house
0,1,000000465790,만 3~5세 유치원 유아 학비 지원(3~5세 누리과정 운영),1) 지원대상 : 어린이집을 이용하는 만 3~5세 아동\n2) 신청인 : 아동의 보...,"만 3~5세 누리과정 운영에 필요한 비용 지원 \n - 보육료, 담당교사 처우 개선...",지원대상 : 어린이집을 이용하는 만 3~5세 아동\n- 취학유예 : 만 6세아 초등...,NaN,무관
1,2,105100000001,근로·자녀장려금,"○ 근로소득, 종교인소득 또는 종교인소득이 있고 아래 요건을 모두 충족하는 자\n\...",○ 전년도 연간 부부합산 총 급여액 등(사업소득과 근로소득의 합계)에 따라\n - ...,"○ 전년도에 정해진 범위의 근로소득, 사업소득 또는 종교인소득이 있고 근로장려금과 ...",NaN,무관
2,3,116010000001,월세 자금보증,○ (주택도시기금 주거안정 월세대출 대상자 요건) 무주택 세대주(세대원 전원 포함)...,○ 주택도시기금 주거안정 월세대출 요건을 충족하는 대상자에 대해 월세자금보증 지원\...,○ 보증금 1억 원 이하 및 월세금 60만 원 이하인 월세 계약을 체결한 주택도시 ...,"주택 세대주,주택 세대원",무관
3,4,116010000027,‘힘내라 대한민국’특별운영자금 (산업은행),○ 산업은행 심사기준 충족 업체,○ 자금용도 : 운영자금\n\n○ 한도 : 기존 대출한도 外 일정범위내 특별한도 부...,"○ 지원대상 : 코로나19 등 질병, 자연재해 등 국가 재난 발생으로 피해를 입은 ...",NaN,무관
4,5,119200000001,친환경 에너지 절감 장비 보급,"○ 기관ㆍ장비ㆍ설비사업의 사업자 선정은 다음의 순위로 하되, 동일 순위자의 세부 우...",○ 유류비 절감을 위한 유류절감장치 설치\n○ 저효율(육상기관 포함)기관 및 사...,"○ 어선 노후기관, 장비를 교체하고자 하는 연근해 어업인",NaN,무관


In [ ]:
df_policy_house['tag_house'].value_counts()

무관     87879
무주택      607
Name: tag_house, dtype: int64

In [ ]:
df_policy_house.loc[206]

정책ID                                                       207
정책서비스ID                                           138300000017
서비스명                                           한부모가족 임대주택 특별공급
선정기준         ○ 입주대상 : 한부모가족지원법상 지원 대상 한부모가족\n - 입주자모집공고일 기준...
지원내용                                         ○ 한부모가족 임대주택 특별공급
지원대상         < 특별공급 대상 >\n○ 입주대상 : 한부모가족지원법상 지원 대상 한부모가족\n ...
태그-가구원                           무주택 세대주,무주택 세대원,주택 세대주,주택 세대원
tag_house                                                  무주택
Name: 206, dtype: object

In [ ]:
df_policy_house['지원대상'].loc[206]

'< 특별공급 대상 >\n○ 입주대상 : 한부모가족지원법상 지원 대상 한부모가족\n - 입주자모집공고일 기준 무주택세대구성원(세대주 및 세대주와 동일 주민등록표등본에 등재되어 있는 직계존･비속인 세대원)\n\n○ 분양대상 : 한국토지주택공사 및 민간 주택사업자 등의 임대주택\n - 한국토지주택공사: 각 지역본부와 협의하여 공급 물량 파악\n - 민간 주택사업자 등: 사업계획 수립 및 승인 단계에서 공급 물량 파악'

### B. 정리 ###

* 가구원 분포가 무주택, 주택 세대주, 세대원, 무관으로 5개 값으로 구분되어 있음
* 주택 tag의 경우 무주택자에 대한 주택지원 정책이 주로 추진되기 때문에 '무주택'과 '무관'으로 분류하는게 더 효과적이다.
* '지원대상'열에서 '무주택'키워드를 포함한 값에 대해 '무주택' 태그 부여

### C. TAG Code 전체 ###

In [ ]:
# 가구원 태그 추출을 위해 '정책ID', '정책서비스ID', '서비스명', '선정기준', '지원내용', '지원대상', '태그-가구원'
# df_policy_house 선언
df_policy_house = df_policy[['정책ID', '정책서비스ID', '서비스명', '선정기준', '지원내용', '지원대상', '태그-가구원']]
df_policy_house.head()

df_policy_house['tag_house'] = np.NaN

# 무주택 분류
for i in range(len(df_policy_house)):
    if ('무주택' in df_policy_house.loc[i, '지원대상']):
        df_policy_house.loc[i,'tag_house']='무주택'
    else:
        df_policy_house.loc[i,'tag_house']='무관'

## 15.자녀 ##

A. TAG Code

B. 정리

C. TAG Code 전체

In [ ]:
wello_compared['신청절차'].value_counts()

오프라인         33056
온라인           8403
복합            3871
온라인, 오프라인     1416
오프라인, 온라인        7
복합, 오프라인         1
Name: 신청절차, dtype: int64

In [ ]:
wello_compared['가구원'].value_counts()

무관                                  72607
주택 세대주, 주택 세대원                        903
무주택 세대주, 무주택 세대원, 주택 세대주, 주택 세대원      295
미혼                                    146
무주택 세대주, 무주택 세대원                      127
무주택 세대주, 주택 세대주                       123
무주택 세대원, 주택 세대원                        80
재혼                                     33
주택 세대주                                 28
무주택 세대주                                25
이혼                                     17
무주택 세대원                                14
무주택 세대주, 주택 세대주, 주택 세대원                11
무주택 세대주, 무주택 세대원, 주택 세대주                7
무관, 주택 세대주, 주택 세대원                      1
주택 세대원, 주택 세대주                          1
Name: 가구원, dtype: int64

In [ ]:
wello_compared['학력'].value_counts()

무관                                                             58807
대학(원)재학                                                         1450
고등학교졸업                                                          1014
대학(원)졸업                                                          769
고등학교졸업, 대학(원)재학                                                  317
고등학교졸업, 대학(원)졸업                                                  152
대학(원)재학, 대학(원)휴학                                                  98
고등학교졸업, 대학(원)재학, 대학(원)졸업                                          64
고등학교졸업, 고등학교졸업                                                    58
대학(원)휴학                                                           55
대학(원)재학, 대학(원)졸업                                                  53
대학(원)졸업, 대학(원)졸업                                                  36
고등학교졸업, 고등학교졸업, 대학(원)재학, 대학(원)졸업                                  35
고등학교졸업, 고등학교졸업, 대학(원)졸업                                           11
고등학교졸업, 고등학교졸업, 대학(원)재학           

In [ ]:
wello_compared['결혼'].value_counts()

무관                56290
기혼                 1525
미혼                  607
이혼                  313
기혼, 재혼              206
미혼, 이혼              181
재혼                  114
이혼, 미혼               17
재혼, 기혼                4
기혼, 재혼, 이혼            3
미혼, 기혼, 이혼, 재혼        2
미혼, 기혼, 이혼            1
Name: 결혼, dtype: int64

In [ ]:
wello_compared['지원유형'].value_counts()

현물                                13713
기타                                 9006
현금, 현물                             8014
의료지원                               6133
현물, 현금                             5405
                                  ...  
민원, 의료지원, 현금, 현물, 서비스, 문화/여가지원        1
민원, 의료지원, 이용권, 서비스, 봉사/기부             1
서비스, 봉사/기부, 이용권                       1
현금, 기타, 서비스                           1
현물, 현금, 의료지원, 서비스                     1
Name: 지원유형, Length: 519, dtype: int64

# **II차. 데이터 Feature Engineering**

## **0. 데이터 불러오기**

In [ ]:
#나눔폰트 설치 후  런타임 재시작 
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree       
Reading state information... Done
fonts-nanum is already the newest version (20170925-1).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.
/usr/share/fonts: caching, new cache contents: 0 fonts, 1 dirs
/usr/share/fonts/truetype: caching, new cache contents: 0 fonts, 3 dirs
/usr/share/fonts/truetype/humor-sans: caching, new cache contents: 1 fonts, 0 dirs
/usr/share/fonts/truetype/liberation: caching, new cache contents: 16 fonts, 0 dirs
/usr/share/fonts/truetype/nanum: caching, new cache contents: 10 fonts, 0 dirs
/usr/local/share/fonts: caching, new cache contents: 0 fonts, 0 dirs
/root/.local/share/fonts: skipping, no such directory
/root/.fonts: skipping, no such directory
/var/cache/fontconfig: cleaning cache directory
/root/.cache/fontconfig: not cleaning non-existent cache directory
/root/.fontconfig: not cleaning non-existent cache directory
fc-cache: succeeded


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
# 경고무시
import warnings
warnings.filterwarnings('ignore')
# 정규표현식
import re

In [ ]:
df_enterprise = pd.read_csv('/content/drive/MyDrive/Wello+project_신희택,윤다솜/Data/df_enterprise.csv').iloc[:,1:]
df_enterprise.head()

,정책ID,정책서비스ID,서비스명,소관기관,소관기관유형,문의처전화번호,온라인신청사이트URL,생애주기,신청절차,선정기준,지원유형,서비스목적,지원내용,지원대상,신청기한_시작,신청기한_끝,상세조회URL,태그-신청절차,태그-자녀,태그-수집유형,태그-가구원,태그-성별,태그-학력,태그-직장,태그-결혼,태그-지원유형,태그-지역(시도),태그-관심상황특성,태그-자녀상세,태그-직장상세,태그-지원유형상세,태그-지역(시군구),태그-대상특성상세,태그-대상특성,태그-대상연령시작,태그-대상연령끝,태그-중위소득시작,태그-중위소득끝,support_content
0,4,116010000027,‘힘내라 대한민국’특별운영자금 (산업은행),금융위원회,중앙행정기관,1588-1500,http://www.kdb.co.kr,NaN,○ 3.26일부터 기업의 신청접수ㆍ지원 중\n\n ➡ 산업은행 전국 영업점을 방문하...,○ 산업은행 심사기준 충족 업체,현금(융자),코로나19로 피해를 입은 중소ㆍ중견기업 지원 프로그램,자금용도 운영자금 한도 기존 대출한도 外 일정범위내 특별한도 부여(중견기업 최대 1...,"코로나19 등 질병, 자연재해 등 국가 재난 발생으로 피해를 입은 중소ㆍ중견기업(의...",NaN,NaN,https://gov.kr/portal/rcvfvrSvc/dtlEx/11601000...,오프라인,NaN,NaN,NaN,NaN,NaN,중소기업,NaN,현금,NaN,NaN,NaN,NaN,NaN,NaN,NaN,기관/시설,0,99,0,0,자금용도 운영자금 한도 기존 대출한도 外 일정범위내 특별한도
1,10,119200000016,원양어업 경영자금 지원,해양수산부,중앙행정기관,02-5891606,-,NaN,○ 우편접수(서울특별시 서초구 논현로 83 한국원양산업협회)\n\n○ 사업시행지침 ...,○ 원양산업발전법에 따른 국적선사 및 합작선사,현금||현금(융자),"영세어업인(신고, 마을, 종묘생산 어업 등)의 금융부담완화 및 지속적인 어업활동 영...",어업경영자금 융자(수협은행 및 일선수협),"어업경영체(어업인, 어업법인 등)",NaN,NaN,https://gov.kr/portal/rcvfvrSvc/dtlEx/11920000...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,현금,NaN,NaN,NaN,"경영/사무/금융/보험직,경영/사무/금융/보험직,농업/축산업/어업,농업/축산업/어업,...",NaN,NaN,NaN,농축수산인,0,99,0,0,어업경영자금 융자
2,14,119200000024,수산장비 구입 지원,해양수산부,중앙행정기관,000-0000-0000,-,NaN,시군구청에 방문하여 신청,(지원자격) ｢수산업ㆍ어촌 발전 기본법｣ 및 ｢농어업경영체 육성 및 지원에 관한 법...,현금(융자),"수산장비 구입 지원을 통해 어업인 등의 경영 부담을 경감하고, 고품질·고성능 수산기...",융자한도액(1억원) 내에서 거래가격의 80% 지원,｢수산업ㆍ어촌 발전 기본법｣ 및 ｢농어업경영체 육성 및 지원에 관한 법률｣에서 규정...,NaN,NaN,https://gov.kr/portal/rcvfvrSvc/dtlEx/11920000...,오프라인,NaN,NaN,NaN,NaN,NaN,중소기업,NaN,현금,NaN,NaN,NaN,"경영/사무/금융/보험직,교육/법률/사회복지/경찰/소방/군인직/공무원직,경영/사무/금...",NaN,NaN,NaN,"농축수산인,기관/시설",0,99,0,0,융자한도액 내에서 거래가격의 80% 지원
3,16,119200000027,산지 및 소비지 유통자금 융자 지원,해양수산부,중앙행정기관,02-2240-2453\n02-2240-8525\n061-931-1647,-,NaN,우편 또는 방문신청\n수협중앙회 : 서울시 송파구 오금로 62 수협중앙회 경제기획부...,"○ 전년도수매실적, 중도매업 종사경력, 신규사업자, 소액신청자 등을 검토하여 사업자...",현금(융자),수산물을 원활히 공급하기 위한 유통자금 지원,"산지위판장 및 수산물 도매시장 어대금 결제자금, 직거래 자금 등 융자지원(금리 1....","산지위판장, 도매시장 법인, 중도매인 등",NaN,NaN,https://gov.kr/portal/rcvfvrSvc/dtlEx/11920000...,오프라인,NaN,NaN,NaN,NaN,NaN,중소기업,NaN,현금,NaN,NaN,NaN,NaN,NaN,NaN,NaN,기관/시설,0,99,0,0,"산지위판장 및 수산물 도매시장 어대금 결제자금, 직"
4,18,119200000044,수산경영인회생자금,해양수산부,중앙행정기관,1544-1515/1644-1515,-,NaN,수협은행 및 일선수협 방문하여 신청,○ 어가부채심사 및 경영평가위원회에서 회생가능여부 심사 후 회생가능 판정을 받은 어업인,현금||현금(융자),"건실하게 수산업을 영위하다가 재해, 어·패류질병, 적조, 수산물 가격의 급락 등 그...",어업경영자금 등 정책자금에 대해 저리의 수산경영회생자금 대환(수협은행 및 일선수협),"어업경영체(어업인, 어업법인 등)",NaN,NaN,https://gov.kr/portal/rcvfvrSvc/dtlEx/11920000...,오프라인,NaN,NaN,NaN,NaN,NaN,NaN,NaN,현금,NaN,NaN,NaN,"경영/사무/금융/보험직,경영/사무/금융/보험직,농업/축산업/어업,농업/축산업/어업,...",NaN,NaN,NaN,농축수산인,0,99,0,0,어업경영자금 등 정책자금에 대해 저리의 수산경영회생자금 대환


In [ ]:
df_policy = pd.read_excel('/content/drive/MyDrive/Wello+project_신희택,윤다솜/Data/policy_metetable(AIB전달용).xlsx')
df_policy = df_policy.sort_values(by='정책ID').reset_index(drop=True)
# 결론적으로 분석에 큰 지장이 없을 것 같아 소관기관유형.1과 소관기관.1을 삭제하도록함.
df_policy = df_policy.drop(['소관기관유형.1', '소관기관.1'],axis=1)
df_policy.head()

,정책ID,정책서비스ID,서비스명,소관기관,소관기관유형,문의처전화번호,온라인신청사이트URL,생애주기,신청절차,선정기준,지원유형,서비스목적,지원내용,지원대상,신청기한_시작,신청기한_끝,상세조회URL,태그-신청절차,태그-자녀,태그-수집유형,태그-가구원,태그-성별,태그-학력,태그-직장,태그-결혼,태그-지원유형,태그-지역(시도),태그-관심상황특성,태그-자녀상세,태그-직장상세,태그-지원유형상세,태그-지역(시군구),태그-대상특성상세,태그-대상특성,태그-대상연령시작,태그-대상연령끝,태그-중위소득시작,태그-중위소득끝
0,1,000000465790,만 3~5세 유치원 유아 학비 지원(3~5세 누리과정 운영),교육부,중앙행정기관,129,http://online.bokjiro.go.kr/,영·유아||아동·청소년||청년||중장년||노년기,신청ㆍ접수(읍면동사무소) - 보장 결정(시군구 사업팀),1) 지원대상 : 어린이집을 이용하는 만 3~5세 아동\n2) 신청인 : 아동의 보...,현금,누리과정이란 만 3~5세를 대상으로 하는 어린이집 표준보육과정과 유치원 교육과정을 ...,"만 3~5세 누리과정 운영에 필요한 비용 지원 \n - 보육료, 담당교사 처우 개선...",지원대상 : 어린이집을 이용하는 만 3~5세 아동\n- 취학유예 : 만 6세아 초등...,NaN,NaN,https://gov.kr/portal/rcvfvrSvc/dtlEx/00000046...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,현금,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,12,0,0
1,2,105100000001,근로·자녀장려금,국세청,중앙행정기관,NaN,https://www.hometax.go.kr,NaN,"○ 개벌 신청 안내받은 경우에는 ARS, 모바일 국세청 홈택스 앱, 인터넷 홈택스 ...","○ 근로소득, 종교인소득 또는 종교인소득이 있고 아래 요건을 모두 충족하는 자\n\...",현금,○ 근로자녀장려세제\n - 저소득계층의 빈곤 탈출을 지원하고 사회안전망을 확충하기 ...,○ 전년도 연간 부부합산 총 급여액 등(사업소득과 근로소득의 합계)에 따라\n - ...,"○ 전년도에 정해진 범위의 근로소득, 사업소득 또는 종교인소득이 있고 근로장려금과 ...",NaN,NaN,https://gov.kr/portal/rcvfvrSvc/dtlEx/10510000...,NaN,있음,NaN,NaN,NaN,NaN,NaN,NaN,현금,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,99,0,0
2,3,116010000001,월세 자금보증,금융위원회,중앙행정기관,-,-,NaN,"신청 방식 : 은행 방문\n신청 절차\n1. 은행 방문 및 상담 신청\n2. 보증,...",○ (주택도시기금 주거안정 월세대출 대상자 요건) 무주택 세대주(세대원 전원 포함)...,상담/법률지원||현금(융자),저소득층 등에게 월세자금보증을 하여 저소득층의 주거 안정성 제고,○ 주택도시기금 주거안정 월세대출 요건을 충족하는 대상자에 대해 월세자금보증 지원\...,○ 보증금 1억 원 이하 및 월세금 60만 원 이하인 월세 계약을 체결한 주택도시 ...,NaN,NaN,https://gov.kr/portal/rcvfvrSvc/dtlEx/11601000...,오프라인,NaN,NaN,"주택 세대주,주택 세대원",NaN,NaN,NaN,NaN,"현금,상담/법률지원",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,99,0,0
3,4,116010000027,‘힘내라 대한민국’특별운영자금 (산업은행),금융위원회,중앙행정기관,1588-1500,http://www.kdb.co.kr,NaN,○ 3.26일부터 기업의 신청접수ㆍ지원 중\n\n ➡ 산업은행 전국 영업점을 방문하...,○ 산업은행 심사기준 충족 업체,현금(융자),코로나19로 피해를 입은 중소ㆍ중견기업 지원 프로그램,○ 자금용도 : 운영자금\n\n○ 한도 : 기존 대출한도 外 일정범위내 특별한도 부...,"○ 지원대상 : 코로나19 등 질병, 자연재해 등 국가 재난 발생으로 피해를 입은 ...",NaN,NaN,https://gov.kr/portal/rcvfvrSvc/dtlEx/11601000...,오프라인,NaN,NaN,NaN,NaN,NaN,중소기업,NaN,현금,NaN,NaN,NaN,NaN,NaN,NaN,NaN,기관/시설,0,99,0,0
4,5,119200000001,친환경 에너지 절감 장비 보급,해양수산부,중앙행정기관,044-200-5526,-,NaN,해당 지자체(시군구청)에 방문하여 신청,"○ 기관ㆍ장비ㆍ설비사업의 사업자 선정은 다음의 순위로 하되, 동일 순위자의 세부 우...",현금,연근해 어업인을 대상으로 노후기관ㆍ장비ㆍ설비 대체 보급으로 안정적 조업기반 조성,○ 유류비 절감을 위한 유류절감장치 설치\n○ 저효율(육상기관 포함)기관 및 사...,"○ 어선 노후기관, 장비를 교체하고자 하는 연근해 어업인",NaN,NaN,https://gov.kr/portal/rcvfvrSvc/dtlEx/11920000...,오프라인,NaN,NaN,NaN,NaN,NaN,NaN,NaN,현금,NaN,NaN,NaN,"농업/축산업/어업,농업/축산업/어업,농업/축산업/어업",NaN,NaN,연근해-구획어업인,농축수산인,0,99,0,0


In [ ]:
# 많이 언급되는 지원대상, ○, :, \n, | 등
# 특히 중위 소득=> 중위소득으로 바꿔줘야함.
def clean_support_target(var):
  var = var.lstrip()
  var = var.replace('지원대상','').replace('○','').replace(':','').replace('\n','').replace('O','').replace('|','').replace('  ',' ').replace(' 소득','소득').replace('\xa0','').replace('*','')
  return var
df_policy['지원대상']=df_policy['지원대상'].apply(clean_support_target)
df_policy['지원내용']=df_policy['지원내용'].apply(clean_support_target)

In [ ]:
for i in range(len(df_policy)):
    if (('기업' in df_policy.loc[i, '지원대상'])|('중소' in df_policy.loc[i, '지원대상'])|('중견' in df_policy.loc[i, '지원대상'])
    |('법인' in df_policy.loc[i, '지원대상'])|('사업체' in df_policy.loc[i, '지원대상'])|('사업장' in df_policy.loc[i, '지원대상'])
    | ('기업' in df_policy.loc[i, '지원내용'])|('중소' in df_policy.loc[i, '지원내용'])|('중견' in df_policy.loc[i, '지원내용'])
    |('법인' in df_policy.loc[i, '지원내용'])|('사업체' in df_policy.loc[i, '지원내용'])|('사업장' in df_policy.loc[i, '지원내용'])):
        df_policy.loc[i, 'company_policy']=1

In [ ]:
df_enterprise = df_policy[df_policy['company_policy']==1].reset_index(drop=True)
df_enterprise

,정책ID,정책서비스ID,서비스명,소관기관,소관기관유형,문의처전화번호,온라인신청사이트URL,생애주기,신청절차,선정기준,지원유형,서비스목적,지원내용,지원대상,신청기한_시작,신청기한_끝,상세조회URL,태그-신청절차,태그-자녀,태그-수집유형,태그-가구원,태그-성별,태그-학력,태그-직장,태그-결혼,태그-지원유형,태그-지역(시도),태그-관심상황특성,태그-자녀상세,태그-직장상세,태그-지원유형상세,태그-지역(시군구),태그-대상특성상세,태그-대상특성,태그-대상연령시작,태그-대상연령끝,태그-중위소득시작,태그-중위소득끝,company_policy
0,4,116010000027,‘힘내라 대한민국’특별운영자금 (산업은행),금융위원회,중앙행정기관,1588-1500,http://www.kdb.co.kr,NaN,○ 3.26일부터 기업의 신청접수ㆍ지원 중\n\n ➡ 산업은행 전국 영업점을 방문하...,○ 산업은행 심사기준 충족 업체,현금(융자),코로나19로 피해를 입은 중소ㆍ중견기업 지원 프로그램,자금용도 운영자금 한도 기존 대출한도 外 일정범위내 특별한도 부여(중견기업 최대 ...,"코로나19 등 질병, 자연재해 등 국가 재난 발생으로 피해를 입은 중소ㆍ중견기업...",NaN,NaN,https://gov.kr/portal/rcvfvrSvc/dtlEx/11601000...,오프라인,NaN,NaN,NaN,NaN,NaN,중소기업,NaN,현금,NaN,NaN,NaN,NaN,NaN,NaN,NaN,기관/시설,0,99,0,0,1.0
1,10,119200000016,원양어업 경영자금 지원,해양수산부,중앙행정기관,02-5891606,-,NaN,○ 우편접수(서울특별시 서초구 논현로 83 한국원양산업협회)\n\n○ 사업시행지침 ...,○ 원양산업발전법에 따른 국적선사 및 합작선사,현금||현금(융자),"영세어업인(신고, 마을, 종묘생산 어업 등)의 금융부담완화 및 지속적인 어업활동 영...",어업경영자금 융자(수협은행 및 일선수협),"어업경영체(어업인, 어업법인 등)",NaN,NaN,https://gov.kr/portal/rcvfvrSvc/dtlEx/11920000...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,현금,NaN,NaN,NaN,"경영/사무/금융/보험직,경영/사무/금융/보험직,농업/축산업/어업,농업/축산업/어업,...",NaN,NaN,NaN,농축수산인,0,99,0,0,1.0
2,14,119200000024,수산장비 구입 지원,해양수산부,중앙행정기관,000-0000-0000,-,NaN,시군구청에 방문하여 신청,(지원자격) ｢수산업ㆍ어촌 발전 기본법｣ 및 ｢농어업경영체 육성 및 지원에 관한 법...,현금(융자),"수산장비 구입 지원을 통해 어업인 등의 경영 부담을 경감하고, 고품질·고성능 수산기...",융자한도액(1억원) 내에서 거래가격의 80% 지원,｢수산업ㆍ어촌 발전 기본법｣ 및 ｢농어업경영체 육성 및 지원에 관한 법률｣에서 규정...,NaN,NaN,https://gov.kr/portal/rcvfvrSvc/dtlEx/11920000...,오프라인,NaN,NaN,NaN,NaN,NaN,중소기업,NaN,현금,NaN,NaN,NaN,"경영/사무/금융/보험직,교육/법률/사회복지/경찰/소방/군인직/공무원직,경영/사무/금...",NaN,NaN,NaN,"농축수산인,기관/시설",0,99,0,0,1.0
3,16,119200000027,산지 및 소비지 유통자금 융자 지원,해양수산부,중앙행정기관,02-2240-2453\n02-2240-8525\n061-931-1647,-,NaN,우편 또는 방문신청\n수협중앙회 : 서울시 송파구 오금로 62 수협중앙회 경제기획부...,"○ 전년도수매실적, 중도매업 종사경력, 신규사업자, 소액신청자 등을 검토하여 사업자...",현금(융자),수산물을 원활히 공급하기 위한 유통자금 지원,"산지위판장 및 수산물 도매시장 어대금 결제자금, 직거래 자금 등 융자지원(금리 1...","산지위판장, 도매시장 법인, 중도매인 등",NaN,NaN,https://gov.kr/portal/rcvfvrSvc/dtlEx/11920000...,오프라인,NaN,NaN,NaN,NaN,NaN,중소기업,NaN,현금,NaN,NaN,NaN,NaN,NaN,NaN,NaN,기관/시설,0,99,0,0,1.0
4,18,119200000044,수산경영인회생자금,해양수산부,중앙행정기관,1544-1515/1644-1515,-,NaN,수협은행 및 일선수협 방문하여 신청,○ 어가부채심사 및 경영평가위원회에서 회생가능여부 심사 후 회생가능 판정을 받은 어업인,현금||현금(융자),"건실하게 수산업을 영위하다가 재해, 어·패류질병, 적조, 수산물 가격의 급락 등 그...",어업경영자금 등 정책자금에 대해 저리의 수산경영회생자금 대환(수협은행 및 일선수협),"어업경영체(어업인, 어업법인 등)",NaN,NaN,https://gov.kr/portal/rcvfvrSvc/dtlEx/11920000...,오프라인,NaN,NaN,NaN,NaN,NaN,NaN,NaN,현금,NaN,NaN,NaN,"경영/사무/금융/보험직,경영/사무/금융/보험직,농업/축산업/어업,농업/축산업/어업,...",NaN,NaN,NaN,농축수산인,0,99,0,0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19620,88452,B55128200018,강북구민운동장 대관료 감면,강북구도시관리공단,공공기관 등,02-944-2913,http://www.gangbukcmc.seoul.kr/,아동·청소년||청년||중장년||노년기,인터넷 온라인 접수,NaN,문화/여가지원\n\n감면/면제,"장애인 및 강북구관내(동호회, 학교, 단체)를 위한 대관료 감면","- 강북구 관내학교(미양중·고, 삼각산중,고) 80%- 장애인 30%- 강북구관내 ...","- 강북구 관내학교(미양중·고, 삼각산중,고)- 장애인- 강북구관내 청소년 단체 및...",NaN,NaN,http://www.gangbukcmc.seoul.kr/,온라인,NaN,NaN,NaN,NaN,NaN,"소상공인,중소기업",NaN,문화/여가지원,NaN,NaN,NaN,"협회/단체/기타서비스업,협회/단체/기타서비스업,협회/단체/기타서비스업",NaN,NaN,NaN,"장애인,기관/시설",13,18,0,0,1.0
19621,88453,B55275200007,바라산 자연휴양림 시설 이용료 감면,의왕도시공사,공공기관 등,031-8086-7482,https://strawberry.mainticket.co.kr/strawberry...,아동·청소년||청년||중장년||노년기,○ 시설이용예약 및 결제 후 감면을 원할 시 바라산 자연휴양림 매표소에서 증빙서류 ...,NaN,문화/여가지원\n\n감면/면제,"바라산 자연휴양림, 산림휴양관 이용료 감면(장애인, 국가유공자,의왕시민, 의왕시 소...","바라산 자연휴양림 이용료 50% 감면 - 장애인, 국가유공자, 19세 미만 자녀 ...","바라산 자연휴양림 이용자 - 장애인, 국가유공자, 19세 이상 3명이상 다자녀, ...",NaN,NaN,https://www.foresttrip.go.kr/indvz/main.do?hmp...,NaN,있음,NaN,NaN,NaN,NaN,중소기업,NaN,문화/여가지원,NaN,NaN,NaN,"협회/단체/기타서비스업,협회/단체/기타서비스업,협회/단체/기타서비스업",NaN,NaN,NaN,"장애인,국가유공자,다자녀가정,기관/시설",0,99,0,0,1.0
19622,88471,611000014088,농작물 병해충 예찰방제 지원,경기도 이천시,시군구,063-238-1048,NaN,청년||중장년||노년기,NaN,병해충 정보가 필요한 모든 국민 대상,정보제공,농작물 병해충 공공데이터를 국민에게 제공하여 농작물 피해 최소화,지역별·병해충별 발생 정보를 활용한 맞춤형 방제정보 제공창업·경영>업종별민원>농업/...,"농업인, 산업체, 공공기관, 일반인 병해충 정보가 필요한 모든 국민 대상기업인중소...",NaN,NaN,NaN,N

In [ ]:
del df_enterprise['company_policy']

In [ ]:
# 1.(토양/산림) & 사용료
a1 = df_enterprise[df_enterprise['지원내용'].apply(lambda x: ('사용료' in x) & ('토양/산림' in x))]['정책ID'].values
# 2. 사용료 & 문화예술
a2 = df_enterprise[df_enterprise['지원내용'].apply(lambda x: ('이용료' in x) & ('문화예술' in x))]['정책ID'].values
a3 =df_enterprise[df_enterprise['지원내용'].apply(lambda x: ('사용료' in x) & ('문화예술' in x))]['정책ID'].values
# 3. 사업 | 산업 & 공공요금
a4 = df_enterprise[df_enterprise['지원내용'].apply(lambda x: ('사업' in x) & ('공공요금' in x))]['정책ID'].values
a5 = df_enterprise[df_enterprise['지원내용'].apply(lambda x: ('산업' in x) & ('공공요금' in x))]['정책ID'].values
# 키워드: 무료입장, 병역, 결혼, 운동, 임신, 여가, 관광, 여행, 출산, 자연휴양림, 봉사활동, 공연,문화, 장기기증, 주차, 장례/화장, 장애인, 
a6 = df_enterprise[(df_enterprise['지원내용'].apply(lambda x: ('무료입장' in x)&('소상공인' not in x)&('기업' not in x)&('사업체' not in x)&('법인' not in x)))|(df_enterprise['지원대상'].apply(lambda x: ('무료입장' in x)&('소상공인' not in x)&('기업' not in x)&('사업체' not in x)&('법인' not in x)))]['정책ID'].values
a7 = df_enterprise[(df_enterprise['지원내용'].apply(lambda x: '병역' in x))|(df_enterprise['지원대상'].apply(lambda x: '병역' in x))]['정책ID'].values
a8 = df_enterprise[(df_enterprise['지원내용'].apply(lambda x: ('결혼' in x)&('소상공인' not in x)&('기업' not in x)&('사업체' not in x)&('법인' not in x)))|(df_enterprise['지원대상'].apply(lambda x: ('결혼' in x)&('소상공인' not in x)&('기업' not in x)&('사업체' not in x)&('법인' not in x)))]['정책ID'].values
a9 = df_enterprise[(df_enterprise['지원내용'].apply(lambda x: ('임신' in x)&('소상공인' not in x)&('기업' not in x)&('사업체' not in x)&('법인' not in x)))|(df_enterprise['지원대상'].apply(lambda x: ('임신' in x)&('소상공인' not in x)&('기업' not in x)&('사업체' not in x)&('법인' not in x)))]['정책ID'].values
a10 = df_enterprise[(df_enterprise['지원내용'].apply(lambda x: ('운동' in x)&('소상공인' not in x)&('기업' not in x)&('사업체' not in x)&('법인' not in x)))|(df_enterprise['지원대상'].apply(lambda x: ('운동' in x)&('소상공인' not in x)&('기업' not in x)&('사업체' not in x)&('법인' not in x)))]['정책ID'].values
a11 = df_enterprise[(df_enterprise['지원내용'].apply(lambda x: ('여가' in x)&('소상공인' not in x)&('기업' not in x)&('사업체' not in x)&('법인' not in x)))|(df_enterprise['지원대상'].apply(lambda x: ('여가' in x)&('소상공인' not in x)&('기업' not in x)&('사업체' not in x)&('법인' not in x)))]['정책ID'].values
a12 = df_enterprise[(df_enterprise['지원내용'].apply(lambda x: ('관광' in x)&('소상공인' not in x)&('기업' not in x)&('사업체' not in x)&('법인' not in x)))|(df_enterprise['지원대상'].apply(lambda x: ('관광' in x)&('소상공인' not in x)&('기업' not in x)&('사업체' not in x)&('법인' not in x)))]['정책ID'].values
a13 = df_enterprise[(df_enterprise['지원내용'].apply(lambda x: ('여행' in x)&('소상공인' not in x)&('기업' not in x)&('사업체' not in x)&('법인' not in x)))|(df_enterprise['지원대상'].apply(lambda x: ('여행' in x)&('소상공인' not in x)&('기업' not in x)&('사업체' not in x)&('법인' not in x)))]['정책ID'].values
a14 = df_enterprise[(df_enterprise['지원내용'].apply(lambda x: ('출산' in x)&('소상공인' not in x)&('기업' not in x)&('사업체' not in x)&('법인' not in x)))|(df_enterprise['지원대상'].apply(lambda x: ('출산' in x)&('소상공인' not in x)&('기업' not in x)&('사업체' not in x)&('법인' not in x)))]['정책ID'].values
a15 = df_enterprise[(df_enterprise['지원내용'].apply(lambda x: '자연휴양림' in x))|(df_enterprise['지원대상'].apply(lambda x: '자연휴양림' in x))]['정책ID'].values
a16 = df_enterprise[(df_enterprise['지원내용'].apply(lambda x: '봉사활동' in x))|(df_enterprise['지원대상'].apply(lambda x: '봉사활동' in x))]['정책ID'].values
a17 = df_enterprise[(df_enterprise['지원내용'].apply(lambda x: ('문화' in x)&('소상공인' not in x)&('기업' not in x)&('사업체' not in x)&('법인' not in x)))|(df_enterprise['지원대상'].apply(lambda x: ('문화' in x)&('소상공인' not in x)&('기업' not in x)&('사업체' not in x)&('법인' not in x)))]['정책ID'].values
a19 = df_enterprise[(df_enterprise['지원내용'].apply(lambda x: ('공연' in x)&('소상공인' not in x)&('기업' not in x)&('사업체' not in x)&('법인' not in x)))&(df_enterprise['지원대상'].apply(lambda x: ('공연' in x)&('소상공인' not in x)&('기업' not in x)&('사업체' not in x)&('법인' not in x)))]['정책ID'].values
a20 = df_enterprise[(df_enterprise['지원내용'].apply(lambda x: '장기기증' in x))|(df_enterprise['지원대상'].apply(lambda x: '장기기증' in x))]['정책ID'].values
a21 = df_enterprise[(df_enterprise['지원내용'].apply(lambda x: '주차' in x))|(df_enterprise['지원대상'].apply(lambda x: '주차' in x))]['정책ID'].values
a22 = df_enterprise[(df_enterprise['지원내용'].apply(lambda x: '장례/화장' in x))|(df_enterprise['지원대상'].apply(lambda x: '장례/화장' in x))]['정책ID'].values
a23 = df_enterprise[(df_enterprise['지원내용'].apply(lambda x: ('장애인' in x)&('소상공인' not in x)&('기업' not in x)&('사업체' not in x)&('법인' not in x)))&(df_enterprise['지원대상'].apply(lambda x: ('장애인' in x)&('소상공인' not in x)&('기업' not in x)&('사업체' not in x)&('법인' not in x)))]['정책ID'].values
a24 = df_enterprise[(df_enterprise['지원내용'].apply(lambda x: ('주민' in x)&('소상공인' not in x)&('기업' not in x)&('사업체' not in x)&('법인' not in x)))&(df_enterprise['지원대상'].apply(lambda x: ('주민' in x)&('소상공인' not in x)&('기업' not in x)&('사업체' not in x)&('법인' not in x)))]['정책ID'].values
a25 = df_enterprise[(df_enterprise['지원내용'].apply(lambda x: ('시민' in x)&('소상공인' not in x)&('기업' not in x)&('사업체' not in x)&('법인' not in x)))&(df_enterprise['지원대상'].apply(lambda x: ('시민' in x)&('소상공인' not in x)&('기업' not in x)&('사업체' not in x)&('법인' not in x)))]['정책ID'].values

In [ ]:
not_enterprise = np.unique(np.concatenate((a1,a2,a3,a4,a5,a6,a7,a8,a9,a10,a11,a12,a13,a14,a15,a16,a17,a18,a19,a20,a21,a22,a23,a24,a25)))
len(not_enterprise)

2689

In [ ]:
# 기업 정책이 아닌 데이터 보기
df_enterprise[df_enterprise['정책ID'].apply(lambda x: x in not_enterprise)]['지원대상'].unique()

array([' 임업인 산림이나 환경 관련 단체의 회원 산림 관련 교육 이수자 산림청장이 설립허가한 법인의 회원 그 밖에 산림보호 관련 활동한 실적이 있는 자',
       ' 장애인을 고용하여 사업을 운영하고 있거나 사업을 운영하고자 하는 모든 사업주',
       ' 중증 장애인을 고용하여 기준에서 정한 자격을 갖춘 작업지도원을 위촉, 선임, 배치하여 작업 지도를 실시한 사업주',
       ...,
       '◌ 「도로교통법 」제2조세22호에 따른 긴급자동차◌ 「국가유공자 등 예우 및 지원에 관한 법률」에 의한 국가유공자 증서를 소지한 자가 본인의 자동차를 직접 운전하거나 장애 정도가 심하여 동승하고 타인에게 대리운전 시(국가유공자 수권유족 포함)◌ 성실납세자 인증서를 받은 자로서 교부일부터 1년간◌ 「자동차 관리법」에 따른 경형 자동차 및 「환경친화적 자동차의 개발 및 보급 촉진에 관한 법률」에 따른 환경친화적 자동차 ◌ 「장애인복지법」에 따른 장애인복지카드(장애 정도가 심한 장애인)를 소지한 자의 자가운전 또는 동승 차량◌ 「장애인복지법」에 따른 장애인복지카드(장애 정도가 심하지 않은 장애인)를 소지한 자의 자가운전 또는 동승 차량◌ 「5·18만주 유공자 예우에 관한 법률 」제4조세2호에 따른 5·18민주화운동 부상자 증서를 소지한 자가 본인의 자동차를 직접 운전하거나 장애 정도가 심하여 동승하고 타인이 대리운전 시◌ 「전통시장 및 상점가 육성을 위한 특별법 」제19조 제1항에 따른 주차장에 상인 및 고객이 주차하는 경우◌ 「수도권 대기환경 개선에 관한 특별법 」제28조 및 같은 법 시행규칙 제38조에 따라 저공해자동차 표지를 부착한 차량◌ 자가용 부제 및 함께 타기 표지부착 차량◌ 「공직선거법 」제2조 및 「지방교육자치에 관한 법률 」제8조, 제22조에 따른 선거에 있어서 투표에 참여한 사람◌ 3자녀 이상 다자녀 세대 차랑◌ 「모자보건법 」제2조세1호에 따른 임산부◌ 여주시에 주민등록이 되어 있는 고령 노인(75세 이상)이 자동차를 직접 운

In [ ]:
# 기업 정책 데이터 보기
df_enterprise[df_enterprise['정책ID'].apply(lambda x: x not in not_enterprise)]['지원내용'].unique()

array([' 자금용도 운영자금 한도 기존 대출한도 外 일정범위내 특별한도 부여(중견기업 최대 1,200억원, 중소기업 최대 600억원) ※ 매출액 등에 따른 기존 운영자금 대출한도 외 추가한도 부여 우대조건 최대 0.90%p 금리 우대, 심사절차 간소화 등',
       ' 어업경영자금 융자(수협은행 및 일선수협)', '융자한도액(1억원) 내에서 거래가격의 80% 지원', ...,
       ' 대상별 교육연구과정 - 사업체 임직원 · 장애인 직업생활상담원 양성 직장 내 장애인 상담 및 고충 처리 업무를 수행할 장애인 직업생활상담원 양성 - 장애인 고용 종사자 · 근로지원인 양성 중증장애인 근로자의 업무수행을 지원하는 근로지원인 양성 · 직무지도원 양성 중증장애인의 취업 전 훈련과정과 취업 후 업무적응을 지원하는 직무지도원 양성 · 취업코칭 운영자 양성 취업코칭 프로그램을 운영하는 강사의 전문역량 강화 · 직업능력평가도구 활용 직업능력평가 서비스의 질적 향상을 위한 공단 개발 직업능력평가도구의 활용 교육 · 동료지원가 양성 비경제활동 및 실업에 상태에 있는 중증장애인의 경제활동 참여유도 - 내부 직원 공단 업무수행에 필요한 내부 직무교육 과정 운영 - 사이버 과정 장애인고용종사자의 전문성 향상과 역량강화를 위한 장애인고용 전문 온라인 교육콘텐츠 제공(장애인고용이론·실무과정 등) 교육장소 한국장애인고용공단 고용개발원(경기도 성남시 분당구 구미로173번길 59(구미동)) ※ 다른 장소에서 교육을 운영하는 경우 별도 공지합니다취업·직장>교육/훈련>직업교육훈련/취업준비교육취업·직장>구인/구직활동>구인/고용취업·직장>근로복지>직장생활지원',
       '지역별·병해충별 발생 정보를 활용한 맞춤형 방제정보 제공창업·경영>업종별민원>농업/축산업/어업창업·경영>업종별민원>공공/행정/국방/방위산업건강·의료·사망>예방/검사>방역/검역',
       ' 환경보전, 농촌공동체 유지, 식품안정 등 공익기능 증진20210401~20210531창업·경영>업종별민원>농업/축산업

In [ ]:
df_enterprise=df_enterprise[df_enterprise['정책ID'].apply(lambda x: x not in not_enterprise)].reset_index(drop=True)

In [ ]:
df_enterprise.shape

(16936, 38)

In [ ]:
df_enterprise.columns

Index(['정책ID', '정책서비스ID', '서비스명', '소관기관', '소관기관유형', '문의처전화번호', '온라인신청사이트URL',
       '생애주기', '신청절차', '선정기준', '지원유형', '서비스목적', '지원내용', '지원대상', '신청기한_시작',
       '신청기한_끝', '상세조회URL', '태그-신청절차', '태그-자녀', '태그-수집유형', '태그-가구원', '태그-성별',
       '태그-학력', '태그-직장', '태그-결혼', '태그-지원유형', '태그-지역(시도)', '태그-관심상황특성',
       '태그-자녀상세', '태그-직장상세', '태그-지원유형상세', '태그-지역(시군구)', '태그-대상특성상세', '태그-대상특성',
       '태그-대상연령시작', '태그-대상연령끝', '태그-중위소득시작', '태그-중위소득끝'],
      dtype='object')

In [ ]:
df_enterprise_raw=df_enterprise.copy()

In [ ]:
df_enterprise[['정책ID','정책서비스ID','서비스명','소관기관','소관기관유형','신청절차','태그-신청절차','온라인신청사이트URL','지원유형','지원내용','지원대상']]

,정책ID,정책서비스ID,서비스명,소관기관,소관기관유형,신청절차,태그-신청절차,온라인신청사이트URL,지원유형,지원내용,지원대상
0,4,116010000027,‘힘내라 대한민국’특별운영자금 (산업은행),금융위원회,중앙행정기관,○ 3.26일부터 기업의 신청접수ㆍ지원 중\n\n ➡ 산업은행 전국 영업점을 방문하...,오프라인,http://www.kdb.co.kr,현금(융자),자금용도 운영자금 한도 기존 대출한도 外 일정범위내 특별한도 부여(중견기업 최대 ...,"코로나19 등 질병, 자연재해 등 국가 재난 발생으로 피해를 입은 중소ㆍ중견기업..."
1,10,119200000016,원양어업 경영자금 지원,해양수산부,중앙행정기관,○ 우편접수(서울특별시 서초구 논현로 83 한국원양산업협회)\n\n○ 사업시행지침 ...,NaN,-,현금||현금(융자),어업경영자금 융자(수협은행 및 일선수협),"어업경영체(어업인, 어업법인 등)"
2,14,119200000024,수산장비 구입 지원,해양수산부,중앙행정기관,시군구청에 방문하여 신청,오프라인,-,현금(융자),융자한도액(1억원) 내에서 거래가격의 80% 지원,｢수산업ㆍ어촌 발전 기본법｣ 및 ｢농어업경영체 육성 및 지원에 관한 법률｣에서 규정...
3,16,119200000027,산지 및 소비지 유통자금 융자 지원,해양수산부,중앙행정기관,우편 또는 방문신청\n수협중앙회 : 서울시 송파구 오금로 62 수협중앙회 경제기획부...,오프라인,-,현금(융자),"산지위판장 및 수산물 도매시장 어대금 결제자금, 직거래 자금 등 융자지원(금리 1...","산지위판장, 도매시장 법인, 중도매인 등"
4,18,119200000044,수산경영인회생자금,해양수산부,중앙행정기관,수협은행 및 일선수협 방문하여 신청,오프라인,-,현금||현금(융자),어업경영자금 등 정책자금에 대해 저리의 수산경영회생자금 대환(수협은행 및 일선수협),"어업경영체(어업인, 어업법인 등)"
...,...,...,...,...,...,...,...,...,...,...,...
16931,88428,PTR000050134,장애인 직업생활상담원 양성교육,한국장애인고용공단,공공기관 등,○ 전화\n - 한국장애인고용공단 콜센터(☎ 031-728-7000)\n\n○ 상담...,온라인,https://cyedu.kead.or.kr/,정보제공,장애인 직업생활상담원 제도란? - 장애인 고용 촉진 및 직업재활법 제75조에 의거...,상시 20명 이상의 장애인 근로자를 고용하고 있는 사업주기업인중소기업/소상공인
16932,88430,PTR000050137,장애인 고용종사자 교육연수,한국장애인고용공단,공공기관 등,EDI 사이버연수원(http://cyedu.kead.or.kr )에서 회원가입 후 ...,온라인,http://cyedu.kead.or.kr,교육/용역,대상별 교육연구과정 - 사업체 임직원 · 장애인 직업생활상담원 양성 직장 내 장애...,"사업체 임직원, 장애인 고용종사자, 공단 임직원, 일반인 등구직자근로자기업인중소기업..."
16933,88471,611000014088,농작물 병해충 예찰방제 지원,경기도 이천시,시군구,NaN,NaN,NaN,정보제공,지역별·병해충별 발생 정보를 활용한 맞춤형 방제정보 제공창업·경영>업종별민원>농업/...,"농업인, 산업체, 공공기관, 일반인 병해충 정보가 필요한 모든 국민 대상기업인중소..."
16934,88479,369000000418,기본형 공익직접지불금 지원,울산광역시 중구,시군구,"○ 지원자격과 요건을 갖추고, 신청 제한기준에 해당되지 않을 경우, 사업계획서 등 ...",NaN,NaN,NaN,"환경보전, 농촌공동체 유지, 식품안정 등 공익기능 증진20210401~202105...",사업대상 지역에서 보험대상 목적물을 재배(사육)하는 농업인 또는 법인기업인중소기업...


In [ ]:
df_enterprise = df_enterprise[['정책ID','정책서비스ID','서비스명','소관기관','소관기관유형','신청절차','태그-신청절차','온라인신청사이트URL','지원유형','지원내용','지원대상']]

## **1. 기업관련태그값추출**

### **1. 기업규모**

#### **A. TAG CODE**

In [ ]:
for i in range(len(df_enterprise)):
  if '소기업' in df_enterprise.loc[i,'지원내용']:
    df_enterprise.loc[i,'소기업']= 1
  else:
    if '소기업' in df_enterprise.loc[i,'지원대상']:
      df_enterprise.loc[i,'소기업']= 1
    else:
      df_enterprise.loc[i,'소기업']= 0

In [ ]:
for i in range(len(df_enterprise)):
  if '중기업' in df_enterprise.loc[i,'지원내용']:
    df_enterprise.loc[i,'중기업']= 1
  else:
    if '중기업' in df_enterprise.loc[i,'지원대상']:
      df_enterprise.loc[i,'중기업']= 1
    else:
      df_enterprise.loc[i,'중기업']= 0

In [ ]:
for i in range(len(df_enterprise)):
  if ('중소기업' in df_enterprise.loc[i,'지원내용'])|('중소 기업' in df_enterprise.loc[i,'지원내용']):
    df_enterprise.loc[i,'소기업']= 1
    df_enterprise.loc[i,'중기업']= 1
  else:
    if '중소기업' in df_enterprise.loc[i,'지원대상']:
      df_enterprise.loc[i,'소기업']= 1
      df_enterprise.loc[i,'중기업']= 1

In [ ]:
for i in range(len(df_enterprise)):
  if '중견기업' in df_enterprise.loc[i,'지원내용']:
    df_enterprise.loc[i,'중견기업']= 1
  else:
    if '중견기업' in df_enterprise.loc[i,'지원대상']:
      df_enterprise.loc[i,'중견기업']= 1
    else:
      df_enterprise.loc[i,'중견기업']= 0

In [ ]:
for i in range(len(df_enterprise)):
  if '대기업' in df_enterprise.loc[i,'지원내용']:
    df_enterprise.loc[i,'대기업']= 1
  else:
    if '대기업' in df_enterprise.loc[i,'지원대상']:
      df_enterprise.loc[i,'대기업']= 1
    else:
      df_enterprise.loc[i,'대기업']= 0

In [ ]:
for i in range(len(df_enterprise)):
  if ('소상공인' in df_enterprise.loc[i,'지원내용'])|('소상 공인' in df_enterprise.loc[i,'지원내용']):
    df_enterprise.loc[i,'소상공인']= 1
  else:
    if '소상공인' in df_enterprise.loc[i,'지원대상']:
      df_enterprise.loc[i,'소상공인']= 1
    else:
      df_enterprise.loc[i,'소상공인']= 0

In [ ]:
print('소기업: ', df_enterprise['소기업'].value_counts().values[1])
print('중기업: ', df_enterprise['중기업'].value_counts().values[1])
print('중견기업: ', df_enterprise['중견기업'].value_counts().values[1])
print('대기업: ', df_enterprise['대기업'].value_counts().values[1])
print('소상공인: ', df_enterprise['소상공인'].value_counts().values[1])

소기업:  8212
중기업:  8229
중견기업:  94
대기업:  116
소상공인:  8291


In [ ]:
# 기업규모 값 만들기
df_enterprise['기업규모']=''

for i in range(len(df_enterprise)):
  values = df_enterprise.loc[i,'소기업':'소상공인'].values
  if values.sum()==0 :
    df_enterprise.loc[i,'기업규모']= '무관'
  else:
    for j in range(len(values)):
      if j == 0:
        if values[j]==1:
          df_enterprise.loc[i,'기업규모']+='소기업'
      elif j == 1:
        if values[j]==1:
          if df_enterprise.loc[i,'기업규모']=='':
            df_enterprise.loc[i,'기업규모']+='중기업'
          else:
            df_enterprise.loc[i,'기업규모']+='/중기업'
  
      elif j == 2:
        if values[j]==1:
          if df_enterprise.loc[i,'기업규모']=='':
            df_enterprise.loc[i,'기업규모']+='중견기업'
          else:
            df_enterprise.loc[i,'기업규모']+='/중견기업'
      elif j == 3:
        if values[j]==1:
          if df_enterprise.loc[i,'기업규모']=='':
            df_enterprise.loc[i,'기업규모']+='대기업'
          else:
            df_enterprise.loc[i,'기업규모']+='/대기업'
      elif j == 4:
        if values[j]==1:
          if df_enterprise.loc[i,'기업규모']=='':
            df_enterprise.loc[i,'기업규모']+='소상공인'
          else:
            df_enterprise.loc[i,'기업규모']+='/소상공인'
      
            
del df_enterprise['소기업']
del df_enterprise['중기업']
del df_enterprise['중견기업']
del df_enterprise['대기업']
del df_enterprise['소상공인']

#### **B. 정리**
1. 분류: 소기업/중기업/중견기업/대기업/소상공인
2. 기업 규모가 따로 정해져 있지 않은 경우 총 7993개에 해당한다.

In [ ]:
df_enterprise['기업규모'].unique()

array(['소기업/중기업/중견기업', '무관', '소기업/중기업', '중견기업/대기업', '중견기업', '소상공인',
       '소기업/중기업/대기업', '대기업', '소기업/중기업/중견기업/대기업', '소기업/중기업/소상공인',
       '소기업/중기업/중견기업/대기업/소상공인', '소기업/중기업/중견기업/소상공인', '소기업/중기업/대기업/소상공인',
       '소기업/소상공인', '소기업', '중기업', '소기업/대기업'], dtype=object)

#### **C. TAG CODE 전체**

In [ ]:
for i in range(len(df_enterprise)):
  if '소기업' in df_enterprise.loc[i,'지원내용']:
    df_enterprise.loc[i,'소기업']= 1
  else:
    if '소기업' in df_enterprise.loc[i,'지원대상']:
      df_enterprise.loc[i,'소기업']= 1
    else:
      df_enterprise.loc[i,'소기업']= 0
for i in range(len(df_enterprise)):
  if '중기업' in df_enterprise.loc[i,'지원내용']:
    df_enterprise.loc[i,'중기업']= 1
  else:
    if '중기업' in df_enterprise.loc[i,'지원대상']:
      df_enterprise.loc[i,'중기업']= 1
    else:
      df_enterprise.loc[i,'중기업']= 0
for i in range(len(df_enterprise)):
  if ('중소기업' in df_enterprise.loc[i,'지원내용'])|('중소 기업' in df_enterprise.loc[i,'지원내용']):
    df_enterprise.loc[i,'소기업']= 1
    df_enterprise.loc[i,'중기업']= 1
  else:
    if '중소기업' in df_enterprise.loc[i,'지원대상']:
      df_enterprise.loc[i,'소기업']= 1
      df_enterprise.loc[i,'중기업']= 1
for i in range(len(df_enterprise)):
  if '중견기업' in df_enterprise.loc[i,'지원내용']:
    df_enterprise.loc[i,'중견기업']= 1
  else:
    if '중견기업' in df_enterprise.loc[i,'지원대상']:
      df_enterprise.loc[i,'중견기업']= 1
    else:
      df_enterprise.loc[i,'중견기업']= 0
for i in range(len(df_enterprise)):
  if '대기업' in df_enterprise.loc[i,'지원내용']:
    df_enterprise.loc[i,'대기업']= 1
  else:
    if '대기업' in df_enterprise.loc[i,'지원대상']:
      df_enterprise.loc[i,'대기업']= 1
    else:
      df_enterprise.loc[i,'대기업']= 0
for i in range(len(df_enterprise)):
  if ('소상공인' in df_enterprise.loc[i,'지원내용'])|('소상 공인' in df_enterprise.loc[i,'지원내용']):
    df_enterprise.loc[i,'소상공인']= 1
  else:
    if '소상공인' in df_enterprise.loc[i,'지원대상']:
      df_enterprise.loc[i,'소상공인']= 1
    else:
      df_enterprise.loc[i,'소상공인']= 0
# 기업규모 값 만들기
df_enterprise['기업규모']=''

for i in range(len(df_enterprise)):
  values = df_enterprise.loc[i,'소기업':'소상공인'].values
  if values.sum()==0 :
    df_enterprise.loc[i,'기업규모']= '무관'
  else:
    for j in range(len(values)):
      if j == 0:
        if values[j]==1:
          df_enterprise.loc[i,'기업규모']+='소기업'
      elif j == 1:
        if values[j]==1:
          if df_enterprise.loc[i,'기업규모']=='':
            df_enterprise.loc[i,'기업규모']+='중기업'
          else:
            df_enterprise.loc[i,'기업규모']+='/중기업'
  
      elif j == 2:
        if values[j]==1:
          if df_enterprise.loc[i,'기업규모']=='':
            df_enterprise.loc[i,'기업규모']+='중견기업'
          else:
            df_enterprise.loc[i,'기업규모']+='/중견기업'
      elif j == 3:
        if values[j]==1:
          if df_enterprise.loc[i,'기업규모']=='':
            df_enterprise.loc[i,'기업규모']+='대기업'
          else:
            df_enterprise.loc[i,'기업규모']+='/대기업'
      elif j == 4:
        if values[j]==1:
          if df_enterprise.loc[i,'기업규모']=='':
            df_enterprise.loc[i,'기업규모']+='소상공인'
          else:
            df_enterprise.loc[i,'기업규모']+='/소상공인'
      
            
del df_enterprise['소기업']
del df_enterprise['중기업']
del df_enterprise['중견기업']
del df_enterprise['대기업']
del df_enterprise['소상공인']

### **2. 산업유형**

#### A. TAG CODE

In [ ]:
industry_criteria = pd.read_excel('/content/drive/MyDrive/Wello+project_신희택,윤다솜/Data/(10차)산업분류_대분류항목값.xlsx').fillna(0)
industry_criteria

,대분류,코드,항목명
0,"농업, 임업 및 어업",11,작물 재배업
1,0,12,축산업
2,0,13,작물재배 및 축산 복합농업
3,0,14,작물재배 및 축산 관련 서비스업
4,0,15,수렵 및 관련 서비스업
...,...,...,...
227,0,969,그 외 기타 개인 서비스업
228,가구 내 고용활동 및 달리 분류되지 않은 자가 소비 생산활동,970,가구 내 고용활동
229,0,981,자가 소비를 위한 가사 생산 활동
230,0,982,자가 소비를 위한 가사 서비스 활동


In [ ]:
# 대분류 항목명에 따른 딕셔너리 만들기
dict_industry_criteria=dict()
for i in range(len(industry_criteria)):
  if industry_criteria.loc[i,'대분류']!=0:
    main_category = industry_criteria.loc[i,'대분류']
    dict_industry_criteria[main_category]=[industry_criteria.loc[i,'항목명']]
  else:
    dict_industry_criteria[main_category].append(industry_criteria.loc[i,'항목명'])

In [ ]:
# 대분류 항목코드(Alphabet)에 따른 딕셔너리 만들기

maincategory = industry_criteria['대분류'].unique()
maincategory=np.delete(maincategory,1)
maincategory
import string 
alphabet_upper = string.ascii_uppercase[:-5]
alphabet_upper

dict_main_category=dict()
for i in range(len(alphabet_upper)):
  dict_main_category[alphabet_upper[i]]=maincategory[i]
dict_main_category

{'A': '농업, 임업 및 어업',
 'B': '광업',
 'C': '제조업',
 'D': '전기, 가스, 증기 및 공기 조절 공급업',
 'E': '수도, 하수 및 폐기물 처리, 원료 재생업',
 'F': '건설업',
 'G': '도매 및 소매업',
 'H': '운수 및 창고업',
 'I': '숙박 및 음식점업',
 'J': '정보통신업',
 'K': '금융 및 보험업',
 'L': '부동산업',
 'M': '전문, 과학 및 기술 서비스업',
 'N': '사업시설 관리, 사업 지원 및 임대 서비스업',
 'O': '공공 행정, 국방 및 사회보장 행정',
 'P': '교육 서비스업',
 'Q': '보건업 및 사회복지 서비스업',
 'R': '예술, 스포츠 및 여가관련 서비스업',
 'S': '협회 및 단체, 수리 및 기타 개인 서비스업',
 'T': '가구 내 고용활동 및 달리 분류되지 않은 자가 소비 생산활동',
 'U': '국제 및 외국기관'}

In [ ]:
print(dict_main_category['A'])
print(dict_industry_criteria[dict_main_category['A']])

농업, 임업 및 어업
['작물 재배업', '축산업', '작물재배 및 축산 복합농업', '작물재배 및 축산 관련 서비스업', '수렵 및 관련 서비스업', '임업', '어로 어업', '양식어업 및 어업관련 서비스업']


In [ ]:
# A
# key word: 농업, 축산, 작물재배, 수렵, 작물 재배, 임업, 양식
for i in range(len(df_enterprise)):
  if ('농업' in df_enterprise.loc[i,'지원내용'])|('축산' in df_enterprise.loc[i,'지원내용'])|('작물재배' in df_enterprise.loc[i,'지원내용'])|('수렵' in df_enterprise.loc[i,'지원내용'])|('임업' in df_enterprise.loc[i,'지원내용'])|('어업' in df_enterprise.loc[i,'지원내용'])|('양식' in df_enterprise.loc[i,'지원내용']):
    df_enterprise.loc[i,'A']= 1
  else:
    if ('농업' in df_enterprise.loc[i,'지원대상'])|('축산' in df_enterprise.loc[i,'지원대상'])|('작물재배' in df_enterprise.loc[i,'지원대상'])|('수렵' in df_enterprise.loc[i,'지원대상'])|('임업' in df_enterprise.loc[i,'지원대상'])|('어업' in df_enterprise.loc[i,'지원대상'])|('양식' in df_enterprise.loc[i,'지원대상']):
      df_enterprise.loc[i,'A']= 1
    else:
      df_enterprise.loc[i,'A']= 0

In [ ]:
print(dict_main_category['B'])
print(dict_industry_criteria[dict_main_category['B']])

광업
['석탄 광업', '원유 및 천연가스 채굴업', '철 광업', '비철금속 광업', '토사석 광업', '기타 비금속광물 광업', '광업 지원 서비스업']


In [ ]:
# B
# key word: 광업, 석탄, 원유, 채굴, 석탄, 금속, 광물
for i in range(len(df_enterprise)):
  if ('광업' in df_enterprise.loc[i,'지원내용'])|('석탄' in df_enterprise.loc[i,'지원내용'])|('원유' in df_enterprise.loc[i,'지원내용'])|('채굴' in df_enterprise.loc[i,'지원내용'])|('석탄' in df_enterprise.loc[i,'지원내용'])|('금속' in df_enterprise.loc[i,'지원내용'])|('광물' in df_enterprise.loc[i,'지원내용']):
    df_enterprise.loc[i,'B']= 1
  else:
    if ('광업' in df_enterprise.loc[i,'지원대상'])|('석탄' in df_enterprise.loc[i,'지원대상'])|('원유' in df_enterprise.loc[i,'지원대상'])|('채굴' in df_enterprise.loc[i,'지원대상'])|('석탄' in df_enterprise.loc[i,'지원대상'])|('금속' in df_enterprise.loc[i,'지원대상'])|('광물' in df_enterprise.loc[i,'지원대상']):
      df_enterprise.loc[i,'B']= 1
    else:
      df_enterprise.loc[i,'B']= 0

In [ ]:
print(dict_main_category['C'])
print(dict_industry_criteria[dict_main_category['C']])

제조업
['도축, 육류 가공 및 저장 처리업', '수산물 가공 및 저장 처리업', '과실, 채소 가공 및 저장 처리업', '동물성 및 식물성 유지 제조업', '낙농제품 및 식용 빙과류 제조업', '곡물 가공품, 전분 및 전분제품 제조업', '기타 식품 제조업', '동물용 사료 및 조제식품 제조업', '알코올 음료 제조업', '비알코올 음료 및 얼음 제조업', '담배 제조업', '방적 및 가공사 제조업', '직물 직조 및 직물제품 제조업', '편조 원단 제조업', '섬유제품 염색, 정리 및 마무리 가공업', '기타 섬유제품 제조업', '봉제의복 제조업', '모피제품 제조업', '편조의복 제조업', '의복 액세서리 제조업', '가죽, 가방 및 유사 제품 제조업', '신발 및 신발 부분품 제조업', '제재 및 목재 가공업', '나무제품 제조업', '코르크 및 조물 제품 제조업', '펄프, 종이 및 판지 제조업', '골판지, 종이 상자 및 종이 용기 제조업', '기타 종이 및 판지 제품 제조업', '인쇄 및 인쇄관련 산업', '기록매체 복제업', '코크스 및 연탄 제조업', '석유 정제품 제조업', '기초 화학물질 제조업', '합성고무 및 플라스틱 물질 제조업', '비료, 농약 및 살균ㆍ살충제 제조업', '기타 화학제품 제조업', '화학섬유 제조업', '기초 의약 물질 및 생물학적 제제 제조업', '의약품 제조업', '의료용품 및 기타 의약 관련제품 제조업', '고무제품 제조업', '플라스틱 제품 제조업', '유리 및 유리제품 제조업', '내화, 비내화 요업제품 제조업', '시멘트, 석회, 플라스터 및 그 제품 제조업', '기타 비금속 광물제품 제조업', '1차 철강 제조업', '1차 비철금속 제조업', '금속 주조업', '구조용 금속제품, 탱크 및 증기발생기 제조업', '무기 및 총포탄 제조업', '기타 금속 가공제품 제조업', '반도체 제조업', '전자 부품 제조업', '컴퓨터 및 주변 장치 제조업', '통신 및 방송장비 제조업', '영상 및 음향 기기 제조업',

In [ ]:
# C
# key word: 제조, 도축, 가공
for i in range(len(df_enterprise)):
  if ('광업' in df_enterprise.loc[i,'지원내용'])|('도축' in df_enterprise.loc[i,'지원내용'])|('가공' in df_enterprise.loc[i,'지원내용']):
    df_enterprise.loc[i,'C']= 1
  else:
    if ('광업' in df_enterprise.loc[i,'지원대상'])|('도축' in df_enterprise.loc[i,'지원대상'])|('가공' in df_enterprise.loc[i,'지원대상']):
      df_enterprise.loc[i,'C']= 1
    else:
      df_enterprise.loc[i,'C']= 0

In [ ]:
print(dict_main_category['D'])
print(dict_industry_criteria[dict_main_category['D']])

전기, 가스, 증기 및 공기 조절 공급업
['전기업', '연료용 가스 제조 및 배관공급업', '증기, 냉ㆍ온수 및 공기 조절 공급업']


In [ ]:
# D
# key word: 전기, 가스, 배관
for i in range(len(df_enterprise)):
  if ('전기' in df_enterprise.loc[i,'지원내용'])|('가스' in df_enterprise.loc[i,'지원내용'])|('배관' in df_enterprise.loc[i,'지원내용']):
    df_enterprise.loc[i,'D']= 1
  else:
    if ('전기' in df_enterprise.loc[i,'지원대상'])|('가스' in df_enterprise.loc[i,'지원대상'])|('배관' in df_enterprise.loc[i,'지원대상']):
      df_enterprise.loc[i,'D']= 1
    else:
      df_enterprise.loc[i,'D']= 0

In [ ]:
print(dict_main_category['E'])
print(dict_industry_criteria[dict_main_category['E']])

수도, 하수 및 폐기물 처리, 원료 재생업
['수도업', '하수, 폐수 및 분뇨 처리업', '폐기물 수집, 운반업', '폐기물 처리업', '해체, 선별 및 원료 재생업', '환경 정화 및 복원업']


In [ ]:
# E
# key word: 수도, 하수, 폐기물, 정화
for i in range(len(df_enterprise)):
  if ('수도' in df_enterprise.loc[i,'지원내용'])|('하수' in df_enterprise.loc[i,'지원내용'])|('폐기물' in df_enterprise.loc[i,'지원내용'])|('정화' in df_enterprise.loc[i,'지원내용']):
    df_enterprise.loc[i,'E']= 1
  else:
    if ('수도' in df_enterprise.loc[i,'지원대상'])|('하수' in df_enterprise.loc[i,'지원대상'])|('폐기물' in df_enterprise.loc[i,'지원대상'])|('정화' in df_enterprise.loc[i,'지원대상']):
      df_enterprise.loc[i,'E']= 1
    else:
      df_enterprise.loc[i,'E']= 0

In [ ]:
print(dict_main_category['F'])
print(dict_industry_criteria[dict_main_category['F']])

건설업
['건물 건설업', '토목 건설업', '기반조성 및 시설물 축조관련 전문공사업', '건물설비 설치 공사업', '전기 및 통신 공사업', '실내건축 및 건축마무리 공사업', '시설물 유지관리 공사업', '건설장비 운영업']


In [ ]:
# F
# key word: 건설, 토목, 공사
for i in range(len(df_enterprise)):
  if ('건설' in df_enterprise.loc[i,'지원내용'])|('토목' in df_enterprise.loc[i,'지원내용'])|('공사' in df_enterprise.loc[i,'지원내용']):
    df_enterprise.loc[i,'F']= 1
  else:
    if ('건설' in df_enterprise.loc[i,'지원대상'])|('토목' in df_enterprise.loc[i,'지원대상'])|('공사' in df_enterprise.loc[i,'지원대상']):
      df_enterprise.loc[i,'F']= 1
    else:
      df_enterprise.loc[i,'F']= 0

In [ ]:
print(dict_main_category['G'])
print(dict_industry_criteria[dict_main_category['G']])

도매 및 소매업
['자동차 판매업', '자동차 부품 및 내장품 판매업', '모터사이클 및 부품 판매업', '상품 중개업', '산업용 농ㆍ축산물 및 동ㆍ식물 도매업', '음ㆍ식료품 및 담배 도매업', '생활용품 도매업', '기계장비 및 관련 물품 도매업', '건축 자재, 철물 및 난방장치 도매업', '기타 전문 도매업', '상품 종합 도매업', '종합 소매업', '음ㆍ식료품 및 담배 소매업', '가전제품 및 정보 통신장비 소매업', '섬유, 의복, 신발 및 가죽제품 소매업', '기타 생활용품 소매업', '문화, 오락 및 여가 용품 소매업', '연료 소매업', '기타 상품 전문 소매업', '무점포 소매업']


In [ ]:
# G
# key word: 도매, 소매, 판매
for i in range(len(df_enterprise)):
  if ('도매' in df_enterprise.loc[i,'지원내용'])|('소매' in df_enterprise.loc[i,'지원내용'])|('판매' in df_enterprise.loc[i,'지원내용']):
    df_enterprise.loc[i,'G']= 1
  else:
    if ('도매' in df_enterprise.loc[i,'지원대상'])|('소매' in df_enterprise.loc[i,'지원대상'])|('판매' in df_enterprise.loc[i,'지원대상']):
      df_enterprise.loc[i,'G']= 1
    else:
      df_enterprise.loc[i,'G']= 0

In [ ]:
print(dict_main_category['H'])
print(dict_industry_criteria[dict_main_category['H']])

운수 및 창고업
['철도 운송업', '육상 여객 운송업', '도로 화물 운송업', '소화물 전문 운송업', '파이프라인 운송업', '해상 운송업', '내륙 수상 및 항만 내 운송업', '항공 여객 운송업', '항공 화물 운송업', '보관 및 창고업', '기타 운송관련 서비스업']


In [ ]:
# H
# key word: 운수, 운송, 창고
for i in range(len(df_enterprise)):
  if ('운수' in df_enterprise.loc[i,'지원내용'])|('운송' in df_enterprise.loc[i,'지원내용'])|('창고' in df_enterprise.loc[i,'지원내용']):
    df_enterprise.loc[i,'H']= 1
  else:
    if ('운수' in df_enterprise.loc[i,'지원대상'])|('운송' in df_enterprise.loc[i,'지원대상'])|('창고' in df_enterprise.loc[i,'지원대상']):
      df_enterprise.loc[i,'H']= 1
    else:
      df_enterprise.loc[i,'H']= 0

In [ ]:
print(dict_main_category['I'])
print(dict_industry_criteria[dict_main_category['I']])

숙박 및 음식점업
['일반 및 생활 숙박시설 운영업', '기타 숙박업', '음식점업', '주점 및 비알코올 음료점업']


In [ ]:
# I
# key word: 숙박, 음식점, 음료점
for i in range(len(df_enterprise)):
  if ('숙박' in df_enterprise.loc[i,'지원내용'])|('음식점' in df_enterprise.loc[i,'지원내용'])|('음료점' in df_enterprise.loc[i,'지원내용']):
    df_enterprise.loc[i,'I']= 1
  else:
    if ('숙박' in df_enterprise.loc[i,'지원대상'])|('음식점' in df_enterprise.loc[i,'지원대상'])|('음료점' in df_enterprise.loc[i,'지원대상']):
      df_enterprise.loc[i,'I']= 1
    else:
      df_enterprise.loc[i,'I']= 0

In [ ]:
print(dict_main_category['J'])
print(dict_industry_criteria[dict_main_category['J']])

정보통신업
['서적, 잡지 및 기타 인쇄물 출판업', '소프트웨어 개발 및 공급업', '영화, 비디오물, 방송 프로그램 제작 및 배급업', '오디오물 출판 및 원판 녹음업', '라디오 방송업', '텔레비전 방송업', '공영 우편업', '전기 통신업', '컴퓨터 프로그래밍, 시스템 통합 및 관리업', '자료 처리, 호스팅, 포털 및 기타 인터넷 정보 매개 서비스업', '기타 정보 서비스업']


In [ ]:
# J
# key word: 정보통신, 인쇄물, 출판, 방송, 프로그래밍
for i in range(len(df_enterprise)):
  if ('정보통신' in df_enterprise.loc[i,'지원내용'])|('인쇄물' in df_enterprise.loc[i,'지원내용'])|('출판' in df_enterprise.loc[i,'지원내용'])|('방송' in df_enterprise.loc[i,'지원내용'])|('프로그래밍' in df_enterprise.loc[i,'지원내용']):
    df_enterprise.loc[i,'J']= 1
  else:
    if ('정보통신' in df_enterprise.loc[i,'지원대상'])|('인쇄물' in df_enterprise.loc[i,'지원대상'])|('출판' in df_enterprise.loc[i,'지원대상'])|('방송' in df_enterprise.loc[i,'지원대상'])|('프로그래밍' in df_enterprise.loc[i,'지원대상']):
      df_enterprise.loc[i,'J']= 1
    else:
      df_enterprise.loc[i,'J']= 0

In [ ]:
print(dict_main_category['K'])
print(dict_industry_criteria[dict_main_category['K']])

금융 및 보험업
['은행 및 저축기관', '신탁업 및 집합 투자업', '기타 금융업', '보험업', '재보험업', '연금 및 공제업', '금융 지원 서비스업', '보험 및 연금관련 서비스업']


In [ ]:
# K
# key word: 은행, 저축, 보험, 투자
for i in range(len(df_enterprise)):
  if ('은행업' in df_enterprise.loc[i,'지원내용'])|('저축업' in df_enterprise.loc[i,'지원내용'])|('보험업' in df_enterprise.loc[i,'지원내용'])|('투자업' in df_enterprise.loc[i,'지원내용']):
    df_enterprise.loc[i,'K']= 1
  else:
    if ('은행업' in df_enterprise.loc[i,'지원대상'])|('저축업' in df_enterprise.loc[i,'지원대상'])|('보험업' in df_enterprise.loc[i,'지원대상'])|('투자업' in df_enterprise.loc[i,'지원대상']):
      df_enterprise.loc[i,'K']= 1
    else:
      df_enterprise.loc[i,'K']= 0

In [ ]:
print(dict_main_category['L'])
print(dict_industry_criteria[dict_main_category['L']])

부동산업
['부동산 임대 및 공급업', '부동산관련 서비스업']


In [ ]:
# L
# key word: 부동산
for i in range(len(df_enterprise)):
  if '부동산' in df_enterprise.loc[i,'지원내용']:
    df_enterprise.loc[i,'L']= 1
  else:
    if '부동산' in df_enterprise.loc[i,'지원대상']:
      df_enterprise.loc[i,'L']= 1
    else:
      df_enterprise.loc[i,'L']= 0

In [ ]:
print(dict_main_category['M'])
print(dict_industry_criteria[dict_main_category['M']])

전문, 과학 및 기술 서비스업
['자연과학 및 공학 연구개발업', '인문 및 사회과학 연구개발업', '법무관련 서비스업', '회계 및 세무관련 서비스업', '광고업', '시장 조사 및 여론 조사업', '회사 본부 및 경영 컨설팅 서비스업', '기타 전문 서비스업', '건축 기술, 엔지니어링 및 관련 기술 서비스업', '기타 과학기술 서비스업', '수의업', '전문 디자인업', '사진 촬영 및 처리업', '그 외 기타 전문, 과학 및 기술 서비스업']


In [ ]:
# M
# key word: 연구, 자연과학, 공학, 기술, 건축
for i in range(len(df_enterprise)):
  if ('연구' in df_enterprise.loc[i,'지원내용'])|('자연과학' in df_enterprise.loc[i,'지원내용'])|('공학' in df_enterprise.loc[i,'지원내용'])|('기술' in df_enterprise.loc[i,'지원내용'])|('건축' in df_enterprise.loc[i,'지원내용']):
    df_enterprise.loc[i,'M']= 1
  else:
    if ('연구' in df_enterprise.loc[i,'지원대상'])|('자연과학' in df_enterprise.loc[i,'지원대상'])|('공학' in df_enterprise.loc[i,'지원대상'])|('기술' in df_enterprise.loc[i,'지원대상'])|('건축' in df_enterprise.loc[i,'지원내용']):
      df_enterprise.loc[i,'M']= 1
    else:
      df_enterprise.loc[i,'M']= 0

In [ ]:
df_enterprise.columns

Index(['정책ID', '정책서비스ID', '서비스명', '소관기관', '소관기관유형', '소관기관.1', '문의처전화번호',
       '소관기관유형.1', '온라인신청사이트URL', '생애주기', '신청절차', '선정기준', '지원유형', '서비스목적',
       '지원내용', '지원대상', '신청기한_시작', '신청기한_끝', '상세조회URL', '태그-신청절차', '태그-자녀',
       '태그-수집유형', '태그-가구원', '태그-성별', '태그-학력', '태그-직장', '태그-결혼', '태그-지원유형',
       '태그-지역(시도)', '태그-관심상황특성', '태그-자녀상세', '태그-직장상세', '태그-지원유형상세',
       '태그-지역(시군구)', '태그-대상특성상세', '태그-대상특성', '태그-대상연령시작', '태그-대상연령끝',
       '태그-중위소득시작', '태그-중위소득끝', 'company_policy', '기업규모', 'A', 'B', 'C', 'D',
       'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M'],
      dtype='object')

In [ ]:
print(dict_main_category['N'])
print(dict_industry_criteria[dict_main_category['N']])

사업시설 관리, 사업 지원 및 임대 서비스업
['사업시설 유지ㆍ관리 서비스업', '건물ㆍ산업설비 청소 및 방제 서비스업', '조경관리 및 유지 서비스업', '고용 알선 및 인력 공급업', '여행사 및 기타 여행 보조 서비스업', '경비, 경호 및 탐정업', '기타 사업 지원 서비스업', '운송장비 임대업', '개인 및 가정용품 임대업', '산업용 기계 및 장비 임대업', '무형 재산권 임대업']


In [ ]:
# N
# key word: 시설관리, 시설 관리, 일자리, 청소, 여행사, 경비, 임대
for i in range(len(df_enterprise)):
  if ('시설관리' in df_enterprise.loc[i,'지원내용'])|('시설 관리' in df_enterprise.loc[i,'지원내용'])|('일자리' in df_enterprise.loc[i,'지원내용'])|('청소' in df_enterprise.loc[i,'지원내용'])|('여행사' in df_enterprise.loc[i,'지원내용'])|('경비' in df_enterprise.loc[i,'지원내용']):
    df_enterprise.loc[i,'N']= 1
  else:
    if ('시설관리' in df_enterprise.loc[i,'지원대상'])|('시설 관리' in df_enterprise.loc[i,'지원대상'])|('일자리' in df_enterprise.loc[i,'지원대상'])|('청소' in df_enterprise.loc[i,'지원대상'])|('여행사' in df_enterprise.loc[i,'지원대상'])|('경비' in df_enterprise.loc[i,'지원대상']):
      df_enterprise.loc[i,'N']= 1
    else:
      df_enterprise.loc[i,'N']= 0

In [ ]:
print(dict_main_category['O'])
print(dict_industry_criteria[dict_main_category['O']])

공공 행정, 국방 및 사회보장 행정
['입법 및 일반 정부 행정', '사회 및 산업정책 행정', '외무 및 국방 행정', '사법 및 공공 질서 행정', '사회보장 행정']


In [ ]:
# O
# key word: 행정, 국방, 정책,
for i in range(len(df_enterprise)):
  if ('행정' in df_enterprise.loc[i,'지원내용'])|('국방' in df_enterprise.loc[i,'지원내용'])|('국방' in df_enterprise.loc[i,'지원내용']):
    df_enterprise.loc[i,'O']= 1
  else:
    if ('행정' in df_enterprise.loc[i,'지원대상'])|('국방' in df_enterprise.loc[i,'지원대상'])|('국방' in df_enterprise.loc[i,'지원대상']):
      df_enterprise.loc[i,'O']= 1
    else:
      df_enterprise.loc[i,'O']= 0

In [ ]:
print(dict_main_category['P'])
print(dict_industry_criteria[dict_main_category['P']])

교육 서비스업
['초등 교육기관', '중등 교육기관', '고등 교육기관', '특수학교, 외국인학교 및 대안학교', '일반 교습학원', '기타 교육기관', '교육 지원 서비스업']


In [ ]:
# P
# key word: 교육, 학교
for i in range(len(df_enterprise)):
  if ('교육' in df_enterprise.loc[i,'지원내용'])|('학교' in df_enterprise.loc[i,'지원내용']):
    df_enterprise.loc[i,'P']= 1
  else:
    if ('교육' in df_enterprise.loc[i,'지원대상'])|('학교' in df_enterprise.loc[i,'지원대상']):
      df_enterprise.loc[i,'P']= 1
    else:
      df_enterprise.loc[i,'P']= 0

In [ ]:
print(dict_main_category['Q'])
print(dict_industry_criteria[dict_main_category['Q']])

보건업 및 사회복지 서비스업
['병원', '의원', '공중 보건 의료업', '기타 보건업', '거주 복지시설 운영업', '비거주 복지시설 운영업']


In [ ]:
# Q
# key word: 병원, 의원, 보건, 복지
for i in range(len(df_enterprise)):
  if ('병원' in df_enterprise.loc[i,'지원내용'])|('의원' in df_enterprise.loc[i,'지원내용'])|('보건' in df_enterprise.loc[i,'지원내용'])|('복지' in df_enterprise.loc[i,'지원내용']):
    df_enterprise.loc[i,'Q']= 1
  else:
    if ('병원' in df_enterprise.loc[i,'지원대상'])|('의원' in df_enterprise.loc[i,'지원내용'])|('보건' in df_enterprise.loc[i,'지원내용'])|('복지' in df_enterprise.loc[i,'지원내용']):
      df_enterprise.loc[i,'Q']= 1
    else:
      df_enterprise.loc[i,'Q']= 0

In [ ]:
print(dict_main_category['R'])
print(dict_industry_criteria[dict_main_category['R']])

예술, 스포츠 및 여가관련 서비스업
['창작 및 예술관련 서비스업', '도서관, 사적지 및 유사 여가관련 서비스업', '스포츠 서비스업', '유원지 및 기타 오락관련 서비스업']


In [ ]:
df_enterprise.columns

Index(['정책ID', '정책서비스ID', '서비스명', '소관기관', '소관기관유형', '소관기관.1', '문의처전화번호',
       '소관기관유형.1', '온라인신청사이트URL', '생애주기', '신청절차', '선정기준', '지원유형', '서비스목적',
       '지원내용', '지원대상', '신청기한_시작', '신청기한_끝', '상세조회URL', '태그-신청절차', '태그-자녀',
       '태그-수집유형', '태그-가구원', '태그-성별', '태그-학력', '태그-직장', '태그-결혼', '태그-지원유형',
       '태그-지역(시도)', '태그-관심상황특성', '태그-자녀상세', '태그-직장상세', '태그-지원유형상세',
       '태그-지역(시군구)', '태그-대상특성상세', '태그-대상특성', '태그-대상연령시작', '태그-대상연령끝',
       '태그-중위소득시작', '태그-중위소득끝', 'company_policy', '기업규모', 'A', 'B', 'C', 'D',
       'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q'],
      dtype='object')

In [ ]:
# R
# key word: 예술, 여가, 스포츠, 오락
for i in range(len(df_enterprise)):
  if ('예술' in df_enterprise.loc[i,'지원내용'])|('여가' in df_enterprise.loc[i,'지원내용'])|('스포츠' in df_enterprise.loc[i,'지원내용'])|('오락' in df_enterprise.loc[i,'지원내용']):
    df_enterprise.loc[i,'R']= 1
  else:
    if ('예술' in df_enterprise.loc[i,'지원대상'])|('여가' in df_enterprise.loc[i,'지원내용'])|('스포츠' in df_enterprise.loc[i,'지원내용'])|('오락' in df_enterprise.loc[i,'지원내용']):
      df_enterprise.loc[i,'R']= 1
    else:
      df_enterprise.loc[i,'R']= 0

In [ ]:
print(dict_main_category['S'])
print(dict_industry_criteria[dict_main_category['S']])

협회 및 단체, 수리 및 기타 개인 서비스업
['산업 및 전문가 단체', '노동조합', '기타 협회 및 단체', '컴퓨터 및 통신장비 수리업', '자동차 및 모터사이클 수리업', '개인 및 가정용품 수리업', '미용, 욕탕 및 유사 서비스업', '그 외 기타 개인 서비스업']


In [ ]:
# S
# key word: 협회, 수리업
for i in range(len(df_enterprise)):
  if ('협회/단체/기타서비스업' in df_enterprise.loc[i,'지원내용'])|('수리업' in df_enterprise.loc[i,'지원내용']):
    df_enterprise.loc[i,'S']= 1
  else:
    if ('협회/단체/기타서비스업' in df_enterprise.loc[i,'지원대상'])|('수리업' in df_enterprise.loc[i,'지원내용']):
      df_enterprise.loc[i,'S']= 1
    else:
      df_enterprise.loc[i,'S']= 0

In [ ]:
# T는 해당 키워드가 없으니 패쓰
print(dict_main_category['T'])
print(dict_industry_criteria[dict_main_category['T']])

가구 내 고용활동 및 달리 분류되지 않은 자가 소비 생산활동
['가구 내 고용활동', '자가 소비를 위한 가사 생산 활동', '자가 소비를 위한 가사 서비스 활동']


In [ ]:
# U는 해당하는 키워드에 맞는 정책이 없으니 패쓰
print(dict_main_category['U'])
print(dict_industry_criteria[dict_main_category['U']])

국제 및 외국기관
['국제 및 외국기관']


In [ ]:
dict_main_category['U']

'국제 및 외국기관'

In [ ]:
# 업종 대분류 값 만들기
df_enterprise['업종(대분류)']=''

for i in range(len(df_enterprise)):
  values = df_enterprise.loc[i,'A':'S'].values
  if values.sum()==0 :
    df_enterprise.loc[i,'업종(대분류)']= '무관'
  else:
    for j in range(len(values)):
      if j == 0:
        if values[j]==1:
          df_enterprise.loc[i,'업종(대분류)']+=dict_main_category['A']
      elif j == 1:
        if values[j]==1:
          if df_enterprise.loc[i,'업종(대분류)']=='':
            df_enterprise.loc[i,'업종(대분류)']+=dict_main_category['B']
          else:
            df_enterprise.loc[i,'업종(대분류)']+='/'+dict_main_category['B']
  
      elif j == 2:
        if values[j]==1:
          if df_enterprise.loc[i,'업종(대분류)']=='':
            df_enterprise.loc[i,'업종(대분류)']+=dict_main_category['C']
          else:
            df_enterprise.loc[i,'업종(대분류)']+='/'+dict_main_category['C']
      elif j == 3:
        if values[j]==1:
          if df_enterprise.loc[i,'업종(대분류)']=='':
            df_enterprise.loc[i,'업종(대분류)']+=dict_main_category['D']
          else:
            df_enterprise.loc[i,'업종(대분류)']+='/'+dict_main_category['D']
      elif j == 4:
        if values[j]==1:
          if df_enterprise.loc[i,'업종(대분류)']=='':
            df_enterprise.loc[i,'업종(대분류)']+=dict_main_category['E']
          else:
            df_enterprise.loc[i,'업종(대분류)']+='/'+dict_main_category['E']
      elif j == 5:
        if values[j]==1:
          if df_enterprise.loc[i,'업종(대분류)']=='':
            df_enterprise.loc[i,'업종(대분류)']+=dict_main_category['F']
          else:
            df_enterprise.loc[i,'업종(대분류)']+='/'+dict_main_category['F']
      elif j == 6:
        if values[j]==1:
          if df_enterprise.loc[i,'업종(대분류)']=='':
            df_enterprise.loc[i,'업종(대분류)']+=dict_main_category['G']
          else:
            df_enterprise.loc[i,'업종(대분류)']+='/'+dict_main_category['G']
      elif j == 7:
        if values[j]==1:
          if df_enterprise.loc[i,'업종(대분류)']=='':
            df_enterprise.loc[i,'업종(대분류)']+=dict_main_category['H']
          else:
            df_enterprise.loc[i,'업종(대분류)']+='/'+dict_main_category['H']
      elif j == 8:
        if values[j]==1:
          if df_enterprise.loc[i,'업종(대분류)']=='':
            df_enterprise.loc[i,'업종(대분류)']+=dict_main_category['I']
          else:
            df_enterprise.loc[i,'업종(대분류)']+='/'+dict_main_category['I']
      elif j == 9:
        if values[j]==1:
          if df_enterprise.loc[i,'업종(대분류)']=='':
            df_enterprise.loc[i,'업종(대분류)']+=dict_main_category['J']
          else:
            df_enterprise.loc[i,'업종(대분류)']+='/'+dict_main_category['J']
      elif j == 10:
        if values[j]==1:
          if df_enterprise.loc[i,'업종(대분류)']=='':
            df_enterprise.loc[i,'업종(대분류)']+=dict_main_category['K']
          else:
            df_enterprise.loc[i,'업종(대분류)']+='/'+dict_main_category['K']
      elif j == 11:
        if values[j]==1:
          if df_enterprise.loc[i,'업종(대분류)']=='':
            df_enterprise.loc[i,'업종(대분류)']+=dict_main_category['L']
          else:
            df_enterprise.loc[i,'업종(대분류)']+='/'+dict_main_category['L']

      elif j == 12:
        if values[j]==1:
          if df_enterprise.loc[i,'업종(대분류)']=='':
            df_enterprise.loc[i,'업종(대분류)']+=dict_main_category['M']
          else:
            df_enterprise.loc[i,'업종(대분류)']+='/'+dict_main_category['M']

      elif j == 13:
        if values[j]==1:
          if df_enterprise.loc[i,'업종(대분류)']=='':
            df_enterprise.loc[i,'업종(대분류)']+=dict_main_category['N']
          else:
            df_enterprise.loc[i,'업종(대분류)']+='/'+dict_main_category['N']

      elif j == 14:
        if values[j]==1:
          if df_enterprise.loc[i,'업종(대분류)']=='':
            df_enterprise.loc[i,'업종(대분류)']+=dict_main_category['O']
          else:
            df_enterprise.loc[i,'업종(대분류)']+='/'+dict_main_category['O']

      elif j == 15:
        if values[j]==1:
          if df_enterprise.loc[i,'업종(대분류)']=='':
            df_enterprise.loc[i,'업종(대분류)']+=dict_main_category['P']
          else:
            df_enterprise.loc[i,'업종(대분류)']+='/'+dict_main_category['P']

      elif j == 16:
        if values[j]==1:
          if df_enterprise.loc[i,'업종(대분류)']=='':
            df_enterprise.loc[i,'업종(대분류)']+=dict_main_category['Q']
          else:
            df_enterprise.loc[i,'업종(대분류)']+='/'+dict_main_category['Q']

      elif j == 17:
        if values[j]==1:
          if df_enterprise.loc[i,'업종(대분류)']=='':
            df_enterprise.loc[i,'업종(대분류)']+=dict_main_category['R']
          else:
            df_enterprise.loc[i,'업종(대분류)']+='/'+dict_main_category['R']

      elif j == 18:
        if values[j]==1:
          if df_enterprise.loc[i,'업종(대분류)']=='':
            df_enterprise.loc[i,'업종(대분류)']+=dict_main_category['S']
          else:
            df_enterprise.loc[i,'업종(대분류)']+='/'+dict_main_category['S']
            
del df_enterprise['A']
del df_enterprise['B']
del df_enterprise['C']
del df_enterprise['D']
del df_enterprise['E']
del df_enterprise['F']
del df_enterprise['G']
del df_enterprise['H']
del df_enterprise['I']
del df_enterprise['J']
del df_enterprise['K']
del df_enterprise['L']
del df_enterprise['M']
del df_enterprise['N']
del df_enterprise['O']
del df_enterprise['P']
del df_enterprise['Q']
del df_enterprise['R']
del df_enterprise['S']

#### **B. 정리**
1. 산업분류표 10차에 해당하는 대분류 코드 'A-S'에 해당하는 업종으로 분류 (T - 가사 일, U - 국제기관이라 제외)
2. 무관에 해당하는 경우 총 16936건 중 2542건에 해당. 나머지는 다 한개 이상의 업종으로 분류

In [ ]:
df_good.shape

(16936, 11)

In [ ]:
df_enterprise['업종(대분류)']

0                                              무관
1                                     농업, 임업 및 어업
2                                 농업, 임업 및 어업/제조업
3                                        도매 및 소매업
4                                     농업, 임업 및 어업
                           ...                   
16931                     교육 서비스업/보건업 및 사회복지 서비스업
16932    전문, 과학 및 기술 서비스업/교육 서비스업/보건업 및 사회복지 서비스업
16933             농업, 임업 및 어업/공공 행정, 국방 및 사회보장 행정
16934                                 농업, 임업 및 어업
16935                        농업, 임업 및 어업/도매 및 소매업
Name: 업종(대분류), Length: 16936, dtype: object

#### C. **TAG CODE 전체**

In [ ]:
# 약 3분 소요
industry_criteria = pd.read_excel('/content/drive/MyDrive/Wello+project_신희택,윤다솜/Data/(10차)산업분류_대분류항목값.xlsx').fillna(0)

# 대분류 항목명에 따른 딕셔너리 만들기
dict_industry_criteria=dict()
for i in range(len(industry_criteria)):
  if industry_criteria.loc[i,'대분류']!=0:
    main_category = industry_criteria.loc[i,'대분류']
    dict_industry_criteria[main_category]=[industry_criteria.loc[i,'항목명']]
  else:
    dict_industry_criteria[main_category].append(industry_criteria.loc[i,'항목명'])

# 대분류 항목코드(Alphabet)에 따른 딕셔너리 만들기
maincategory = industry_criteria['대분류'].unique()
maincategory=np.delete(maincategory,1)

import string 
alphabet_upper = string.ascii_uppercase[:-5]

dict_main_category=dict()
for i in range(len(alphabet_upper)):
  dict_main_category[alphabet_upper[i]]=maincategory[i]
dict_main_category
# A
# key word: 농업, 축산, 작물재배, 수렵, 작물 재배, 임업, 양식
for i in range(len(df_enterprise)):
  if ('농업' in df_enterprise.loc[i,'지원내용'])|('축산' in df_enterprise.loc[i,'지원내용'])|('작물재배' in df_enterprise.loc[i,'지원내용'])|('수렵' in df_enterprise.loc[i,'지원내용'])|('임업' in df_enterprise.loc[i,'지원내용'])|('어업' in df_enterprise.loc[i,'지원내용'])|('양식' in df_enterprise.loc[i,'지원내용']):
    df_enterprise.loc[i,'A']= 1
  else:
    if ('농업' in df_enterprise.loc[i,'지원대상'])|('축산' in df_enterprise.loc[i,'지원대상'])|('작물재배' in df_enterprise.loc[i,'지원대상'])|('수렵' in df_enterprise.loc[i,'지원대상'])|('임업' in df_enterprise.loc[i,'지원대상'])|('어업' in df_enterprise.loc[i,'지원대상'])|('양식' in df_enterprise.loc[i,'지원대상']):
      df_enterprise.loc[i,'A']= 1
    else:
      df_enterprise.loc[i,'A']= 0
# B
# key word: 광업, 석탄, 원유, 채굴, 석탄, 금속, 광물
for i in range(len(df_enterprise)):
  if ('광업' in df_enterprise.loc[i,'지원내용'])|('석탄' in df_enterprise.loc[i,'지원내용'])|('원유' in df_enterprise.loc[i,'지원내용'])|('채굴' in df_enterprise.loc[i,'지원내용'])|('석탄' in df_enterprise.loc[i,'지원내용'])|('금속' in df_enterprise.loc[i,'지원내용'])|('광물' in df_enterprise.loc[i,'지원내용']):
    df_enterprise.loc[i,'B']= 1
  else:
    if ('광업' in df_enterprise.loc[i,'지원대상'])|('석탄' in df_enterprise.loc[i,'지원대상'])|('원유' in df_enterprise.loc[i,'지원대상'])|('채굴' in df_enterprise.loc[i,'지원대상'])|('석탄' in df_enterprise.loc[i,'지원대상'])|('금속' in df_enterprise.loc[i,'지원대상'])|('광물' in df_enterprise.loc[i,'지원대상']):
      df_enterprise.loc[i,'B']= 1
    else:
      df_enterprise.loc[i,'B']= 0
# C
# key word: 제조, 도축, 가공
for i in range(len(df_enterprise)):
  if ('광업' in df_enterprise.loc[i,'지원내용'])|('도축' in df_enterprise.loc[i,'지원내용'])|('가공' in df_enterprise.loc[i,'지원내용']):
    df_enterprise.loc[i,'C']= 1
  else:
    if ('광업' in df_enterprise.loc[i,'지원대상'])|('도축' in df_enterprise.loc[i,'지원대상'])|('가공' in df_enterprise.loc[i,'지원대상']):
      df_enterprise.loc[i,'C']= 1
    else:
      df_enterprise.loc[i,'C']= 0
# D
# key word: 전기, 가스, 배관
for i in range(len(df_enterprise)):
  if ('전기' in df_enterprise.loc[i,'지원내용'])|('가스' in df_enterprise.loc[i,'지원내용'])|('배관' in df_enterprise.loc[i,'지원내용']):
    df_enterprise.loc[i,'D']= 1
  else:
    if ('전기' in df_enterprise.loc[i,'지원대상'])|('가스' in df_enterprise.loc[i,'지원대상'])|('배관' in df_enterprise.loc[i,'지원대상']):
      df_enterprise.loc[i,'D']= 1
    else:
      df_enterprise.loc[i,'D']= 0
# E
# key word: 수도, 하수, 폐기물, 정화
for i in range(len(df_enterprise)):
  if ('수도' in df_enterprise.loc[i,'지원내용'])|('하수' in df_enterprise.loc[i,'지원내용'])|('폐기물' in df_enterprise.loc[i,'지원내용'])|('정화' in df_enterprise.loc[i,'지원내용']):
    df_enterprise.loc[i,'E']= 1
  else:
    if ('수도' in df_enterprise.loc[i,'지원대상'])|('하수' in df_enterprise.loc[i,'지원대상'])|('폐기물' in df_enterprise.loc[i,'지원대상'])|('정화' in df_enterprise.loc[i,'지원대상']):
      df_enterprise.loc[i,'E']= 1
    else:
      df_enterprise.loc[i,'E']= 0
# F
# key word: 건설, 토목, 공사
for i in range(len(df_enterprise)):
  if ('건설' in df_enterprise.loc[i,'지원내용'])|('토목' in df_enterprise.loc[i,'지원내용'])|('공사' in df_enterprise.loc[i,'지원내용']):
    df_enterprise.loc[i,'F']= 1
  else:
    if ('건설' in df_enterprise.loc[i,'지원대상'])|('토목' in df_enterprise.loc[i,'지원대상'])|('공사' in df_enterprise.loc[i,'지원대상']):
      df_enterprise.loc[i,'F']= 1
    else:
      df_enterprise.loc[i,'F']= 0
# G
# key word: 도매, 소매, 판매
for i in range(len(df_enterprise)):
  if ('도매' in df_enterprise.loc[i,'지원내용'])|('소매' in df_enterprise.loc[i,'지원내용'])|('판매' in df_enterprise.loc[i,'지원내용']):
    df_enterprise.loc[i,'G']= 1
  else:
    if ('도매' in df_enterprise.loc[i,'지원대상'])|('소매' in df_enterprise.loc[i,'지원대상'])|('판매' in df_enterprise.loc[i,'지원대상']):
      df_enterprise.loc[i,'G']= 1
    else:
      df_enterprise.loc[i,'G']= 0
# H
# key word: 운수, 운송, 창고
for i in range(len(df_enterprise)):
  if ('운수' in df_enterprise.loc[i,'지원내용'])|('운송' in df_enterprise.loc[i,'지원내용'])|('창고' in df_enterprise.loc[i,'지원내용']):
    df_enterprise.loc[i,'H']= 1
  else:
    if ('운수' in df_enterprise.loc[i,'지원대상'])|('운송' in df_enterprise.loc[i,'지원대상'])|('창고' in df_enterprise.loc[i,'지원대상']):
      df_enterprise.loc[i,'H']= 1
    else:
      df_enterprise.loc[i,'H']= 0
# I
# key word: 숙박, 음식점, 음료점
for i in range(len(df_enterprise)):
  if ('숙박' in df_enterprise.loc[i,'지원내용'])|('음식점' in df_enterprise.loc[i,'지원내용'])|('음료점' in df_enterprise.loc[i,'지원내용']):
    df_enterprise.loc[i,'I']= 1
  else:
    if ('숙박' in df_enterprise.loc[i,'지원대상'])|('음식점' in df_enterprise.loc[i,'지원대상'])|('음료점' in df_enterprise.loc[i,'지원대상']):
      df_enterprise.loc[i,'I']= 1
    else:
      df_enterprise.loc[i,'I']= 0
# J
# key word: 정보통신, 인쇄물, 출판, 방송, 프로그래밍
for i in range(len(df_enterprise)):
  if ('정보통신' in df_enterprise.loc[i,'지원내용'])|('인쇄물' in df_enterprise.loc[i,'지원내용'])|('출판' in df_enterprise.loc[i,'지원내용'])|('방송' in df_enterprise.loc[i,'지원내용'])|('프로그래밍' in df_enterprise.loc[i,'지원내용']):
    df_enterprise.loc[i,'J']= 1
  else:
    if ('정보통신' in df_enterprise.loc[i,'지원대상'])|('인쇄물' in df_enterprise.loc[i,'지원대상'])|('출판' in df_enterprise.loc[i,'지원대상'])|('방송' in df_enterprise.loc[i,'지원대상'])|('프로그래밍' in df_enterprise.loc[i,'지원대상']):
      df_enterprise.loc[i,'J']= 1
    else:
      df_enterprise.loc[i,'J']= 0
# K
# key word: 은행, 저축, 보험, 투자
for i in range(len(df_enterprise)):
  if ('은행업' in df_enterprise.loc[i,'지원내용'])|('저축업' in df_enterprise.loc[i,'지원내용'])|('보험업' in df_enterprise.loc[i,'지원내용'])|('투자업' in df_enterprise.loc[i,'지원내용']):
    df_enterprise.loc[i,'K']= 1
  else:
    if ('은행업' in df_enterprise.loc[i,'지원대상'])|('저축업' in df_enterprise.loc[i,'지원대상'])|('보험업' in df_enterprise.loc[i,'지원대상'])|('투자업' in df_enterprise.loc[i,'지원대상']):
      df_enterprise.loc[i,'K']= 1
    else:
      df_enterprise.loc[i,'K']= 0
# L
# key word: 부동산
for i in range(len(df_enterprise)):
  if '부동산' in df_enterprise.loc[i,'지원내용']:
    df_enterprise.loc[i,'L']= 1
  else:
    if '부동산' in df_enterprise.loc[i,'지원대상']:
      df_enterprise.loc[i,'L']= 1
    else:
      df_enterprise.loc[i,'L']= 0
# M
# key word: 연구, 자연과학, 공학, 기술, 건축
for i in range(len(df_enterprise)):
  if ('연구' in df_enterprise.loc[i,'지원내용'])|('자연과학' in df_enterprise.loc[i,'지원내용'])|('공학' in df_enterprise.loc[i,'지원내용'])|('기술' in df_enterprise.loc[i,'지원내용'])|('건축' in df_enterprise.loc[i,'지원내용']):
    df_enterprise.loc[i,'M']= 1
  else:
    if ('연구' in df_enterprise.loc[i,'지원대상'])|('자연과학' in df_enterprise.loc[i,'지원대상'])|('공학' in df_enterprise.loc[i,'지원대상'])|('기술' in df_enterprise.loc[i,'지원대상'])|('건축' in df_enterprise.loc[i,'지원내용']):
      df_enterprise.loc[i,'M']= 1
    else:
      df_enterprise.loc[i,'M']= 0
# N
# key word: 시설관리, 시설 관리, 일자리, 청소, 여행사, 경비, 임대
for i in range(len(df_enterprise)):
  if ('시설관리' in df_enterprise.loc[i,'지원내용'])|('시설 관리' in df_enterprise.loc[i,'지원내용'])|('일자리' in df_enterprise.loc[i,'지원내용'])|('청소' in df_enterprise.loc[i,'지원내용'])|('여행사' in df_enterprise.loc[i,'지원내용'])|('경비' in df_enterprise.loc[i,'지원내용']):
    df_enterprise.loc[i,'N']= 1
  else:
    if ('시설관리' in df_enterprise.loc[i,'지원대상'])|('시설 관리' in df_enterprise.loc[i,'지원대상'])|('일자리' in df_enterprise.loc[i,'지원대상'])|('청소' in df_enterprise.loc[i,'지원대상'])|('여행사' in df_enterprise.loc[i,'지원대상'])|('경비' in df_enterprise.loc[i,'지원대상']):
      df_enterprise.loc[i,'N']= 1
    else:
      df_enterprise.loc[i,'N']= 0
# O
# key word: 행정, 국방, 정책,
for i in range(len(df_enterprise)):
  if ('행정' in df_enterprise.loc[i,'지원내용'])|('국방' in df_enterprise.loc[i,'지원내용'])|('국방' in df_enterprise.loc[i,'지원내용']):
    df_enterprise.loc[i,'O']= 1
  else:
    if ('행정' in df_enterprise.loc[i,'지원대상'])|('국방' in df_enterprise.loc[i,'지원대상'])|('국방' in df_enterprise.loc[i,'지원대상']):
      df_enterprise.loc[i,'O']= 1
    else:
      df_enterprise.loc[i,'O']= 0
# P
# key word: 교육, 학교
for i in range(len(df_enterprise)):
  if ('교육' in df_enterprise.loc[i,'지원내용'])|('학교' in df_enterprise.loc[i,'지원내용']):
    df_enterprise.loc[i,'P']= 1
  else:
    if ('교육' in df_enterprise.loc[i,'지원대상'])|('학교' in df_enterprise.loc[i,'지원대상']):
      df_enterprise.loc[i,'P']= 1
    else:
      df_enterprise.loc[i,'P']= 0
# Q
# key word: 병원, 의원, 보건, 복지
for i in range(len(df_enterprise)):
  if ('병원' in df_enterprise.loc[i,'지원내용'])|('의원' in df_enterprise.loc[i,'지원내용'])|('보건' in df_enterprise.loc[i,'지원내용'])|('복지' in df_enterprise.loc[i,'지원내용']):
    df_enterprise.loc[i,'Q']= 1
  else:
    if ('병원' in df_enterprise.loc[i,'지원대상'])|('의원' in df_enterprise.loc[i,'지원내용'])|('보건' in df_enterprise.loc[i,'지원내용'])|('복지' in df_enterprise.loc[i,'지원내용']):
      df_enterprise.loc[i,'Q']= 1
    else:
      df_enterprise.loc[i,'Q']= 0
# R
# key word: 예술, 여가, 스포츠, 오락
for i in range(len(df_enterprise)):
  if ('예술' in df_enterprise.loc[i,'지원내용'])|('여가' in df_enterprise.loc[i,'지원내용'])|('스포츠' in df_enterprise.loc[i,'지원내용'])|('오락' in df_enterprise.loc[i,'지원내용']):
    df_enterprise.loc[i,'R']= 1
  else:
    if ('예술' in df_enterprise.loc[i,'지원대상'])|('여가' in df_enterprise.loc[i,'지원내용'])|('스포츠' in df_enterprise.loc[i,'지원내용'])|('오락' in df_enterprise.loc[i,'지원내용']):
      df_enterprise.loc[i,'R']= 1
    else:
      df_enterprise.loc[i,'R']= 0
# S
# key word: 협회, 수리업
for i in range(len(df_enterprise)):
  if ('협회/단체/기타서비스업' in df_enterprise.loc[i,'지원내용'])|('수리업' in df_enterprise.loc[i,'지원내용']):
    df_enterprise.loc[i,'S']= 1
  else:
    if ('협회/단체/기타서비스업' in df_enterprise.loc[i,'지원대상'])|('수리업' in df_enterprise.loc[i,'지원내용']):
      df_enterprise.loc[i,'S']= 1
    else:
      df_enterprise.loc[i,'S']= 0
# 업종 대분류 값 만들기
df_enterprise['업종(대분류)']=''

for i in range(len(df_enterprise)):
  values = df_enterprise.loc[i,'A':'S'].values
  if values.sum()==0 :
    df_enterprise.loc[i,'업종(대분류)']= '무관'
  else:
    for j in range(len(values)):
      if j == 0:
        if values[j]==1:
          df_enterprise.loc[i,'업종(대분류)']+=dict_main_category['A']
      elif j == 1:
        if values[j]==1:
          if df_enterprise.loc[i,'업종(대분류)']=='':
            df_enterprise.loc[i,'업종(대분류)']+=dict_main_category['B']
          else:
            df_enterprise.loc[i,'업종(대분류)']+='/'+dict_main_category['B']
  
      elif j == 2:
        if values[j]==1:
          if df_enterprise.loc[i,'업종(대분류)']=='':
            df_enterprise.loc[i,'업종(대분류)']+=dict_main_category['C']
          else:
            df_enterprise.loc[i,'업종(대분류)']+='/'+dict_main_category['C']
      elif j == 3:
        if values[j]==1:
          if df_enterprise.loc[i,'업종(대분류)']=='':
            df_enterprise.loc[i,'업종(대분류)']+=dict_main_category['D']
          else:
            df_enterprise.loc[i,'업종(대분류)']+='/'+dict_main_category['D']
      elif j == 4:
        if values[j]==1:
          if df_enterprise.loc[i,'업종(대분류)']=='':
            df_enterprise.loc[i,'업종(대분류)']+=dict_main_category['E']
          else:
            df_enterprise.loc[i,'업종(대분류)']+='/'+dict_main_category['E']
      elif j == 5:
        if values[j]==1:
          if df_enterprise.loc[i,'업종(대분류)']=='':
            df_enterprise.loc[i,'업종(대분류)']+=dict_main_category['F']
          else:
            df_enterprise.loc[i,'업종(대분류)']+='/'+dict_main_category['F']
      elif j == 6:
        if values[j]==1:
          if df_enterprise.loc[i,'업종(대분류)']=='':
            df_enterprise.loc[i,'업종(대분류)']+=dict_main_category['G']
          else:
            df_enterprise.loc[i,'업종(대분류)']+='/'+dict_main_category['G']
      elif j == 7:
        if values[j]==1:
          if df_enterprise.loc[i,'업종(대분류)']=='':
            df_enterprise.loc[i,'업종(대분류)']+=dict_main_category['H']
          else:
            df_enterprise.loc[i,'업종(대분류)']+='/'+dict_main_category['H']
      elif j == 8:
        if values[j]==1:
          if df_enterprise.loc[i,'업종(대분류)']=='':
            df_enterprise.loc[i,'업종(대분류)']+=dict_main_category['I']
          else:
            df_enterprise.loc[i,'업종(대분류)']+='/'+dict_main_category['I']
      elif j == 9:
        if values[j]==1:
          if df_enterprise.loc[i,'업종(대분류)']=='':
            df_enterprise.loc[i,'업종(대분류)']+=dict_main_category['J']
          else:
            df_enterprise.loc[i,'업종(대분류)']+='/'+dict_main_category['J']
      elif j == 10:
        if values[j]==1:
          if df_enterprise.loc[i,'업종(대분류)']=='':
            df_enterprise.loc[i,'업종(대분류)']+=dict_main_category['K']
          else:
            df_enterprise.loc[i,'업종(대분류)']+='/'+dict_main_category['K']
      elif j == 11:
        if values[j]==1:
          if df_enterprise.loc[i,'업종(대분류)']=='':
            df_enterprise.loc[i,'업종(대분류)']+=dict_main_category['L']
          else:
            df_enterprise.loc[i,'업종(대분류)']+='/'+dict_main_category['L']

      elif j == 12:
        if values[j]==1:
          if df_enterprise.loc[i,'업종(대분류)']=='':
            df_enterprise.loc[i,'업종(대분류)']+=dict_main_category['M']
          else:
            df_enterprise.loc[i,'업종(대분류)']+='/'+dict_main_category['M']

      elif j == 13:
        if values[j]==1:
          if df_enterprise.loc[i,'업종(대분류)']=='':
            df_enterprise.loc[i,'업종(대분류)']+=dict_main_category['N']
          else:
            df_enterprise.loc[i,'업종(대분류)']+='/'+dict_main_category['N']

      elif j == 14:
        if values[j]==1:
          if df_enterprise.loc[i,'업종(대분류)']=='':
            df_enterprise.loc[i,'업종(대분류)']+=dict_main_category['O']
          else:
            df_enterprise.loc[i,'업종(대분류)']+='/'+dict_main_category['O']

      elif j == 15:
        if values[j]==1:
          if df_enterprise.loc[i,'업종(대분류)']=='':
            df_enterprise.loc[i,'업종(대분류)']+=dict_main_category['P']
          else:
            df_enterprise.loc[i,'업종(대분류)']+='/'+dict_main_category['P']

      elif j == 16:
        if values[j]==1:
          if df_enterprise.loc[i,'업종(대분류)']=='':
            df_enterprise.loc[i,'업종(대분류)']+=dict_main_category['Q']
          else:
            df_enterprise.loc[i,'업종(대분류)']+='/'+dict_main_category['Q']

      elif j == 17:
        if values[j]==1:
          if df_enterprise.loc[i,'업종(대분류)']=='':
            df_enterprise.loc[i,'업종(대분류)']+=dict_main_category['R']
          else:
            df_enterprise.loc[i,'업종(대분류)']+='/'+dict_main_category['R']

      elif j == 18:
        if values[j]==1:
          if df_enterprise.loc[i,'업종(대분류)']=='':
            df_enterprise.loc[i,'업종(대분류)']+=dict_main_category['S']
          else:
            df_enterprise.loc[i,'업종(대분류)']+='/'+dict_main_category['S']
            
del df_enterprise['A']
del df_enterprise['B']
del df_enterprise['C']
del df_enterprise['D']
del df_enterprise['E']
del df_enterprise['F']
del df_enterprise['G']
del df_enterprise['H']
del df_enterprise['I']
del df_enterprise['J']
del df_enterprise['K']
del df_enterprise['L']
del df_enterprise['M']
del df_enterprise['N']
del df_enterprise['O']
del df_enterprise['P']
del df_enterprise['Q']
del df_enterprise['R']
del df_enterprise['S']

In [ ]:
df_enterprise.columns

Index(['정책ID', '정책서비스ID', '서비스명', '소관기관', '소관기관유형', '온라인신청사이트URL', '지원유형',
       '지원내용', '지원대상', '기업규모', '업종(대분류)'],
      dtype='object')

In [ ]:
df_good[df_good['태그-업종(대분류)']=='무관'].shape

(2542, 11)

### **3. 지역(시도)**

In [ ]:
# 지역(시도) 변수 생성
# 아래 시군구를 통해 지역(시도)값 더 보충한다.
def sido(df):
  for i in range(len(df)):
    AGENCY_TYPE = df.loc[i,'소관기관유형']
    # 소관기관 유형이 광역시도인 경우 소관기관을 시도값으로 넣는다.
    if AGENCY_TYPE=='광역시도':
      df.loc[i,'sido'] = df.loc[i,'소관기관']
    # 소관기관 유형이 시군구인 경우 띄어쓰기 앞에있는 시도값을 넣는다.
    elif AGENCY_TYPE=='시군구':
      sd = df.loc[i,'소관기관'].split(' ')[0]
      df.loc[i,'sido'] = sd
    # 소관기관 유형이 교육청인 경우 교육청을 제외한 값이 시도값이므로 넣는다.
    elif AGENCY_TYPE == '교육청':
      df.loc[i,'sido'] = df.loc[i,'소관기관'][:-3]
    # 이외의 값은 일단 전국으로 넣어준다.
    else:
      df.loc[i,'sido'] = '전국'
  return df
# 약 14초 소요 실행
sido(df_enterprise)

,정책ID,정책서비스ID,서비스명,소관기관,소관기관유형,신청절차,태그-신청절차,온라인신청사이트URL,지원유형,지원내용,지원대상,기업규모,업종(대분류),sido
0,4,116010000027,‘힘내라 대한민국’특별운영자금 (산업은행),금융위원회,중앙행정기관,○ 3.26일부터 기업의 신청접수ㆍ지원 중\n\n ➡ 산업은행 전국 영업점을 방문하...,오프라인,http://www.kdb.co.kr,현금(융자),자금용도 운영자금 한도 기존 대출한도 外 일정범위내 특별한도 부여(중견기업 최대 ...,"코로나19 등 질병, 자연재해 등 국가 재난 발생으로 피해를 입은 중소ㆍ중견기업...",소기업/중기업/중견기업,무관,전국
1,10,119200000016,원양어업 경영자금 지원,해양수산부,중앙행정기관,○ 우편접수(서울특별시 서초구 논현로 83 한국원양산업협회)\n\n○ 사업시행지침 ...,NaN,-,현금||현금(융자),어업경영자금 융자(수협은행 및 일선수협),"어업경영체(어업인, 어업법인 등)",무관,"농업, 임업 및 어업",전국
2,14,119200000024,수산장비 구입 지원,해양수산부,중앙행정기관,시군구청에 방문하여 신청,오프라인,-,현금(융자),융자한도액(1억원) 내에서 거래가격의 80% 지원,｢수산업ㆍ어촌 발전 기본법｣ 및 ｢농어업경영체 육성 및 지원에 관한 법률｣에서 규정...,무관,"농업, 임업 및 어업/제조업",전국
3,16,119200000027,산지 및 소비지 유통자금 융자 지원,해양수산부,중앙행정기관,우편 또는 방문신청\n수협중앙회 : 서울시 송파구 오금로 62 수협중앙회 경제기획부...,오프라인,-,현금(융자),"산지위판장 및 수산물 도매시장 어대금 결제자금, 직거래 자금 등 융자지원(금리 1...","산지위판장, 도매시장 법인, 중도매인 등",무관,도매 및 소매업,전국
4,18,119200000044,수산경영인회생자금,해양수산부,중앙행정기관,수협은행 및 일선수협 방문하여 신청,오프라인,-,현금||현금(융자),어업경영자금 등 정책자금에 대해 저리의 수산경영회생자금 대환(수협은행 및 일선수협),"어업경영체(어업인, 어업법인 등)",무관,"농업, 임업 및 어업",전국
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16931,88428,PTR000050134,장애인 직업생활상담원 양성교육,한국장애인고용공단,공공기관 등,○ 전화\n - 한국장애인고용공단 콜센터(☎ 031-728-7000)\n\n○ 상담...,온라인,https://cyedu.kead.or.kr/,정보제공,장애인 직업생활상담원 제도란? - 장애인 고용 촉진 및 직업재활법 제75조에 의거...,상시 20명 이상의 장애인 근로자를 고용하고 있는 사업주기업인중소기업/소상공인,소기업/중기업/소상공인,교육 서비스업/보건업 및 사회복지 서비스업,전국
16932,88430,PTR000050137,장애인 고용종사자 교육연수,한국장애인고용공단,공공기관 등,EDI 사이버연수원(http://cyedu.kead.or.kr )에서 회원가입 후 ...,온라인,http://cyedu.kead.or.kr,교육/용역,대상별 교육연구과정 - 사업체 임직원 · 장애인 직업생활상담원 양성 직장 내 장애...,"사업체 임직원, 장애인 고용종사자, 공단 임직원, 일반인 등구직자근로자기업인중소기업...",소기업/중기업/소상공인,"전문, 과학 및 기술 서비스업/교육 서비스업/보건업 및 사회복지 서비스업",전국
16933,88471,611000014088,농작물 병해충 예찰방제 지원,경기도 이천시,시군구,NaN,NaN,NaN,정보제공,지역별·병해충별 발생 정보를 활용한 맞춤형 방제정보 제공창업·경영>업종별민원>농업/...,"농업인, 산업체, 공공기관, 일반인 병해충 정보가 필요한 모든 국민 대상기업인중소...",소기업/중기업/소상공인,"농업, 임업 및 어업/공공 행정, 국방 및 사회보장 행정",경기도
16934,88479,369000000418,기본형 공익직접지불금 지원,울산광역시 중구,시군구,"○ 지원자격과 요건을 갖추고, 신청 제한기준에 해당되지 않을 경우, 사업계획서 등 ...",NaN,NaN,NaN,"환경보전, 농촌공동체 유지, 식품안정 등 공익기능 증진20210401~202105...",사업대상 지역에서 보험대상 목적물을 재배(사육)하는 농업인 또는 법인기업인중소기업...,소기업/중기업/소상공인,"농업, 임업 및 어업",울산광역시


### **4.지역(시군구)**

In [ ]:
# 1. 도와 시군구 테이블 생성
# 시군구에 대한 값을 추출해보았다.
DSGG = df_enterprise[df_enterprise['소관기관유형']=='시군구']['소관기관'].unique()
df_DSGG = pd.DataFrame(columns=['도','시군구'])

# 소관기관유형 시군구인 소관기관에서 각 도와 시군구값 뺴기
for i in range(len(DSGG)):
  val = DSGG[i].split(' ')
  df_DSGG.loc[i,'도'] = val[0]
  df_DSGG.loc[i,'시군구'] = val[1]

def namesigun(var):
  return var[:2]

df_DSGG['시군구명'] = df_DSGG['시군구'].apply(namesigun)

UNIQ = df_enterprise[df_enterprise['소관기관유형']=='공공기관 등']['소관기관'].unique()

# 2. 광역시도 관련된 광역시도명 테이블 만들기
sido_total = df_enterprise[df_enterprise['소관기관유형']=='광역시도']['소관기관'].unique()
df_sido_total = pd.DataFrame(columns=['광역시도','광역시도명'])

for i in range(len(sido_total)):
  val = sido_total[i]
  df_sido_total.loc[i,'광역시도']=val
  # if ('광역' in val) or ('특별' in val):
  #   df_sido_total.loc[i,'광역시도명']=val[:2]
  # else:
  df_sido_total.loc[i,'광역시도명']=val[:2]
  if ('북' in val) or ('남' in val):
    sum_val = val[0]+val[2]
    df_sido_total.loc[len(df_sido_total)]=[val,sum_val]

# 지역(시군구) 생성
def sigungu(df):
  name_gysd = df_sido_total['광역시도명'].values
  name_sgg = df_DSGG['시군구명'].values
  for i in range(len(df)):
    # 시군구인 경우엔 바로 시군구 값을 넣어준다
    if df.loc[i,'소관기관유형']=='시군구':
      sd = df.loc[i,'소관기관'].split(' ')[1]
      df.loc[i,'sigungu'] = sd
    # 공공기관의 경우에 해당하면
    elif df.loc[i,'소관기관유형']=='공공기관 등':
      # 소관기관의 이름과 앞의 두글자를 딴 변수를 구한다.
      val = df.loc[i,'소관기관']
      preval = df.loc[i,'소관기관'][:2]
      
      # 공공기관이면서 광역시 혹은 특별시가 들어간 경우
      if ('광역시' in val) or ('특별시' in val):
        # 00광역시로 시작하므로 시도값은 00광역시까지의 값
        df.loc[i,'sido'] = val[:5]
        # 00광역시00구 까지 써있는 경우엔 구값을 구해 시군구에 넣어준다.
        if '구' in val:
          num = val.find('구')
          SIGUN = val[5:num+1]
          df.loc[i,'sigungu'] = SIGUN
        # 00광역시만 있으면 구는 일단 NAN으로 처리
        else:
          df.loc[i,'sigungu'] = '전체'
      # 광주환경공단은 광역시가 아닌 광주광역시에서 주도하는 단체
      elif val == '광주환경공단':
        df.loc[i,'sido']='광주광역시'
        df.loc[i,'sigungu']='전체'
      # 앞의 두글자를 딴 변수 값이 위에서 만든 df_DSGG 시군구명에 들어간다면
      elif preval in name_sgg:
        # 해당 시군구명과 시도값을 넣어준다.
        val_df = df_DSGG[df_DSGG['시군구명']== preval]
        SIGUN = val_df.iloc[0,1]
        SIDO = val_df.iloc[0,0]
        df.loc[i,'sigungu']=SIGUN
        df.loc[i,'sido']=SIDO
      # 특정 광역시도가 들어간 곳 찾기
      elif preval in name_gysd:
        val_df = df_sido_total[df_sido_total['광역시도명']==preval]
        SIDO = val_df.iloc[0,0]
        df.loc[i,'sigungu']='전체'
        df.loc[i,'sido']=SIDO
      # 특정 광역시도가 들어간 곳 찾기
      elif preval in name_gysd:
        val_df = df_sido_total[df_sido_total['광역시도명']==preval]
        if len(val_df)>1:
          SIDO = val_df.iloc[0,0]
          df.loc[i,'sigungu']='전체'
          df.loc[i,'sido']=SIDO
        else:
          SIDO = val[:4]
          df.loc[i,'sigungu']='전체'
          df.loc[i,'sido']=SIDO
      # 그 외엔 시군구 NAN
      else:
        df.loc[i,'sigungu']='전체'
    else:
      df.loc[i,'sigungu']='전체'
  return df

sigungu(df_enterprise)

,정책ID,정책서비스ID,서비스명,소관기관,소관기관유형,신청절차,태그-신청절차,온라인신청사이트URL,지원유형,지원내용,지원대상,기업규모,업종(대분류),sido,sigungu
0,4,116010000027,‘힘내라 대한민국’특별운영자금 (산업은행),금융위원회,중앙행정기관,○ 3.26일부터 기업의 신청접수ㆍ지원 중\n\n ➡ 산업은행 전국 영업점을 방문하...,오프라인,http://www.kdb.co.kr,현금(융자),자금용도 운영자금 한도 기존 대출한도 外 일정범위내 특별한도 부여(중견기업 최대 ...,"코로나19 등 질병, 자연재해 등 국가 재난 발생으로 피해를 입은 중소ㆍ중견기업...",소기업/중기업/중견기업,무관,전국,전체
1,10,119200000016,원양어업 경영자금 지원,해양수산부,중앙행정기관,○ 우편접수(서울특별시 서초구 논현로 83 한국원양산업협회)\n\n○ 사업시행지침 ...,NaN,-,현금||현금(융자),어업경영자금 융자(수협은행 및 일선수협),"어업경영체(어업인, 어업법인 등)",무관,"농업, 임업 및 어업",전국,전체
2,14,119200000024,수산장비 구입 지원,해양수산부,중앙행정기관,시군구청에 방문하여 신청,오프라인,-,현금(융자),융자한도액(1억원) 내에서 거래가격의 80% 지원,｢수산업ㆍ어촌 발전 기본법｣ 및 ｢농어업경영체 육성 및 지원에 관한 법률｣에서 규정...,무관,"농업, 임업 및 어업/제조업",전국,전체
3,16,119200000027,산지 및 소비지 유통자금 융자 지원,해양수산부,중앙행정기관,우편 또는 방문신청\n수협중앙회 : 서울시 송파구 오금로 62 수협중앙회 경제기획부...,오프라인,-,현금(융자),"산지위판장 및 수산물 도매시장 어대금 결제자금, 직거래 자금 등 융자지원(금리 1...","산지위판장, 도매시장 법인, 중도매인 등",무관,도매 및 소매업,전국,전체
4,18,119200000044,수산경영인회생자금,해양수산부,중앙행정기관,수협은행 및 일선수협 방문하여 신청,오프라인,-,현금||현금(융자),어업경영자금 등 정책자금에 대해 저리의 수산경영회생자금 대환(수협은행 및 일선수협),"어업경영체(어업인, 어업법인 등)",무관,"농업, 임업 및 어업",전국,전체
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16931,88428,PTR000050134,장애인 직업생활상담원 양성교육,한국장애인고용공단,공공기관 등,○ 전화\n - 한국장애인고용공단 콜센터(☎ 031-728-7000)\n\n○ 상담...,온라인,https://cyedu.kead.or.kr/,정보제공,장애인 직업생활상담원 제도란? - 장애인 고용 촉진 및 직업재활법 제75조에 의거...,상시 20명 이상의 장애인 근로자를 고용하고 있는 사업주기업인중소기업/소상공인,소기업/중기업/소상공인,교육 서비스업/보건업 및 사회복지 서비스업,전국,전체
16932,88430,PTR000050137,장애인 고용종사자 교육연수,한국장애인고용공단,공공기관 등,EDI 사이버연수원(http://cyedu.kead.or.kr )에서 회원가입 후 ...,온라인,http://cyedu.kead.or.kr,교육/용역,대상별 교육연구과정 - 사업체 임직원 · 장애인 직업생활상담원 양성 직장 내 장애...,"사업체 임직원, 장애인 고용종사자, 공단 임직원, 일반인 등구직자근로자기업인중소기업...",소기업/중기업/소상공인,"전문, 과학 및 기술 서비스업/교육 서비스업/보건업 및 사회복지 서비스업",전국,전체
16933,88471,611000014088,농작물 병해충 예찰방제 지원,경기도 이천시,시군구,NaN,NaN,NaN,정보제공,지역별·병해충별 발생 정보를 활용한 맞춤형 방제정보 제공창업·경영>업종별민원>농업/...,"농업인, 산업체, 공공기관, 일반인 병해충 정보가 필요한 모든 국민 대상기업인중소...",소기업/중기업/소상공인,"농업, 임업 및 어업/공공 행정, 국방 및 사회보장 행정",경기도,이천시
16934,88479,369000000418,기본형 공익직접지불금 지원,울산광역시 중구,시군구,"○ 지원자격과 요건을 갖추고, 신청 제한기준에 해당되지 않을 경우, 사업계획서 등 ...",NaN,NaN,NaN,"환경보전, 농촌공동체 유지, 식품안정 등 공익기능 증진20210401~202105...",사업대상 지역에서 보험대상 목적물을 재배(사육)하는 농업인 또는 법인기업인중소기업...,소기업/중기업/소상공인,"농업, 임업 및 어업",울산광역시,중구


### **5. 지원유형**

In [ ]:
# \n과 ||와 ,,를 없애주는 함수
def clean_provision_type(var):
  var = var.replace('\n',',').replace('||',',').replace(',,',',')
  return var
  
# 지원유형 dtype이 object인데 중간에 Float이 있어 안 돌아갈 수 있으니 str 변환
df_enterprise['지원유형']=df_enterprise['지원유형'].astype(str)
df_enterprise['지원유형']=df_enterprise['지원유형'].apply(clean_provision_type)
df_enterprise=df_enterprise.rename(columns={'sido':'지역(시도)','sigungu':'지역(시군구)'})

### **6. 신청절차**

In [ ]:
# 온라인, 오프라인 여부를 따로 측정을 위한 열 생성
df_enterprise.loc[:,'온라인'] = np.NaN
df_enterprise.loc[:,'오프라인'] = np.NaN
df_enterprise.loc[:,'자동'] = np.NaN
df_enterprise.loc[:,'종합'] = np.NaN

# 일부 값은 float형이 있어, '신청절차'열을 문자열 형태로 변형
df_enterprise['신청절차'] = df_enterprise['신청절차'].astype(str)
df_enterprise['온라인신청사이트URL'] = df_enterprise['온라인신청사이트URL'].astype(str)

# 온/오프라인/자동 분류
## 온라인
for i in range(len(df_enterprise)):
    if (('페이지' in df_enterprise.loc[i,'신청절차'])|('사이트' in df_enterprise.loc[i,'신청절차'])
    |('이메일' in df_enterprise.loc[i,'신청절차'])|('인터넷' in df_enterprise.loc[i,'신청절차'])
    |('온라인' in df_enterprise.loc[i,'신청절차'])|('누리집' in df_enterprise.loc[i,'신청절차'])
    |('복지로' in df_enterprise.loc[i,'신청절차'])|('전산' in df_enterprise.loc[i,'신청절차'])
    |('E-mail' in df_enterprise.loc[i,'신청절차'])|('모바일' in df_enterprise.loc[i,'신청절차'])
    |('시스템' in df_enterprise.loc[i,'신청절차'])):
        df_enterprise.loc[i,'온라인']='온라인'
## 오프라인
for i in range(len(df_enterprise)):
    if (('방문' in df_enterprise.loc[i,'신청절차'])|('우편' in df_enterprise.loc[i,'신청절차'])
    |('오프라인' in df_enterprise.loc[i,'신청절차'])|('시·군·구' in df_enterprise.loc[i,'신청절차'])
    |('영업점' in df_enterprise.loc[i,'신청절차'])|('은행' in df_enterprise.loc[i,'신청절차'])
    |('사무소' in df_enterprise.loc[i,'신청절차'])|('기관' in df_enterprise.loc[i,'신청절차'])
    |('지자체' in df_enterprise.loc[i,'신청절차'])|('시․군․구' in df_enterprise.loc[i,'신청절차'])
    |('시‧군‧구' in df_enterprise.loc[i,'신청절차'])|('수협' in df_enterprise.loc[i,'신청절차'])
    |('위원회' in df_enterprise.loc[i,'신청절차'])|('센터' in df_enterprise.loc[i,'신청절차'])
    |('시군' in df_enterprise.loc[i,'신청절차'])|('검사' in df_enterprise.loc[i,'신청절차'])
    |('경찰' in df_enterprise.loc[i,'신청절차'])|('현장' in df_enterprise.loc[i,'신청절차'])
    |('시도' in df_enterprise.loc[i,'신청절차'])|('보건소' in df_enterprise.loc[i,'신청절차'])
    |('민간보조' in df_enterprise.loc[i,'신청절차'])|('유선' in df_enterprise.loc[i,'신청절차'])):
        df_enterprise.loc[i,'오프라인']='오프라인'
## 자동
for i in range(len(df_enterprise)):
    if (('통일부' in df_enterprise.loc[i,'신청절차'])|('하나원' in df_enterprise.loc[i,'신청절차'])
    |('통계교육원' in df_enterprise.loc[i,'신청절차'])|('대입원서' in df_enterprise.loc[i,'신청절차'])):
        df_enterprise.loc[i,'자동']='자동'
## '온라인신청사이트URL'의 값에 따른 온라인 분류
for i in range(len(df_enterprise)):
    if (('http' in df_enterprise.loc[i,'온라인신청사이트URL'])|('go' in df_enterprise.loc[i,'온라인신청사이트URL'])
    |('kr' in df_enterprise.loc[i,'온라인신청사이트URL'])|('@' in df_enterprise.loc[i,'온라인신청사이트URL'])
    |('인터넷' in df_enterprise.loc[i,'온라인신청사이트URL'])|('www' in df_enterprise.loc[i,'온라인신청사이트URL'])
    |('WWW' in df_enterprise.loc[i,'온라인신청사이트URL'])|('net' in df_enterprise.loc[i,'온라인신청사이트URL'])):
        df_enterprise.loc[i,'온라인']='온라인'
## '온라인신청사이트URL'의 값에 따른 오프라인 분류
for i in range(len(df_enterprise)):
    if (('방문' in df_enterprise.loc[i,'온라인신청사이트URL'])|('불가능' in df_enterprise.loc[i,'온라인신청사이트URL'])):
        df_enterprise.loc[i,'오프라인']='오프라인'        
    elif ((df_enterprise.loc[i,'온라인신청사이트URL'] == 'nan') or (df_enterprise.loc[i,'온라인신청사이트URL'] =='-')
    or (df_enterprise.loc[i,'온라인신청사이트URL'] == '정상섭')):
        df_enterprise.loc[i,'오프라인']='오프라인'

# 온라인 열에 '온라인'값과 오프라인 열에 '오프라인'값이 모두 반영되어 있는 경우: 온/오프라인 복합
# 온라인 열에만 '온라인'값이 반영된 경우: 온라인
# 오프라인 열에만 '오프라인'값이 반영된 경우: 오프라인
# 자동 열에만 '자동'값이 반영된 경우: 자동
for i in range(len(df_enterprise)):
    if df_enterprise['온라인'][i] == '온라인' and df_enterprise['오프라인'][i] == '오프라인':
        df_enterprise['종합'][i] = '복합'
    elif df_enterprise['온라인'][i] != '온라인' and df_enterprise['오프라인'][i] == '오프라인':
        df_enterprise['종합'][i] = '오프라인'
    elif df_enterprise['온라인'][i] == '온라인' and df_enterprise['오프라인'][i] != '오프라인':
        df_enterprise['종합'][i] = '온라인'
    elif df_enterprise['자동'][i] == '자동':
        df_enterprise['종합'][i] = '자동'

# # df_enterprise['복합']을 df_policy['태그-신청절차']로 사용
# # df_policy['태그-신청절차'] = df_enterprise['복합']

### **7. 업력**

In [ ]:
# '지원대상' 문자열 데이터타입으로 변경
df_enterprise['지원대상'] = df_enterprise['지원대상'].astype(str)

# 숫자 리스트 추출 및 업력 태그를 위한 변수 지정
df_enterprise['num_list'] = 0
df_enterprise['new_num_list'] = 0
df_enterprise['태그_업력(시작)']=np.NaN
df_enterprise['태그_업력(종료)']=np.NaN

# 지원 내용 값에서 숫자 리스트 추출
for i in range(len(df_enterprise)):
    df_enterprise['num_list'][i] = re.findall("\d+", df_enterprise['지원대상'][i])
df_enterprise['new_num_list'] = df_enterprise['num_list']

# 최소업력 추출
for i in range(len(df_enterprise)):
    for j in df_enterprise['num_list'][i]:
        if (('창업' in df_enterprise.loc[i, '지원대상'])|('사업 개시' in df_enterprise.loc[i, '지원대상'])|('업력' in df_enterprise.loc[i, '지원대상'])
        |('사업개시' in df_enterprise.loc[i, '지원대상'])|('사업시작' in df_enterprise.loc[i, '지원대상'])|('사업 시작' in df_enterprise.loc[i, '지원대상'])):
            if len(df_enterprise['num_list']) != 0:
                if (((str(j)+'년 ~') in df_enterprise.loc[i, '지원대상'])|((str(j)+'년~') in df_enterprise.loc[i, '지원대상'])
                |((str(j)+'이상') in df_enterprise.loc[i, '지원대상'])|((str(j)+'년이상') in df_enterprise.loc[i, '지원대상'])
                |((str(j)+'년 이상') in df_enterprise.loc[i, '지원대상'])):
                    df_enterprise['태그_업력(시작)'][i] = (int(j) * 12)
                elif (((str(j)+'개월 ~') in df_enterprise.loc[i, '지원대상'])|((str(j)+'개월~') in df_enterprise.loc[i, '지원대상'])
                |((str(j)+'이상') in df_enterprise.loc[i, '지원대상'])|((str(j)+'개월이상') in df_enterprise.loc[i, '지원대상'])
                |((str(j)+'개월 이상') in df_enterprise.loc[i, '지원대상'])):
                    df_enterprise['태그_업력(시작)'][i] = int(j)

# 최대업력 추출
for i in range(len(df_enterprise)):
    for j in df_enterprise['num_list'][i]:
        if (('창업' in df_enterprise.loc[i, '지원대상'])|('사업 개시' in df_enterprise.loc[i, '지원대상'])|('업력' in df_enterprise.loc[i, '지원대상'])
        |('사업개시' in df_enterprise.loc[i, '지원대상'])|('사업시작' in df_enterprise.loc[i, '지원대상'])|('사업 종료' in df_enterprise.loc[i, '지원대상'])):
            if len(df_enterprise['num_list']) != 0:
                if (((str(j)+'년 미만') in df_enterprise.loc[i, '지원대상'])|((str(j)+'년미만') in df_enterprise.loc[i, '지원대상'])
                |((str(j)+'년 이하') in df_enterprise.loc[i, '지원대상'])|((str(j)+'년이하') in df_enterprise.loc[i, '지원대상'])
                |((str(j)+'년이내') in df_enterprise.loc[i, '지원대상'])|((str(j)+'년 이내') in df_enterprise.loc[i, '지원대상'])):
                    df_enterprise['태그_업력(종료)'][i] = (int(j) * 12)
                elif (((str(j)+'개월 미만') in df_enterprise.loc[i, '지원대상'])|((str(j)+'개월미만') in df_enterprise.loc[i, '지원대상'])
                |((str(j)+'개월 이하') in df_enterprise.loc[i, '지원대상'])|((str(j)+'개월이하') in df_enterprise.loc[i, '지원대상'])
                |((str(j)+'개월이내') in df_enterprise.loc[i, '지원대상'])|((str(j)+'개월 이내') in df_enterprise.loc[i, '지원대상'])):
                    df_enterprise['태그_업력(종료)'][i] = int(j)

# 특정 대상이 있는 값은 업력 제외처리: 0으로 대체
for i in range(len(df_enterprise)):
    if (('임업인' in df_enterprise.loc[i, '지원대상'])|('농업인' in df_enterprise.loc[i, '지원대상'])|('어업인'in df_enterprise.loc[i, '지원대상'])
    |('근로자' in df_enterprise.loc[i, '지원대상'])):
        df_enterprise['태그_업력(시작)'][i] = np.NaN
        df_enterprise['태그_업력(종료)'][i] = np.NaN

## **3. 데이터 정리**

In [ ]:
df_good = df_enterprise[['정책ID','정책서비스ID','서비스명','종합', '지원유형', '지역(시도)', '지역(시군구)', '기업규모', '업종(대분류)','지원대상','지원내용','태그_업력(시작)','태그_업력(종료)']].copy().rename(columns={'종합':'태그-신청절차','지원유형':'태그-지원유형','지역(시도)':'태그-지역(시도)','지역(시군구)':'태그-지역(시군구)','기업규모':'태그-기업규모','업종(대분류)':'태그-업종(대분류)', '태그_업력(시작)':'태그-업력(시작)','태그_업력(종료)':'태그-업력(종료)'})

In [ ]:
df_good.to_csv('df_enterprise_ver2.csv',encoding='utf-8-sig',index=False)

# **III차. 데이터 Feature Engineering**

In [ ]:
import pandas as pd
import numpy as np
# 경고무시
import warnings
warnings.filterwarnings('ignore')
# 코드 실행시간 측정
import time
from datetime import timedelta
import re

In [ ]:
abc = pd.read_csv('/content/drive/MyDrive/Wello+project_신희택,윤다솜/Data/기업규모&노폐업.csv')
abc

,사업자등록번호,기업체명,기업기본주소,종업원수,기업설립일자,금융감독원법인고유번호,업종,법인등록번호,기업규모
0,3.128135e+09,(주)다코,충청남도 천안시 청당동 419-12,0,19970611.0,434003,제조업,1.615110e+12,중소기업
1,2.208193e+09,주식회사포스미디어,경기도 군포시 금정동 690-4,0,20000710.0,430229,제조업,1.101110e+12,중소기업
2,1.058181e+09,주식회사 예전미디어,서울특별시 마포구 서교동 347-18 윤우빌딩 지하,0,19960726.0,405658,정보통신업,1.101110e+12,중소기업
3,1.108135e+09,(주)승진씨앤씨,서울특별시 중구 서소문동 75,0,19721208.0,218229,부동산업,1.650110e+12,중소기업
4,2.020000e+02,글로벌렌탈 주식회사,서울특별시 강남구 삼성동 141-28 동신빌딩 4층,0,19950518.0,230407,"사업시설 관리, 사업 지원 및 임대 서비스업",1.101110e+12,중소기업
...,...,...,...,...,...,...,...,...,...
2228,2.148625e+09,주식회사 스틸코리아,서울특별시 서초구 양재동 67-7 동산빌딩 6층,0,19980817.0,444772,제조업,1.101110e+12,중소기업
2229,2.208626e+09,(주)인엑스하우징,서울특별시 강남구 대치동 891-23 대우아이빌명문가 235호,0,20011210.0,670252,부동산업,1.101110e+12,중소기업
2230,1.238635e+09,(주)우리테크놀로지,경기도 군포시 산본동 1-12 2층,0,20120907.0,977711,제조업,1.351110e+12,중소기업
2231,1.168175e+09,(주)에스앤제이케이,서울특별시 강남구 대치동 1008 길진빌딩,0,19990710.0,877697,부동산업,1.101110e+12,중소기업


In [ ]:
abc['업종'].unique()

array(['제조업', '정보통신업', '부동산업', '사업시설 관리, 사업 지원 및 임대 서비스업', '교육 서비스업',
       '금융 및 보험업', '전문, 과학 및 기술 서비스업', '건설업', '도매 및 소매업', '운수 및 창고업',
       '협회 및 단체, 수리 및 기타 개인 서비스업', '예술, 스포츠 및 여가관련 서비스업', '신용조합',
       '전기, 가스, 증기 및 공기 조절 공급업', '수도, 하수 및 폐기물 처리, 원료 재생업', '광업',
       '보건업 및 사회복지 서비스업', '공공 행정, 국방 및 사회보장 행정', '농업, 임업 및 어업'],
      dtype=object)

In [ ]:
# 약 3분 소요
industry_criteria = pd.read_excel('/content/drive/MyDrive/Wello+project_신희택,윤다솜/Data/(10차)산업분류_대분류항목값.xlsx').fillna(0)

# 대분류 항목명에 따른 딕셔너리 만들기
dict_industry_criteria=dict()
for i in range(len(industry_criteria)):
  if industry_criteria.loc[i,'대분류']!=0:
    main_category = industry_criteria.loc[i,'대분류']
    dict_industry_criteria[main_category]=[industry_criteria.loc[i,'항목명']]
  else:
    dict_industry_criteria[main_category].append(industry_criteria.loc[i,'항목명'])

In [ ]:
dict_industry_criteria

{'가구 내 고용활동 및 달리 분류되지 않은 자가 소비 생산활동': ['가구 내 고용활동',
  '자가 소비를 위한 가사 생산 활동',
  '자가 소비를 위한 가사 서비스 활동'],
 '건설업': ['건물 건설업',
  '토목 건설업',
  '기반조성 및 시설물 축조관련 전문공사업',
  '건물설비 설치 공사업',
  '전기 및 통신 공사업',
  '실내건축 및 건축마무리 공사업',
  '시설물 유지관리 공사업',
  '건설장비 운영업'],
 '공공 행정, 국방 및 사회보장 행정': ['입법 및 일반 정부 행정',
  '사회 및 산업정책 행정',
  '외무 및 국방 행정',
  '사법 및 공공 질서 행정',
  '사회보장 행정'],
 '광업': ['석탄 광업',
  '원유 및 천연가스 채굴업',
  '철 광업',
  '비철금속 광업',
  '토사석 광업',
  '기타 비금속광물 광업',
  '광업 지원 서비스업'],
 '교육 서비스업': ['초등 교육기관',
  '중등 교육기관',
  '고등 교육기관',
  '특수학교, 외국인학교 및 대안학교',
  '일반 교습학원',
  '기타 교육기관',
  '교육 지원 서비스업'],
 '국제 및 외국기관': ['국제 및 외국기관'],
 '금융 및 보험업': ['은행 및 저축기관',
  '신탁업 및 집합 투자업',
  '기타 금융업',
  '보험업',
  '재보험업',
  '연금 및 공제업',
  '금융 지원 서비스업',
  '보험 및 연금관련 서비스업'],
 '농업, 임업 및 어업': ['작물 재배업',
  '축산업',
  '작물재배 및 축산 복합농업',
  '작물재배 및 축산 관련 서비스업',
  '수렵 및 관련 서비스업',
  '임업',
  '어로 어업',
  '양식어업 및 어업관련 서비스업'],
 '도매 및 소매업': ['자동차 판매업',
  '자동차 부품 및 내장품 판매업',
  '모터사이클 및 부품 판매업',
  '상품 중개업',
  '산업용 농ㆍ축산물 및 동ㆍ식물 도매업',
  '음ㆍ식료품 및 담배 도매업',


In [ ]:
abc2 = []

In [ ]:
# 업종이 다른 것으로 되어있나 확인

# keys_industry = dict_industry_criteria.keys()
# for i in range(len(abc)):
#     if abc.loc[i,'업종'] in keys_industry:
#         pass
#     else:
#         abc2.append(abc.loc[i,'업종'])
# 결과 신용조합만 업종이 따로 분류되지 않음 => '금융, 보험업'으로 분류

In [ ]:
abc[abc['업종']=='신용조합']

In [ ]:
'신용조합' in dict_industry_criteria.values()

False

In [ ]:
keys_industry

dict_keys(['농업, 임업 및 어업', '광업', '제조업', '전기, 가스, 증기 및 공기 조절 공급업', '수도, 하수 및 폐기물 처리, 원료 재생업', '건설업', '도매 및 소매업', '운수 및 창고업', '숙박 및 음식점업', '정보통신업', '금융 및 보험업', '부동산업', '전문, 과학 및 기술 서비스업', '사업시설 관리, 사업 지원 및 임대 서비스업', '공공 행정, 국방 및 사회보장 행정', '교육 서비스업', '보건업 및 사회복지 서비스업', '예술, 스포츠 및 여가관련 서비스업', '협회 및 단체, 수리 및 기타 개인 서비스업', '가구 내 고용활동 및 달리 분류되지 않은 자가 소비 생산활동', '국제 및 외국기관'])

In [ ]:
dict_industry_criteria

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
p_ent_new = pd.read_csv('/content/drive/MyDrive/Wello+project_신희택,윤다솜/Data/p_ent_new.csv')
p_ent_new =p_ent_new.drop(['pid'])

In [ ]:
# ralral = p_ent_new.drop(['pid','신청방법','소관기관','접수기관명'])

,pid,서비스ID,서비스명,서비스대상지역,지원대상,지원내용,신청방법,소관기관,접수기관명,문의처전화번호,온라인신청사이트URL
0,29996,BIZC00070957,[경기] 양주시 2021년 3차 해외 규격인증지원사업 모집 공고(사후지원),NaN,NaN,사업개요\n\n\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t...,NaN,NaN,NaN,NaN,https://www.bizinfo.go.kr/see/seea/selectSEEA1...
1,29994,BIZC00070956,[광주] 양동전통시장 테마음식 핵 점포 추진 및 공간조성 참여 예비창업자 모집 공고,NaN,NaN,사업개요\n\n\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t...,NaN,NaN,NaN,NaN,https://www.bizinfo.go.kr/see/seea/selectSEEA1...
2,29989,BIZC00070955,2021년 청년몰 연말 오프라인 행사지원 참여 청년몰 모집 공고,NaN,NaN,사업개요\n\n\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t...,NaN,NaN,NaN,NaN,https://www.bizinfo.go.kr/see/seea/selectSEEA1...
3,29991,BIZC00070954,2022년 두바이 인터섹(소방산업) 전시회 한국공동관 참가기업 모집 공고,NaN,NaN,사업개요\n\n\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t...,NaN,NaN,NaN,NaN,https://www.bizinfo.go.kr/see/seea/selectSEEA1...
4,29984,BIZC00070953,2021년 ICT 기반 혁신제품 서비스 아이디어 Biz Project 공모전 공고,전국 / undefined,"중소ㆍ벤처기업(법인), 개인\n\nㅇ 지원자격\n- (Track1) 중소ㆍ벤처기업 ...",ㅇ 공모분야 : ICT분야 ‘단기(1.5년 이내)’ 사업화 가능한 ICT 혁신 제품...,- 신청 방법 : 온라인 접수\n정보통신기획평가원 사업관리시스템(iitp.kr)\n...,정보통신기획평가원,정보통신기획평가원 기업지원팀,042-612-8593,https://www.bizinfo.go.kr/see/seea/selectSEEA1...
...,...,...,...,...,...,...,...,...,...,...,...
998,1002,BIZC00069904,[부산] 2021년 크리에이터스튜디오 상상플러스부산 숏폼 콘텐츠 제작지원 사업 추가...,NaN,NaN,사업개요\n\n\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t...,: 이메일 접수- E-mail : tnqls8898@busanit.or.kr,NaN,NaN,NaN,https://www.bizinfo.go.kr/see/seea/selectSEEA1...
999,1001,BIZC00069903,[충북] 지역 중소기업 대상 찾아가는 R&D사업 1:1 코칭 안내,NaN,NaN,사업개요\n\n\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t...,: 이메일 접수- E-mail : ljw7594@korea.kr,NaN,NaN,NaN,https://www.bizinfo.go.kr/see/seea/selectSEEA1...
1000,1000,BIZC00069902,[대전] 2021년 나노융합분야 국내전시회 참가지원사업 추가모집 연장 공고,NaN,NaN,사업개요\n\n\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t...,: 온라인 접수- 대전테크노파크 사업정보관리시스템(pims.djtp.or.kr),NaN,NaN,NaN,https://www.bizinfo.go.kr/see/seea/selectSEEA1...
1001,999,BIZC00069901,2021년 2차 중소ㆍ중견기업 현장애로기술 지원사업 시행 공고,NaN,NaN,사업개요\n\n\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t...,: 이메일 제출\n- deonkim@kaeri.re.kr,NaN,NaN,NaN,https://www.bizinfo.go.kr/see/seea/selectSEEA1...


## **1. 지역**

In [ ]:
# 많이 언급되는 지원대상, ○, :, \n, | 등
# 특히 중위 소득=> 중위소득으로 바꿔줘야함.
def clean_support_target(var):
  var = var.lstrip()
  var = var.replace('사업개요','').replace('○','').replace(':','').replace('\t','').replace('\n','').replace('O','').replace('|','').replace('  ',' ').replace(' 소득','소득').replace('\xa0','').replace('*','')
  return var
p_ent_new['지원내용']=p_ent_new['지원내용'].apply(clean_support_target)

In [ ]:
df_sidosigungu = pd.read_excel('/content/drive/MyDrive/Wello+project_신희택,윤다솜/Data/시도-시군구.xlsx')
df_sidosigungu = df_sidosigungu.fillna(0)

# 시도에 따른 딕셔너리 만들기
dict_df_sidosigungu=dict()
for i in range(len(df_sidosigungu)):
  if df_sidosigungu.loc[i,'시도']!=0:
    main_category = df_sidosigungu.loc[i,'시도']
    dict_df_sidosigungu[main_category]=[df_sidosigungu.loc[i,'시군구']]
  else:
    dict_df_sidosigungu[main_category].append(df_sidosigungu.loc[i,'시군구'])
# 시도에 포함된 전체 구는 제외
for i in dict_df_sidosigungu.keys():
    if '전체' in dict_df_sidosigungu[i]:
        dict_df_sidosigungu[i].pop(0)

In [ ]:
# 축약된 시도와 시도값을 딕셔너리로 만들기
sido = pd.read_excel('/content/drive/MyDrive/Wello+project_신희택,윤다솜/Data/시도.xlsx').dropna()
sido['축약시도']=dict_df_sidosigungu.keys()

dict_convertsido = sido.set_index('축약시도').to_dict()['시도']
dict_convertsido

{'강원': '강원도',
 '경기': '경기도',
 '경남': '경상남도',
 '경북': '경상북도',
 '광주': '광주광역시',
 '대구': '대구광역시',
 '대전': '대전광역시',
 '부산': '부산광역시',
 '서울': '서울특별시',
 '세종': '세종특별자치시',
 '울산': '울산광역시',
 '인천': '인천광역시',
 '전남': '전라남도',
 '전북': '전라북도',
 '제주': '제주특별자치도',
 '충남': '충청남도',
 '충북': '충청북도'}

In [ ]:
# 해당 시도 키워드가 들어있으면 해당 시도의 시군구가 있으면 그 시군구 뽑기
# 이외 전국, 전체
p_ent_new['지역(시도)'] = ''
p_ent_new['지역(시군구)'] = ''
for i in range(len(p_ent_new)):
    sidos = dict_df_sidosigungu.keys()
    for j in sidos:
        if (j in p_ent_new.loc[i,'서비스명']):
            p_ent_new.loc[i,'지역(시도)']+= '/'+dict_convertsido[j]
            sigungu = dict_df_sidosigungu[j]
            for k in sigungu:
                if (k in p_ent_new.loc[i,'서비스명'])|(k in p_ent_new.loc[i,'지원내용']):
                    p_ent_new.loc[i,'지역(시군구)'] += '/'+ k
p_ent_new['지역(시도)']=p_ent_new['지역(시도)'].apply(lambda x: x[1:])
p_ent_new['지역(시군구)']=p_ent_new['지역(시군구)'].apply(lambda x: x[1:])
for i in range(len(p_ent_new)):
    if p_ent_new.loc[i,'지역(시도)']=='':
        if p_ent_new.loc[i,'지역(시군구)']=='':
            p_ent_new.loc[i,'지역(시도)']='전국'
            p_ent_new.loc[i,'지역(시군구)']='전체'
    elif p_ent_new.loc[i,'지역(시군구)']=='':
        p_ent_new.loc[i,'지역(시군구)']='전체'

## **2. 업종**

In [ ]:
p_ent_new_raw = p_ent_new.copy()

In [ ]:
# 약 3분 소요
industry_criteria = pd.read_excel('/content/drive/MyDrive/Wello+project_신희택,윤다솜/Data/(10차)산업분류_대분류항목값.xlsx').fillna(0)

# 대분류 항목명에 따른 딕셔너리 만들기
dict_industry_criteria=dict()
for i in range(len(industry_criteria)):
  if industry_criteria.loc[i,'대분류']!=0:
    main_category = industry_criteria.loc[i,'대분류']
    dict_industry_criteria[main_category]=[industry_criteria.loc[i,'항목명']]
  else:
    dict_industry_criteria[main_category].append(industry_criteria.loc[i,'항목명'])

# 대분류 항목코드(Alphabet)에 따른 딕셔너리 만들기
maincategory = industry_criteria['대분류'].unique()
maincategory=np.delete(maincategory,1)

import string 
alphabet_upper = string.ascii_uppercase[:-5]

dict_main_category=dict()
for i in range(len(alphabet_upper)):
  dict_main_category[alphabet_upper[i]]=maincategory[i]
dict_main_category
# A
# key word: 농업, 축산, 작물재배, 수렵, 작물 재배, 임업, 양식
for i in range(len(p_ent_new)):
  if ('농산물' in p_ent_new.loc[i,'지원내용'])|('수산물' in p_ent_new.loc[i,'지원내용'])|('송아지' in p_ent_new.loc[i,'지원내용'])|('임산물' in p_ent_new.loc[i,'지원내용'])|('농업' in p_ent_new.loc[i,'지원내용'])|('축산' in p_ent_new.loc[i,'지원내용'])|('작물재배' in p_ent_new.loc[i,'지원내용'])|('수렵' in p_ent_new.loc[i,'지원내용'])|('임업' in p_ent_new.loc[i,'지원내용'])|('어업' in p_ent_new.loc[i,'지원내용'])|('양식' in p_ent_new.loc[i,'지원내용']):
    p_ent_new.loc[i,'A']= 1
  else:
      p_ent_new.loc[i,'A']= 0
# B
# key word: 광업, 석탄, 원유, 채굴, 석탄, 금속, 광물
for i in range(len(p_ent_new)):
  if ('광업' in p_ent_new.loc[i,'지원내용'])|('석탄' in p_ent_new.loc[i,'지원내용'])|('원유' in p_ent_new.loc[i,'지원내용'])|('채굴' in p_ent_new.loc[i,'지원내용'])|('석탄' in p_ent_new.loc[i,'지원내용'])|('금속' in p_ent_new.loc[i,'지원내용'])|('광물' in p_ent_new.loc[i,'지원내용']):
    p_ent_new.loc[i,'B']= 1
  else:
      p_ent_new.loc[i,'B']= 0
# C
# key word: 제조, 도축, 가공
for i in range(len(p_ent_new)):
  if ('제조업' in p_ent_new.loc[i,'지원내용'])|('제조기업' in p_ent_new.loc[i,'지원내용'])|('광업' in p_ent_new.loc[i,'지원내용'])|('도축' in p_ent_new.loc[i,'지원내용'])|('가공' in p_ent_new.loc[i,'지원내용']):
    p_ent_new.loc[i,'C']= 1
  else:
      p_ent_new.loc[i,'C']= 0
# D
# key word: 전기, 가스, 배관
for i in range(len(p_ent_new)):
  if ('전기' in p_ent_new.loc[i,'지원내용'])|('가스' in p_ent_new.loc[i,'지원내용'])|('배관' in p_ent_new.loc[i,'지원내용']):
    p_ent_new.loc[i,'D']= 1
  else:
      p_ent_new.loc[i,'D']= 0
# E
# key word: 수도, 하수, 폐기물, 정화
for i in range(len(p_ent_new)):
  if ('수도' in p_ent_new.loc[i,'지원내용'])|('하수' in p_ent_new.loc[i,'지원내용'])|('폐기물' in p_ent_new.loc[i,'지원내용'])|('정화' in p_ent_new.loc[i,'지원내용']):
    p_ent_new.loc[i,'E']= 1
  else:
      p_ent_new.loc[i,'E']= 0
# F
# key word: 건설, 토목, 공사
for i in range(len(p_ent_new)):
  if ('건설' in p_ent_new.loc[i,'지원내용'])|('토목' in p_ent_new.loc[i,'지원내용'])|('공사' in p_ent_new.loc[i,'지원내용']):
    p_ent_new.loc[i,'F']= 1
  else:
      p_ent_new.loc[i,'F']= 0
# G
# key word: 도매, 소매, 판매
for i in range(len(p_ent_new)):
  if ('도매' in p_ent_new.loc[i,'지원내용'])|('소매' in p_ent_new.loc[i,'지원내용'])|('판매' in p_ent_new.loc[i,'지원내용']):
    p_ent_new.loc[i,'G']= 1
  else:
      p_ent_new.loc[i,'G']= 0
# H
# key word: 운수, 운송, 창고
for i in range(len(p_ent_new)):
  if ('수출' in p_ent_new.loc[i,'지원내용'])|('수입' in p_ent_new.loc[i,'지원내용'])|('운수' in p_ent_new.loc[i,'지원내용'])|('운송' in p_ent_new.loc[i,'지원내용'])|('창고' in p_ent_new.loc[i,'지원내용']):
    p_ent_new.loc[i,'H']= 1
  else:
      p_ent_new.loc[i,'H']= 0
# I
# key word: 숙박, 음식점, 음료점
for i in range(len(p_ent_new)):
  if ('숙박' in p_ent_new.loc[i,'지원내용'])|('음식점' in p_ent_new.loc[i,'지원내용'])|('음료점' in p_ent_new.loc[i,'지원내용']):
    p_ent_new.loc[i,'I']= 1
  else:
      p_ent_new.loc[i,'I']= 0
# J
# key word: 정보통신, 인쇄물, 출판, 방송, 프로그래밍
for i in range(len(p_ent_new)):
  if ('ICT' in p_ent_new.loc[i,'지원내용'])|('ict' in p_ent_new.loc[i,'지원내용'])|('정보통신' in p_ent_new.loc[i,'지원내용'])|('인쇄물' in p_ent_new.loc[i,'지원내용'])|('출판' in p_ent_new.loc[i,'지원내용'])|('방송' in p_ent_new.loc[i,'지원내용'])|('프로그래밍' in p_ent_new.loc[i,'지원내용']):
    p_ent_new.loc[i,'J']= 1
  else:
      p_ent_new.loc[i,'J']= 0
# K
# key word: 은행, 저축, 보험, 투자
for i in range(len(p_ent_new)):
  if ('은행업' in p_ent_new.loc[i,'지원내용'])|('저축업' in p_ent_new.loc[i,'지원내용'])|('보험업' in p_ent_new.loc[i,'지원내용'])|('투자업' in p_ent_new.loc[i,'지원내용']):
    p_ent_new.loc[i,'K']= 1
  else:
      p_ent_new.loc[i,'K']= 0
# L
# key word: 부동산
for i in range(len(p_ent_new)):
  if '부동산' in p_ent_new.loc[i,'지원내용']:
    p_ent_new.loc[i,'L']= 1
  else:
      p_ent_new.loc[i,'L']= 0
# M
# key word: 연구, 자연과학, 공학, 기술, 건축
for i in range(len(p_ent_new)):
  if ('연구' in p_ent_new.loc[i,'지원내용'])|('자연과학' in p_ent_new.loc[i,'지원내용'])|('공학' in p_ent_new.loc[i,'지원내용'])|('기술' in p_ent_new.loc[i,'지원내용'])|('건축' in p_ent_new.loc[i,'지원내용']):
    p_ent_new.loc[i,'M']= 1
  else:
      p_ent_new.loc[i,'M']= 0
# N
# key word: 시설관리, 시설 관리, 일자리, 청소, 여행사, 경비, 임대
for i in range(len(p_ent_new)):
  if ('시설관리' in p_ent_new.loc[i,'지원내용'])|('시설 관리' in p_ent_new.loc[i,'지원내용'])|('일자리' in p_ent_new.loc[i,'지원내용'])|('청소' in p_ent_new.loc[i,'지원내용'])|('여행사' in p_ent_new.loc[i,'지원내용'])|('경비' in p_ent_new.loc[i,'지원내용']):
    p_ent_new.loc[i,'N']= 1
  else:
      p_ent_new.loc[i,'N']= 0
# O
# key word: 행정, 국방, 정책,
for i in range(len(p_ent_new)):
  if ('행정' in p_ent_new.loc[i,'지원내용'])|('국방' in p_ent_new.loc[i,'지원내용'])|('국방' in p_ent_new.loc[i,'지원내용']):
    p_ent_new.loc[i,'O']= 1
  else:
      p_ent_new.loc[i,'O']= 0
# P
# key word: 교육, 학교
for i in range(len(p_ent_new)):
  if ('교육' in p_ent_new.loc[i,'지원내용'])|('학교' in p_ent_new.loc[i,'지원내용']):
    p_ent_new.loc[i,'P']= 1
  else:
      p_ent_new.loc[i,'P']= 0
# Q
# key word: 병원, 의원, 보건, 복지
for i in range(len(p_ent_new)):
  if ('병원' in p_ent_new.loc[i,'지원내용'])|('의원' in p_ent_new.loc[i,'지원내용'])|('보건' in p_ent_new.loc[i,'지원내용'])|('복지' in p_ent_new.loc[i,'지원내용']):
    p_ent_new.loc[i,'Q']= 1
  else:
      p_ent_new.loc[i,'Q']= 0
# R
# key word: 예술, 여가, 스포츠, 오락
for i in range(len(p_ent_new)):
  if ('예술' in p_ent_new.loc[i,'지원내용'])|('여가' in p_ent_new.loc[i,'지원내용'])|('스포츠' in p_ent_new.loc[i,'지원내용'])|('오락' in p_ent_new.loc[i,'지원내용']):
    p_ent_new.loc[i,'R']= 1
  else:
      p_ent_new.loc[i,'R']= 0
# S
# key word: 협회, 수리업
for i in range(len(p_ent_new)):
  if ('협회/단체/기타서비스업' in p_ent_new.loc[i,'지원내용'])|('수리업' in p_ent_new.loc[i,'지원내용']):
    p_ent_new.loc[i,'S']= 1
  else:
      p_ent_new.loc[i,'S']= 0

# 업종 대분류 값 만들기
p_ent_new['업종(대분류)']=''

for i in range(len(p_ent_new)):
  values = p_ent_new.loc[i,'A':'S'].values
  if values.sum()==0 :
    p_ent_new.loc[i,'업종(대분류)']= '무관'
  else:
    for j in range(len(values)):
      if j == 0:
        if values[j]==1:
          p_ent_new.loc[i,'업종(대분류)']+=dict_main_category['A']
      elif j == 1:
        if values[j]==1:
          if p_ent_new.loc[i,'업종(대분류)']=='':
            p_ent_new.loc[i,'업종(대분류)']+=dict_main_category['B']
          else:
            p_ent_new.loc[i,'업종(대분류)']+='/'+dict_main_category['B']
  
      elif j == 2:
        if values[j]==1:
          if p_ent_new.loc[i,'업종(대분류)']=='':
            p_ent_new.loc[i,'업종(대분류)']+=dict_main_category['C']
          else:
            p_ent_new.loc[i,'업종(대분류)']+='/'+dict_main_category['C']
      elif j == 3:
        if values[j]==1:
          if p_ent_new.loc[i,'업종(대분류)']=='':
            p_ent_new.loc[i,'업종(대분류)']+=dict_main_category['D']
          else:
            p_ent_new.loc[i,'업종(대분류)']+='/'+dict_main_category['D']
      elif j == 4:
        if values[j]==1:
          if p_ent_new.loc[i,'업종(대분류)']=='':
            p_ent_new.loc[i,'업종(대분류)']+=dict_main_category['E']
          else:
            p_ent_new.loc[i,'업종(대분류)']+='/'+dict_main_category['E']
      elif j == 5:
        if values[j]==1:
          if p_ent_new.loc[i,'업종(대분류)']=='':
            p_ent_new.loc[i,'업종(대분류)']+=dict_main_category['F']
          else:
            p_ent_new.loc[i,'업종(대분류)']+='/'+dict_main_category['F']
      elif j == 6:
        if values[j]==1:
          if p_ent_new.loc[i,'업종(대분류)']=='':
            p_ent_new.loc[i,'업종(대분류)']+=dict_main_category['G']
          else:
            p_ent_new.loc[i,'업종(대분류)']+='/'+dict_main_category['G']
      elif j == 7:
        if values[j]==1:
          if p_ent_new.loc[i,'업종(대분류)']=='':
            p_ent_new.loc[i,'업종(대분류)']+=dict_main_category['H']
          else:
            p_ent_new.loc[i,'업종(대분류)']+='/'+dict_main_category['H']
      elif j == 8:
        if values[j]==1:
          if p_ent_new.loc[i,'업종(대분류)']=='':
            p_ent_new.loc[i,'업종(대분류)']+=dict_main_category['I']
          else:
            p_ent_new.loc[i,'업종(대분류)']+='/'+dict_main_category['I']
      elif j == 9:
        if values[j]==1:
          if p_ent_new.loc[i,'업종(대분류)']=='':
            p_ent_new.loc[i,'업종(대분류)']+=dict_main_category['J']
          else:
            p_ent_new.loc[i,'업종(대분류)']+='/'+dict_main_category['J']
      elif j == 10:
        if values[j]==1:
          if p_ent_new.loc[i,'업종(대분류)']=='':
            p_ent_new.loc[i,'업종(대분류)']+=dict_main_category['K']
          else:
            p_ent_new.loc[i,'업종(대분류)']+='/'+dict_main_category['K']
      elif j == 11:
        if values[j]==1:
          if p_ent_new.loc[i,'업종(대분류)']=='':
            p_ent_new.loc[i,'업종(대분류)']+=dict_main_category['L']
          else:
            p_ent_new.loc[i,'업종(대분류)']+='/'+dict_main_category['L']

      elif j == 12:
        if values[j]==1:
          if p_ent_new.loc[i,'업종(대분류)']=='':
            p_ent_new.loc[i,'업종(대분류)']+=dict_main_category['M']
          else:
            p_ent_new.loc[i,'업종(대분류)']+='/'+dict_main_category['M']

      elif j == 13:
        if values[j]==1:
          if p_ent_new.loc[i,'업종(대분류)']=='':
            p_ent_new.loc[i,'업종(대분류)']+=dict_main_category['N']
          else:
            p_ent_new.loc[i,'업종(대분류)']+='/'+dict_main_category['N']

      elif j == 14:
        if values[j]==1:
          if p_ent_new.loc[i,'업종(대분류)']=='':
            p_ent_new.loc[i,'업종(대분류)']+=dict_main_category['O']
          else:
            p_ent_new.loc[i,'업종(대분류)']+='/'+dict_main_category['O']

      elif j == 15:
        if values[j]==1:
          if p_ent_new.loc[i,'업종(대분류)']=='':
            p_ent_new.loc[i,'업종(대분류)']+=dict_main_category['P']
          else:
            p_ent_new.loc[i,'업종(대분류)']+='/'+dict_main_category['P']

      elif j == 16:
        if values[j]==1:
          if p_ent_new.loc[i,'업종(대분류)']=='':
            p_ent_new.loc[i,'업종(대분류)']+=dict_main_category['Q']
          else:
            p_ent_new.loc[i,'업종(대분류)']+='/'+dict_main_category['Q']

      elif j == 17:
        if values[j]==1:
          if p_ent_new.loc[i,'업종(대분류)']=='':
            p_ent_new.loc[i,'업종(대분류)']+=dict_main_category['R']
          else:
            p_ent_new.loc[i,'업종(대분류)']+='/'+dict_main_category['R']

      elif j == 18:
        if values[j]==1:
          if p_ent_new.loc[i,'업종(대분류)']=='':
            p_ent_new.loc[i,'업종(대분류)']+=dict_main_category['S']
          else:
            p_ent_new.loc[i,'업종(대분류)']+='/'+dict_main_category['S']
            
del p_ent_new['A']
del p_ent_new['B']
del p_ent_new['C']
del p_ent_new['D']
del p_ent_new['E']
del p_ent_new['F']
del p_ent_new['G']
del p_ent_new['H']
del p_ent_new['I']
del p_ent_new['J']
del p_ent_new['K']
del p_ent_new['L']
del p_ent_new['M']
del p_ent_new['N']
del p_ent_new['O']
del p_ent_new['P']
del p_ent_new['Q']
del p_ent_new['R']
del p_ent_new['S']

In [ ]:
p_ent_new

,pid,서비스ID,서비스명,서비스대상지역,지원대상,지원내용,신청방법,소관기관,접수기관명,문의처전화번호,온라인신청사이트URL,지역(시도),지역(시군구),업종(대분류)
0,29996,BIZC00070957,[경기] 양주시 2021년 3차 해외 규격인증지원사업 모집 공고(사후지원),NaN,NaN,제품 신뢰성 제고를 통한 기업 경쟁력을 향상시키기 위하여 양주시 소재 중소제조기업을...,NaN,NaN,NaN,NaN,https://www.bizinfo.go.kr/see/seea/selectSEEA1...,경기도,양주시,제조업
1,29994,BIZC00070956,[광주] 양동전통시장 테마음식 핵 점포 추진 및 공간조성 참여 예비창업자 모집 공고,NaN,NaN,양동전통시장 내 테마음식 특화거리 조성하여 예비 창업자의 성공적인 창업을 위해 다양...,NaN,NaN,NaN,NaN,https://www.bizinfo.go.kr/see/seea/selectSEEA1...,광주광역시,서구,"농업, 임업 및 어업/공공 행정, 국방 및 사회보장 행정/교육 서비스업"
2,29989,BIZC00070955,2021년 청년몰 연말 오프라인 행사지원 참여 청년몰 모집 공고,NaN,NaN,청년몰 활성화 및 청년상인 매출확대 도모를 위해 '2021년 청년몰 연말 오프라인 ...,NaN,NaN,NaN,NaN,https://www.bizinfo.go.kr/see/seea/selectSEEA1...,전국,전체,무관
3,29991,BIZC00070954,2022년 두바이 인터섹(소방산업) 전시회 한국공동관 참가기업 모집 공고,NaN,NaN,국내 소방관련 제조업체의 해외시장 진출 확대 및 수출 촉진을 도모하고자 '2022년...,NaN,NaN,NaN,NaN,https://www.bizinfo.go.kr/see/seea/selectSEEA1...,전국,전체,"제조업/운수 및 창고업/전문, 과학 및 기술 서비스업"
4,29984,BIZC00070953,2021년 ICT 기반 혁신제품 서비스 아이디어 Biz Project 공모전 공고,전국 / undefined,"중소ㆍ벤처기업(법인), 개인\n\nㅇ 지원자격\n- (Track1) 중소ㆍ벤처기업 ...",ㅇ 공모분야 ICT분야 ‘단기(1.5년 이내)’ 사업화 가능한 ICT 혁신 제품ㆍ서...,- 신청 방법 : 온라인 접수\n정보통신기획평가원 사업관리시스템(iitp.kr)\n...,정보통신기획평가원,정보통신기획평가원 기업지원팀,042-612-8593,https://www.bizinfo.go.kr/see/seea/selectSEEA1...,전국,전체,"정보통신업/전문, 과학 및 기술 서비스업"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
998,1002,BIZC00069904,[부산] 2021년 크리에이터스튜디오 상상플러스부산 숏폼 콘텐츠 제작지원 사업 추가...,NaN,NaN,"부산 거주 또는 부산 출신 구독자 1,000명 이상 현업 크리에이터를 대상으로 숏폼...",: 이메일 접수- E-mail : tnqls8898@busanit.or.kr,NaN,NaN,NaN,https://www.bizinfo.go.kr/see/seea/selectSEEA1...,부산광역시,전체,무관
999,1001,BIZC00069903,[충북] 지역 중소기업 대상 찾아가는 R&D사업 1:1 코칭 안내,NaN,NaN,기술개발과제 수행에 따른 경험 부족으로 보고서 작성 및 사업비 집행 등에 어려움을 ...,: 이메일 접수- E-mail : ljw7594@korea.kr,NaN,NaN,NaN,https://www.bizinfo.go.kr/see/seea/selectSEEA1...,충청북도,전체,"전문, 과학 및 기술 서비스업"
1000,1000,BIZC00069902,[대전] 2021년 나노융합분야 국내전시회 참가지원사업 추가모집 연장 공고,NaN,NaN,대전지역 나노융합 관련 산업 중소기업의 제품 홍보 및 판로 확대를 도모하고자 202...,: 온라인 접수- 대전테크노파크 사업정보관리시스템(pims.djtp.or.kr),NaN,NaN,NaN,https://www.bizinfo.go.kr/see/seea/selectSEEA1...,대전광역시,전체,"운수 및 창고업/숙박 및 음식점업/전문, 과학 및 기술 서비스업"
1001,999,BIZC00069901,2021년 2차 중소ㆍ중견기업 현장애로기술 지원사업 시행 공고,NaN,NaN,"신제품 개발, 기존 제품 개선, 공정 개선, 자격 및 인증 획득, 기술이전 후 상용...",: 이메일 제출\n- deonkim@kaeri.re.kr,NaN,NaN,NaN,https://www.bizinfo.go.kr/see/seea/selectSEEA1...,전국,전체,"전문, 과학 및 기술 서비스업"


## **3. 기업규모**

In [ ]:
p_ent_new['지원대상']=p_ent_new['지원대상'].astype(str)

In [ ]:
for i in range(len(p_ent_new)):
  if '소기업' in p_ent_new.loc[i,'지원내용']:
    p_ent_new.loc[i,'소기업']= 1
  else:
    if '소기업' in p_ent_new.loc[i,'서비스명']:
      p_ent_new.loc[i,'소기업']= 1
    else:
      p_ent_new.loc[i,'소기업']= 0
for i in range(len(p_ent_new)):
  if '중기업' in p_ent_new.loc[i,'지원내용']:
    p_ent_new.loc[i,'중기업']= 1
  else:
    if '중기업' in p_ent_new.loc[i,'서비스명']:
      p_ent_new.loc[i,'중기업']= 1
    else:
      p_ent_new.loc[i,'중기업']= 0
for i in range(len(p_ent_new)):
  if ('중소' in p_ent_new.loc[i,'지원내용'])|('중소기업' in p_ent_new.loc[i,'지원내용'])|('중소 기업' in p_ent_new.loc[i,'지원내용']):
    p_ent_new.loc[i,'소기업']= 1
    p_ent_new.loc[i,'중기업']= 1
  else:
    if ('중소' in p_ent_new.loc[i,'서비스명'])|('중소기업' in p_ent_new.loc[i,'서비스명']):
      p_ent_new.loc[i,'소기업']= 1
      p_ent_new.loc[i,'중기업']= 1
for i in range(len(p_ent_new)):
  if ('중견' in p_ent_new.loc[i,'지원내용'])|('중견기업' in p_ent_new.loc[i,'지원내용']):
    p_ent_new.loc[i,'중견기업']= 1
  else:
    if ('중견' in p_ent_new.loc[i,'서비스명'])|('중견기업' in p_ent_new.loc[i,'서비스명']):
      p_ent_new.loc[i,'중견기업']= 1
    else:
      p_ent_new.loc[i,'중견기업']= 0
for i in range(len(p_ent_new)):
  if '대기업' in p_ent_new.loc[i,'지원내용']:
    p_ent_new.loc[i,'대기업']= 1
  else:
    if '대기업' in p_ent_new.loc[i,'서비스명']:
      p_ent_new.loc[i,'대기업']= 1
    else:
      p_ent_new.loc[i,'대기업']= 0
for i in range(len(p_ent_new)):
  if ('소상공인' in p_ent_new.loc[i,'지원내용'])|('소상 공인' in p_ent_new.loc[i,'지원내용']):
    p_ent_new.loc[i,'소상공인']= 1
  else:
    if '소상공인' in p_ent_new.loc[i,'서비스명']:
      p_ent_new.loc[i,'소상공인']= 1
    else:
      p_ent_new.loc[i,'소상공인']= 0
# 기업규모 값 만들기
p_ent_new['기업규모']=''

for i in range(len(p_ent_new)):
  values = p_ent_new.loc[i,'소기업':'소상공인'].values
  if values.sum()==0 :
    p_ent_new.loc[i,'기업규모']= '무관'
  else:
    for j in range(len(values)):
      if j == 0:
        if values[j]==1:
          p_ent_new.loc[i,'기업규모']+='소기업'
      elif j == 1:
        if values[j]==1:
          if p_ent_new.loc[i,'기업규모']=='':
            p_ent_new.loc[i,'기업규모']+='중기업'
          else:
            p_ent_new.loc[i,'기업규모']+='/중기업'
  
      elif j == 2:
        if values[j]==1:
          if p_ent_new.loc[i,'기업규모']=='':
            p_ent_new.loc[i,'기업규모']+='중견기업'
          else:
            p_ent_new.loc[i,'기업규모']+='/중견기업'
      elif j == 3:
        if values[j]==1:
          if p_ent_new.loc[i,'기업규모']=='':
            p_ent_new.loc[i,'기업규모']+='대기업'
          else:
            p_ent_new.loc[i,'기업규모']+='/대기업'
      elif j == 4:
        if values[j]==1:
          if p_ent_new.loc[i,'기업규모']=='':
            p_ent_new.loc[i,'기업규모']+='소상공인'
          else:
            p_ent_new.loc[i,'기업규모']+='/소상공인'
      
            
del p_ent_new['소기업']
del p_ent_new['중기업']
del p_ent_new['중견기업']
del p_ent_new['대기업']
del p_ent_new['소상공인']

In [ ]:
p_ent_new

,pid,서비스ID,서비스명,서비스대상지역,지원대상,지원내용,신청방법,소관기관,접수기관명,문의처전화번호,온라인신청사이트URL,지역(시도),지역(시군구),업종(대분류),기업규모
0,29996,BIZC00070957,[경기] 양주시 2021년 3차 해외 규격인증지원사업 모집 공고(사후지원),NaN,nan,제품 신뢰성 제고를 통한 기업 경쟁력을 향상시키기 위하여 양주시 소재 중소제조기업을...,NaN,NaN,NaN,NaN,https://www.bizinfo.go.kr/see/seea/selectSEEA1...,경기도,양주시,제조업,소기업/중기업
1,29994,BIZC00070956,[광주] 양동전통시장 테마음식 핵 점포 추진 및 공간조성 참여 예비창업자 모집 공고,NaN,nan,양동전통시장 내 테마음식 특화거리 조성하여 예비 창업자의 성공적인 창업을 위해 다양...,NaN,NaN,NaN,NaN,https://www.bizinfo.go.kr/see/seea/selectSEEA1...,광주광역시,서구,"농업, 임업 및 어업/공공 행정, 국방 및 사회보장 행정/교육 서비스업",소기업/중기업/소상공인
2,29989,BIZC00070955,2021년 청년몰 연말 오프라인 행사지원 참여 청년몰 모집 공고,NaN,nan,청년몰 활성화 및 청년상인 매출확대 도모를 위해 '2021년 청년몰 연말 오프라인 ...,NaN,NaN,NaN,NaN,https://www.bizinfo.go.kr/see/seea/selectSEEA1...,전국,전체,무관,소상공인
3,29991,BIZC00070954,2022년 두바이 인터섹(소방산업) 전시회 한국공동관 참가기업 모집 공고,NaN,nan,국내 소방관련 제조업체의 해외시장 진출 확대 및 수출 촉진을 도모하고자 '2022년...,NaN,NaN,NaN,NaN,https://www.bizinfo.go.kr/see/seea/selectSEEA1...,전국,전체,"제조업/운수 및 창고업/전문, 과학 및 기술 서비스업",무관
4,29984,BIZC00070953,2021년 ICT 기반 혁신제품 서비스 아이디어 Biz Project 공모전 공고,전국 / undefined,"중소ㆍ벤처기업(법인), 개인\n\nㅇ 지원자격\n- (Track1) 중소ㆍ벤처기업 ...",ㅇ 공모분야 ICT분야 ‘단기(1.5년 이내)’ 사업화 가능한 ICT 혁신 제품ㆍ서...,- 신청 방법 : 온라인 접수\n정보통신기획평가원 사업관리시스템(iitp.kr)\n...,정보통신기획평가원,정보통신기획평가원 기업지원팀,042-612-8593,https://www.bizinfo.go.kr/see/seea/selectSEEA1...,전국,전체,"정보통신업/전문, 과학 및 기술 서비스업",무관
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
998,1002,BIZC00069904,[부산] 2021년 크리에이터스튜디오 상상플러스부산 숏폼 콘텐츠 제작지원 사업 추가...,NaN,nan,"부산 거주 또는 부산 출신 구독자 1,000명 이상 현업 크리에이터를 대상으로 숏폼...",: 이메일 접수- E-mail : tnqls8898@busanit.or.kr,NaN,NaN,NaN,https://www.bizinfo.go.kr/see/seea/selectSEEA1...,부산광역시,전체,무관,무관
999,1001,BIZC00069903,[충북] 지역 중소기업 대상 찾아가는 R&D사업 1:1 코칭 안내,NaN,nan,기술개발과제 수행에 따른 경험 부족으로 보고서 작성 및 사업비 집행 등에 어려움을 ...,: 이메일 접수- E-mail : ljw7594@korea.kr,NaN,NaN,NaN,https://www.bizinfo.go.kr/see/seea/selectSEEA1...,충청북도,전체,"전문, 과학 및 기술 서비스업",소기업/중기업
1000,1000,BIZC00069902,[대전] 2021년 나노융합분야 국내전시회 참가지원사업 추가모집 연장 공고,NaN,nan,대전지역 나노융합 관련 산업 중소기업의 제품 홍보 및 판로 확대를 도모하고자 202...,: 온라인 접수- 대전테크노파크 사업정보관리시스템(pims.djtp.or.kr),NaN,NaN,NaN,https://www.bizinfo.go.kr/see/seea/selectSEEA1...,대전광역시,전체,"운수 및 창고업/숙박 및 음식점업/전문, 과학 및 기술 서비스업",소기업/중기업
1001,999,BIZC00069901,2021년 2차 중소ㆍ중견기업 현장애로기술 지원사업 시행 공고,NaN,nan,"신제품 개발, 기존 제품 개선, 공정 개선, 자격 및 인증 획득, 기술이전 후 상용...",: 이메일 제출\n- deonkim@kaeri.re.kr,NaN,NaN,NaN,https://www.bizinfo.go.kr/see/seea/selectSEEA1...,전국,전체,"전문, 과학 및 기술 서비스업",소기업/중기업/중견기업/대기업


## **4. 신청절차**

## **5. 업력**

In [ ]:
# '지원내용' 문자열 데이터타입으로 변경
p_ent_new['지원내용'] = p_ent_new['지원내용'].astype(str)

# 숫자 리스트 추출 및 업력 태그를 위한 변수 지정
p_ent_new['num_list'] = 0
p_ent_new['new_num_list'] = 0
p_ent_new['태그_업력(시작)']=np.NaN
p_ent_new['태그_업력(종료)']=np.NaN

# 지원 내용 값에서 숫자 리스트 추출
for i in range(len(p_ent_new)):
    p_ent_new['num_list'][i] = re.findall("\d+", p_ent_new['지원내용'][i])
p_ent_new['new_num_list'] = p_ent_new['num_list']

# 최소업력 추출
for i in range(len(p_ent_new)):
    for j in p_ent_new['num_list'][i]:
        if (('창업' in p_ent_new.loc[i, '지원내용'])|('사업 개시' in p_ent_new.loc[i, '지원내용'])|('업력' in p_ent_new.loc[i, '지원내용'])
        |('사업개시' in p_ent_new.loc[i, '지원내용'])|('사업시작' in p_ent_new.loc[i, '지원내용'])|('사업 시작' in p_ent_new.loc[i, '지원내용'])):
            if len(p_ent_new['num_list']) != 0:
                if (((str(j)+'년 ~') in p_ent_new.loc[i, '지원내용'])|((str(j)+'년~') in p_ent_new.loc[i, '지원내용'])
                |((str(j)+'이상') in p_ent_new.loc[i, '지원내용'])|((str(j)+'년이상') in p_ent_new.loc[i, '지원내용'])
                |((str(j)+'년 이상') in p_ent_new.loc[i, '지원내용'])):
                    p_ent_new['태그_업력(시작)'][i] = (int(j) * 12)
                elif (((str(j)+'개월 ~') in p_ent_new.loc[i, '지원내용'])|((str(j)+'개월~') in p_ent_new.loc[i, '지원내용'])
                |((str(j)+'이상') in p_ent_new.loc[i, '지원내용'])|((str(j)+'개월이상') in p_ent_new.loc[i, '지원내용'])
                |((str(j)+'개월 이상') in p_ent_new.loc[i, '지원내용'])):
                    p_ent_new['태그_업력(시작)'][i] = int(j)

# 최대업력 추출
for i in range(len(p_ent_new)):
    for j in p_ent_new['num_list'][i]:
        if (('창업' in p_ent_new.loc[i, '지원내용'])|('사업 개시' in p_ent_new.loc[i, '지원내용'])|('업력' in p_ent_new.loc[i, '지원내용'])
        |('사업개시' in p_ent_new.loc[i, '지원내용'])|('사업시작' in p_ent_new.loc[i, '지원내용'])|('사업 종료' in p_ent_new.loc[i, '지원내용'])):
            if len(p_ent_new['num_list']) != 0:
                if (((str(j)+'년 미만') in p_ent_new.loc[i, '지원내용'])|((str(j)+'년미만') in p_ent_new.loc[i, '지원내용'])
                |((str(j)+'년 이하') in p_ent_new.loc[i, '지원내용'])|((str(j)+'년이하') in p_ent_new.loc[i, '지원내용'])
                |((str(j)+'년이내') in p_ent_new.loc[i, '지원내용'])|((str(j)+'년 이내') in p_ent_new.loc[i, '지원내용'])):
                    p_ent_new['태그_업력(종료)'][i] = (int(j) * 12)
                elif (((str(j)+'개월 미만') in p_ent_new.loc[i, '지원내용'])|((str(j)+'개월미만') in p_ent_new.loc[i, '지원내용'])
                |((str(j)+'개월 이하') in p_ent_new.loc[i, '지원내용'])|((str(j)+'개월이하') in p_ent_new.loc[i, '지원내용'])
                |((str(j)+'개월이내') in p_ent_new.loc[i, '지원내용'])|((str(j)+'개월 이내') in p_ent_new.loc[i, '지원내용'])):
                    p_ent_new['태그_업력(종료)'][i] = int(j)

# 특정 대상이 있는 값은 업력 제외처리: 0으로 대체
for i in range(len(p_ent_new)):
    if (('임업인' in p_ent_new.loc[i, '지원내용'])|('농업인' in p_ent_new.loc[i, '지원내용'])|('어업인'in p_ent_new.loc[i, '지원내용'])
    |('근로자' in p_ent_new.loc[i, '지원내용'])):
        p_ent_new['태그_업력(시작)'][i] = np.NaN
        p_ent_new['태그_업력(종료)'][i] = np.NaN
del p_ent_new['num_list']
del p_ent_new['new_num_list']
p_ent_new['태그_업력(시작)']=p_ent_new['태그_업력(시작)'].fillna(0)
p_ent_new['태그_업력(종료)']=p_ent_new['태그_업력(종료)'].fillna(0)

In [ ]:
p_ent_new

,pid,서비스ID,서비스명,서비스대상지역,지원대상,지원내용,신청방법,소관기관,접수기관명,문의처전화번호,온라인신청사이트URL,지역(시도),지역(시군구),업종(대분류),기업규모,태그_업력(시작),태그_업력(종료)
0,29996,BIZC00070957,[경기] 양주시 2021년 3차 해외 규격인증지원사업 모집 공고(사후지원),NaN,nan,제품 신뢰성 제고를 통한 기업 경쟁력을 향상시키기 위하여 양주시 소재 중소제조기업을...,NaN,NaN,NaN,NaN,https://www.bizinfo.go.kr/see/seea/selectSEEA1...,경기도,양주시,제조업,소기업/중기업,0.0,0.0
1,29994,BIZC00070956,[광주] 양동전통시장 테마음식 핵 점포 추진 및 공간조성 참여 예비창업자 모집 공고,NaN,nan,양동전통시장 내 테마음식 특화거리 조성하여 예비 창업자의 성공적인 창업을 위해 다양...,NaN,NaN,NaN,NaN,https://www.bizinfo.go.kr/see/seea/selectSEEA1...,광주광역시,서구,"농업, 임업 및 어업/공공 행정, 국방 및 사회보장 행정/교육 서비스업",소기업/중기업/소상공인,0.0,0.0
2,29989,BIZC00070955,2021년 청년몰 연말 오프라인 행사지원 참여 청년몰 모집 공고,NaN,nan,청년몰 활성화 및 청년상인 매출확대 도모를 위해 '2021년 청년몰 연말 오프라인 ...,NaN,NaN,NaN,NaN,https://www.bizinfo.go.kr/see/seea/selectSEEA1...,전국,전체,무관,소상공인,0.0,0.0
3,29991,BIZC00070954,2022년 두바이 인터섹(소방산업) 전시회 한국공동관 참가기업 모집 공고,NaN,nan,국내 소방관련 제조업체의 해외시장 진출 확대 및 수출 촉진을 도모하고자 '2022년...,NaN,NaN,NaN,NaN,https://www.bizinfo.go.kr/see/seea/selectSEEA1...,전국,전체,"제조업/운수 및 창고업/전문, 과학 및 기술 서비스업",무관,0.0,0.0
4,29984,BIZC00070953,2021년 ICT 기반 혁신제품 서비스 아이디어 Biz Project 공모전 공고,전국 / undefined,"중소ㆍ벤처기업(법인), 개인\n\nㅇ 지원자격\n- (Track1) 중소ㆍ벤처기업 ...",ㅇ 공모분야 ICT분야 ‘단기(1.5년 이내)’ 사업화 가능한 ICT 혁신 제품ㆍ서...,- 신청 방법 : 온라인 접수\n정보통신기획평가원 사업관리시스템(iitp.kr)\n...,정보통신기획평가원,정보통신기획평가원 기업지원팀,042-612-8593,https://www.bizinfo.go.kr/see/seea/selectSEEA1...,전국,전체,"정보통신업/전문, 과학 및 기술 서비스업",무관,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
998,1002,BIZC00069904,[부산] 2021년 크리에이터스튜디오 상상플러스부산 숏폼 콘텐츠 제작지원 사업 추가...,NaN,nan,"부산 거주 또는 부산 출신 구독자 1,000명 이상 현업 크리에이터를 대상으로 숏폼...",: 이메일 접수- E-mail : tnqls8898@busanit.or.kr,NaN,NaN,NaN,https://www.bizinfo.go.kr/see/seea/selectSEEA1...,부산광역시,전체,무관,무관,0.0,0.0
999,1001,BIZC00069903,[충북] 지역 중소기업 대상 찾아가는 R&D사업 1:1 코칭 안내,NaN,nan,기술개발과제 수행에 따른 경험 부족으로 보고서 작성 및 사업비 집행 등에 어려움을 ...,: 이메일 접수- E-mail : ljw7594@korea.kr,NaN,NaN,NaN,https://www.bizinfo.go.kr/see/seea/selectSEEA1...,충청북도,전체,"전문, 과학 및 기술 서비스업",소기업/중기업,0.0,0.0
1000,1000,BIZC00069902,[대전] 2021년 나노융합분야 국내전시회 참가지원사업 추가모집 연장 공고,NaN,nan,대전지역 나노융합 관련 산업 중소기업의 제품 홍보 및 판로 확대를 도모하고자 202...,: 온라인 접수- 대전테크노파크 사업정보관리시스템(pims.djtp.or.kr),NaN,NaN,NaN,https://www.bizinfo.go.kr/see/seea/selectSEEA1...,대전광역시,전체,"운수 및 창고업/숙박 및 음식점업/전문, 과학 및 기술 서비스업",소기업/중기업,0.0,0.0
1001,999,BIZC00069901,2021년 2차 중소ㆍ중견기업 현장애로기술 지원사업 시행 공고,NaN,nan,"신제품 개발, 기존 제품 개선, 공정 개선, 자격 및 인증 획득, 기술이전 후 상용...",: 이메일 제출\n- deonkim@kaeri.re.kr,NaN,NaN,NaN,https://www.bizinfo.go.kr/see/seea/selectSEEA1...,전국,전체,"전문, 과학 및 기술 서비스업",소기업/중기업/중견기업/대기업,0.0,6.0


In [ ]:
p_ent_new

,pid,서비스ID,서비스명,서비스대상지역,지원대상,지원내용,신청방법,소관기관,접수기관명,문의처전화번호,온라인신청사이트URL,지역(시도),지역(시군구),업종(대분류),기업규모,태그_업력(시작),태그_업력(종료),신청절차,"현금,현물",정보제공,복지,"민원,상담,법률","봉사,기부","교육,용역",지원유형
0,29996,BIZC00070957,[경기] 양주시 2021년 3차 해외 규격인증지원사업 모집 공고(사후지원),NaN,nan,제품 신뢰성 제고를 통한 기업 경쟁력을 향상시키기 위하여 양주시 소재 중소제조기업을...,NaN,NaN,NaN,NaN,https://www.bizinfo.go.kr/see/seea/selectSEEA1...,경기도,양주시,제조업,소기업/중기업,0.0,0.0,복합,NaN,1.0,1.0,NaN,NaN,NaN,정보제공/복지
1,29994,BIZC00070956,[광주] 양동전통시장 테마음식 핵 점포 추진 및 공간조성 참여 예비창업자 모집 공고,NaN,nan,양동전통시장 내 테마음식 특화거리 조성하여 예비 창업자의 성공적인 창업을 위해 다양...,NaN,NaN,NaN,NaN,https://www.bizinfo.go.kr/see/seea/selectSEEA1...,광주광역시,서구,"농업, 임업 및 어업/공공 행정, 국방 및 사회보장 행정/교육 서비스업",소기업/중기업/소상공인,0.0,0.0,복합,NaN,NaN,NaN,NaN,NaN,1.0,"교육,용역"
2,29989,BIZC00070955,2021년 청년몰 연말 오프라인 행사지원 참여 청년몰 모집 공고,NaN,nan,청년몰 활성화 및 청년상인 매출확대 도모를 위해 '2021년 청년몰 연말 오프라인 ...,NaN,NaN,NaN,NaN,https://www.bizinfo.go.kr/see/seea/selectSEEA1...,전국,전체,무관,소상공인,0.0,0.0,복합,NaN,NaN,NaN,NaN,NaN,NaN,
3,29991,BIZC00070954,2022년 두바이 인터섹(소방산업) 전시회 한국공동관 참가기업 모집 공고,NaN,nan,국내 소방관련 제조업체의 해외시장 진출 확대 및 수출 촉진을 도모하고자 '2022년...,NaN,NaN,NaN,NaN,https://www.bizinfo.go.kr/see/seea/selectSEEA1...,전국,전체,"제조업/운수 및 창고업/전문, 과학 및 기술 서비스업",무관,0.0,0.0,복합,NaN,1.0,NaN,1.0,NaN,NaN,"정보제공/민원,상담,법률"
4,29984,BIZC00070953,2021년 ICT 기반 혁신제품 서비스 아이디어 Biz Project 공모전 공고,전국 / undefined,"중소ㆍ벤처기업(법인), 개인\n\nㅇ 지원자격\n- (Track1) 중소ㆍ벤처기업 ...",ㅇ 공모분야 ICT분야 ‘단기(1.5년 이내)’ 사업화 가능한 ICT 혁신 제품ㆍ서...,- 신청 방법 : 온라인 접수\n정보통신기획평가원 사업관리시스템(iitp.kr)\n...,정보통신기획평가원,정보통신기획평가원 기업지원팀,042-612-8593,https://www.bizinfo.go.kr/see/seea/selectSEEA1...,전국,전체,"정보통신업/전문, 과학 및 기술 서비스업",무관,0.0,0.0,복합,NaN,NaN,1.0,NaN,NaN,NaN,복지
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
998,1002,BIZC00069904,[부산] 2021년 크리에이터스튜디오 상상플러스부산 숏폼 콘텐츠 제작지원 사업 추가...,NaN,nan,"부산 거주 또는 부산 출신 구독자 1,000명 이상 현업 크리에이터를 대상으로 숏폼...",: 이메일 접수- E-mail : tnqls8898@busanit.or.kr,NaN,NaN,NaN,https://www.bizinfo.go.kr/see/seea/selectSEEA1...,부산광역시,전체,무관,무관,0.0,0.0,온라인,NaN,1.0,NaN,NaN,NaN,NaN,정보제공
999,1001,BIZC00069903,[충북] 지역 중소기업 대상 찾아가는 R&D사업 1:1 코칭 안내,NaN,nan,기술개발과제 수행에 따른 경험 부족으로 보고서 작성 및 사업비 집행 등에 어려움을 ...,: 이메일 접수- E-mail : ljw7594@korea.kr,NaN,NaN,NaN,https://www.bizinfo.go.kr/see/seea/selectSEEA1...,충청북도,전체,"전문, 과학 및 기술 서비스업",소기업/중기업,0.0,0.0,복합,NaN,NaN,NaN,NaN,NaN,NaN,
1000,1000,BIZC00069902,[대전] 2021년 나노융합분야 국내전시회 참가지원사업 추가모집 연장 공고,NaN,nan,대전지역 나노융합 관련 산업 중소기업의 제품 홍보 및 판로 확대를 도모하고자 202...,: 온라인 접수- 대전테크노파크 사업정보관리시스템(pims.djtp.or.kr),NaN,NaN,NaN,https://www.bizinfo.go.kr/see/seea/selectSEEA1...,대전광역시,전체,"운수 및 창고업/숙박 및 음식점업/전문, 과학 및 기술 서비스업",소기업/중기업,0.0,0.0,복합,NaN,1.0,1.0,NaN,NaN,NaN,정보제공/복지
1001,999,BIZC00069901,2021년 2차 중소ㆍ중견기업 현장애로기술 지원사업 시행 공고,NaN,nan,"신제품 개발, 기존 제품 개선, 공정 개선, 자격 및 인증 획득, 기술이전 후 상용...",: 이메일 제출\n- deonkim@kaeri.re.kr,NaN,NaN,NaN,https://www.bizinfo.go.kr/see/seea/selectSEEA1...,전국,전체,"전문, 과학 및 기술 서비스업",소기업/중기업/중견기업/대기업,0.0,6.0,복합,1.0,NaN,NaN,NaN,NaN,NaN,"현금,현물"


## **6. 신청절차**

In [ ]:
# 온라인, 오프라인 여부를 따로 측정을 위한 열 생성
p_ent_new.loc[:,'온라인'] = np.NaN
p_ent_new.loc[:,'오프라인'] = np.NaN
p_ent_new.loc[:,'자동'] = np.NaN
p_ent_new.loc[:,'종합'] = np.NaN

# 일부 값은 float형이 있어, '지원내용'열을 문자열 형태로 변형
p_ent_new['지원내용'] = p_ent_new['지원내용'].astype(str)
p_ent_new['온라인신청사이트URL'] = p_ent_new['온라인신청사이트URL'].astype(str)

# 온/오프라인/자동 분류
## 온라인
for i in range(len(p_ent_new)):
    if (('페이지' in p_ent_new.loc[i,'지원내용'])|('사이트' in p_ent_new.loc[i,'지원내용'])
    |('이메일' in p_ent_new.loc[i,'지원내용'])|('인터넷' in p_ent_new.loc[i,'지원내용'])
    |('온라인' in p_ent_new.loc[i,'지원내용'])|('누리집' in p_ent_new.loc[i,'지원내용'])
    |('복지로' in p_ent_new.loc[i,'지원내용'])|('전산' in p_ent_new.loc[i,'지원내용'])
    |('E-mail' in p_ent_new.loc[i,'지원내용'])|('모바일' in p_ent_new.loc[i,'지원내용'])
    |('시스템' in p_ent_new.loc[i,'지원내용'])):
        p_ent_new.loc[i,'온라인']='온라인'
## 오프라인
for i in range(len(p_ent_new)):
    if (('방문' in p_ent_new.loc[i,'지원내용'])|('우편' in p_ent_new.loc[i,'지원내용'])
    |('오프라인' in p_ent_new.loc[i,'지원내용'])|('시·군·구' in p_ent_new.loc[i,'지원내용'])
    |('영업점' in p_ent_new.loc[i,'지원내용'])|('은행' in p_ent_new.loc[i,'지원내용'])
    |('사무소' in p_ent_new.loc[i,'지원내용'])|('기관' in p_ent_new.loc[i,'지원내용'])
    |('지자체' in p_ent_new.loc[i,'지원내용'])|('시․군․구' in p_ent_new.loc[i,'지원내용'])
    |('시‧군‧구' in p_ent_new.loc[i,'지원내용'])|('수협' in p_ent_new.loc[i,'지원내용'])
    |('위원회' in p_ent_new.loc[i,'지원내용'])|('센터' in p_ent_new.loc[i,'지원내용'])
    |('시군' in p_ent_new.loc[i,'지원내용'])|('검사' in p_ent_new.loc[i,'지원내용'])
    |('경찰' in p_ent_new.loc[i,'지원내용'])|('현장' in p_ent_new.loc[i,'지원내용'])
    |('시도' in p_ent_new.loc[i,'지원내용'])|('보건소' in p_ent_new.loc[i,'지원내용'])
    |('민간보조' in p_ent_new.loc[i,'지원내용'])|('유선' in p_ent_new.loc[i,'지원내용'])):
        p_ent_new.loc[i,'오프라인']='오프라인'
## 자동
for i in range(len(p_ent_new)):
    if (('통일부' in p_ent_new.loc[i,'지원내용'])|('하나원' in p_ent_new.loc[i,'지원내용'])
    |('통계교육원' in p_ent_new.loc[i,'지원내용'])|('대입원서' in p_ent_new.loc[i,'지원내용'])):
        p_ent_new.loc[i,'자동']='자동'
## '온라인신청사이트URL'의 값에 따른 온라인 분류
for i in range(len(p_ent_new)):
    if (('http' in p_ent_new.loc[i,'온라인신청사이트URL'])|('go' in p_ent_new.loc[i,'온라인신청사이트URL'])
    |('kr' in p_ent_new.loc[i,'온라인신청사이트URL'])|('@' in p_ent_new.loc[i,'온라인신청사이트URL'])
    |('인터넷' in p_ent_new.loc[i,'온라인신청사이트URL'])|('www' in p_ent_new.loc[i,'온라인신청사이트URL'])
    |('WWW' in p_ent_new.loc[i,'온라인신청사이트URL'])|('net' in p_ent_new.loc[i,'온라인신청사이트URL'])):
        p_ent_new.loc[i,'온라인']='온라인'
## '온라인신청사이트URL'의 값에 따른 오프라인 분류
for i in range(len(p_ent_new)):
    if (('방문' in p_ent_new.loc[i,'지원내용'])|('불가능' in p_ent_new.loc[i,'지원내용'])):
        p_ent_new.loc[i,'오프라인']='오프라인'        
    elif ((p_ent_new.loc[i,'온라인신청사이트URL'] == 'nan') or (p_ent_new.loc[i,'온라인신청사이트URL'] =='-')
    or (p_ent_new.loc[i,'온라인신청사이트URL'] == '정상섭')):
        p_ent_new.loc[i,'오프라인']='오프라인'

# 온라인 열에 '온라인'값과 오프라인 열에 '오프라인'값이 모두 반영되어 있는 경우: 온/오프라인 복합
# 온라인 열에만 '온라인'값이 반영된 경우: 온라인
# 오프라인 열에만 '오프라인'값이 반영된 경우: 오프라인
# 자동 열에만 '자동'값이 반영된 경우: 자동
for i in range(len(p_ent_new)):
    if p_ent_new['온라인'][i] == '온라인' and p_ent_new['오프라인'][i] == '오프라인':
        p_ent_new['종합'][i] = '복합'
    elif p_ent_new['온라인'][i] != '온라인' and p_ent_new['오프라인'][i] == '오프라인':
        p_ent_new['종합'][i] = '오프라인'
    elif p_ent_new['온라인'][i] == '온라인' and p_ent_new['오프라인'][i] != '오프라인':
        p_ent_new['종합'][i] = '온라인'
    elif p_ent_new['자동'][i] == '자동':
        p_ent_new['종합'][i] = '자동'
del p_ent_new['온라인']
del p_ent_new['오프라인']
del p_ent_new['자동']
# del p_ent_new['신청방법']
p_ent_new = p_ent_new.rename(columns={'종합':'신청절차'})
# p_ent_new['종합']을 df_policy['태그-지원내용']로 사용
# df_policy['태그-지원내용'] = p_ent_new['종합']

In [ ]:
p_ent_new = p_ent_new_raw.copy()

In [ ]:
p_ent_new

,pid,서비스ID,서비스명,서비스대상지역,지원대상,지원내용,신청방법,소관기관,접수기관명,문의처전화번호,온라인신청사이트URL,지역(시도),지역(시군구),업종(대분류),기업규모,태그_업력(시작),태그_업력(종료),신청절차
0,29996,BIZC00070957,[경기] 양주시 2021년 3차 해외 규격인증지원사업 모집 공고(사후지원),NaN,nan,제품 신뢰성 제고를 통한 기업 경쟁력을 향상시키기 위하여 양주시 소재 중소제조기업을...,NaN,NaN,NaN,NaN,https://www.bizinfo.go.kr/see/seea/selectSEEA1...,경기도,양주시,제조업,소기업/중기업,0.0,0.0,복합
1,29994,BIZC00070956,[광주] 양동전통시장 테마음식 핵 점포 추진 및 공간조성 참여 예비창업자 모집 공고,NaN,nan,양동전통시장 내 테마음식 특화거리 조성하여 예비 창업자의 성공적인 창업을 위해 다양...,NaN,NaN,NaN,NaN,https://www.bizinfo.go.kr/see/seea/selectSEEA1...,광주광역시,서구,"농업, 임업 및 어업/공공 행정, 국방 및 사회보장 행정/교육 서비스업",소기업/중기업/소상공인,0.0,0.0,복합
2,29989,BIZC00070955,2021년 청년몰 연말 오프라인 행사지원 참여 청년몰 모집 공고,NaN,nan,청년몰 활성화 및 청년상인 매출확대 도모를 위해 '2021년 청년몰 연말 오프라인 ...,NaN,NaN,NaN,NaN,https://www.bizinfo.go.kr/see/seea/selectSEEA1...,전국,전체,무관,소상공인,0.0,0.0,복합
3,29991,BIZC00070954,2022년 두바이 인터섹(소방산업) 전시회 한국공동관 참가기업 모집 공고,NaN,nan,국내 소방관련 제조업체의 해외시장 진출 확대 및 수출 촉진을 도모하고자 '2022년...,NaN,NaN,NaN,NaN,https://www.bizinfo.go.kr/see/seea/selectSEEA1...,전국,전체,"제조업/운수 및 창고업/전문, 과학 및 기술 서비스업",무관,0.0,0.0,복합
4,29984,BIZC00070953,2021년 ICT 기반 혁신제품 서비스 아이디어 Biz Project 공모전 공고,전국 / undefined,"중소ㆍ벤처기업(법인), 개인\n\nㅇ 지원자격\n- (Track1) 중소ㆍ벤처기업 ...",ㅇ 공모분야 ICT분야 ‘단기(1.5년 이내)’ 사업화 가능한 ICT 혁신 제품ㆍ서...,- 신청 방법 : 온라인 접수\n정보통신기획평가원 사업관리시스템(iitp.kr)\n...,정보통신기획평가원,정보통신기획평가원 기업지원팀,042-612-8593,https://www.bizinfo.go.kr/see/seea/selectSEEA1...,전국,전체,"정보통신업/전문, 과학 및 기술 서비스업",무관,0.0,0.0,복합
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
998,1002,BIZC00069904,[부산] 2021년 크리에이터스튜디오 상상플러스부산 숏폼 콘텐츠 제작지원 사업 추가...,NaN,nan,"부산 거주 또는 부산 출신 구독자 1,000명 이상 현업 크리에이터를 대상으로 숏폼...",: 이메일 접수- E-mail : tnqls8898@busanit.or.kr,NaN,NaN,NaN,https://www.bizinfo.go.kr/see/seea/selectSEEA1...,부산광역시,전체,무관,무관,0.0,0.0,온라인
999,1001,BIZC00069903,[충북] 지역 중소기업 대상 찾아가는 R&D사업 1:1 코칭 안내,NaN,nan,기술개발과제 수행에 따른 경험 부족으로 보고서 작성 및 사업비 집행 등에 어려움을 ...,: 이메일 접수- E-mail : ljw7594@korea.kr,NaN,NaN,NaN,https://www.bizinfo.go.kr/see/seea/selectSEEA1...,충청북도,전체,"전문, 과학 및 기술 서비스업",소기업/중기업,0.0,0.0,복합
1000,1000,BIZC00069902,[대전] 2021년 나노융합분야 국내전시회 참가지원사업 추가모집 연장 공고,NaN,nan,대전지역 나노융합 관련 산업 중소기업의 제품 홍보 및 판로 확대를 도모하고자 202...,: 온라인 접수- 대전테크노파크 사업정보관리시스템(pims.djtp.or.kr),NaN,NaN,NaN,https://www.bizinfo.go.kr/see/seea/selectSEEA1...,대전광역시,전체,"운수 및 창고업/숙박 및 음식점업/전문, 과학 및 기술 서비스업",소기업/중기업,0.0,0.0,복합
1001,999,BIZC00069901,2021년 2차 중소ㆍ중견기업 현장애로기술 지원사업 시행 공고,NaN,nan,"신제품 개발, 기존 제품 개선, 공정 개선, 자격 및 인증 획득, 기술이전 후 상용...",: 이메일 제출\n- deonkim@kaeri.re.kr,NaN,NaN,NaN,https://www.bizinfo.go.kr/see/seea/selectSEEA1...,전국,전체,"전문, 과학 및 기술 서비스업",소기업/중기업/중견기업/대기업,0.0,6.0,복합


In [ ]:
p_ent_new

,pid,서비스ID,서비스명,서비스대상지역,지원대상,지원내용,신청방법,소관기관,접수기관명,문의처전화번호,온라인신청사이트URL,지역(시도),지역(시군구),업종(대분류),기업규모,태그_업력(시작),태그_업력(종료),신청절차
0,29996,BIZC00070957,[경기] 양주시 2021년 3차 해외 규격인증지원사업 모집 공고(사후지원),NaN,nan,제품 신뢰성 제고를 통한 기업 경쟁력을 향상시키기 위하여 양주시 소재 중소제조기업을...,NaN,NaN,NaN,NaN,https://www.bizinfo.go.kr/see/seea/selectSEEA1...,경기도,양주시,제조업,소기업/중기업,0.0,0.0,복합
1,29994,BIZC00070956,[광주] 양동전통시장 테마음식 핵 점포 추진 및 공간조성 참여 예비창업자 모집 공고,NaN,nan,양동전통시장 내 테마음식 특화거리 조성하여 예비 창업자의 성공적인 창업을 위해 다양...,NaN,NaN,NaN,NaN,https://www.bizinfo.go.kr/see/seea/selectSEEA1...,광주광역시,서구,"농업, 임업 및 어업/공공 행정, 국방 및 사회보장 행정/교육 서비스업",소기업/중기업/소상공인,0.0,0.0,복합
2,29989,BIZC00070955,2021년 청년몰 연말 오프라인 행사지원 참여 청년몰 모집 공고,NaN,nan,청년몰 활성화 및 청년상인 매출확대 도모를 위해 '2021년 청년몰 연말 오프라인 ...,NaN,NaN,NaN,NaN,https://www.bizinfo.go.kr/see/seea/selectSEEA1...,전국,전체,무관,소상공인,0.0,0.0,복합
3,29991,BIZC00070954,2022년 두바이 인터섹(소방산업) 전시회 한국공동관 참가기업 모집 공고,NaN,nan,국내 소방관련 제조업체의 해외시장 진출 확대 및 수출 촉진을 도모하고자 '2022년...,NaN,NaN,NaN,NaN,https://www.bizinfo.go.kr/see/seea/selectSEEA1...,전국,전체,"제조업/운수 및 창고업/전문, 과학 및 기술 서비스업",무관,0.0,0.0,복합
4,29984,BIZC00070953,2021년 ICT 기반 혁신제품 서비스 아이디어 Biz Project 공모전 공고,전국 / undefined,"중소ㆍ벤처기업(법인), 개인\n\nㅇ 지원자격\n- (Track1) 중소ㆍ벤처기업 ...",ㅇ 공모분야 ICT분야 ‘단기(1.5년 이내)’ 사업화 가능한 ICT 혁신 제품ㆍ서...,- 신청 방법 : 온라인 접수\n정보통신기획평가원 사업관리시스템(iitp.kr)\n...,정보통신기획평가원,정보통신기획평가원 기업지원팀,042-612-8593,https://www.bizinfo.go.kr/see/seea/selectSEEA1...,전국,전체,"정보통신업/전문, 과학 및 기술 서비스업",무관,0.0,0.0,복합
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
998,1002,BIZC00069904,[부산] 2021년 크리에이터스튜디오 상상플러스부산 숏폼 콘텐츠 제작지원 사업 추가...,NaN,nan,"부산 거주 또는 부산 출신 구독자 1,000명 이상 현업 크리에이터를 대상으로 숏폼...",: 이메일 접수- E-mail : tnqls8898@busanit.or.kr,NaN,NaN,NaN,https://www.bizinfo.go.kr/see/seea/selectSEEA1...,부산광역시,전체,무관,무관,0.0,0.0,온라인
999,1001,BIZC00069903,[충북] 지역 중소기업 대상 찾아가는 R&D사업 1:1 코칭 안내,NaN,nan,기술개발과제 수행에 따른 경험 부족으로 보고서 작성 및 사업비 집행 등에 어려움을 ...,: 이메일 접수- E-mail : ljw7594@korea.kr,NaN,NaN,NaN,https://www.bizinfo.go.kr/see/seea/selectSEEA1...,충청북도,전체,"전문, 과학 및 기술 서비스업",소기업/중기업,0.0,0.0,복합
1000,1000,BIZC00069902,[대전] 2021년 나노융합분야 국내전시회 참가지원사업 추가모집 연장 공고,NaN,nan,대전지역 나노융합 관련 산업 중소기업의 제품 홍보 및 판로 확대를 도모하고자 202...,: 온라인 접수- 대전테크노파크 사업정보관리시스템(pims.djtp.or.kr),NaN,NaN,NaN,https://www.bizinfo.go.kr/see/seea/selectSEEA1...,대전광역시,전체,"운수 및 창고업/숙박 및 음식점업/전문, 과학 및 기술 서비스업",소기업/중기업,0.0,0.0,복합
1001,999,BIZC00069901,2021년 2차 중소ㆍ중견기업 현장애로기술 지원사업 시행 공고,NaN,nan,"신제품 개발, 기존 제품 개선, 공정 개선, 자격 및 인증 획득, 기술이전 후 상용...",: 이메일 제출\n- deonkim@kaeri.re.kr,NaN,NaN,NaN,https://www.bizinfo.go.kr/see/seea/selectSEEA1...,전국,전체,"전문, 과학 및 기술 서비스업",소기업/중기업/중견기업/대기업,0.0,6.0,복합


## **7. 지원유형**

In [ ]:
for i in range(len(p_ent_new)):
    if (('현금' in p_ent_new.loc[i, '지원내용'])|('현물' in p_ent_new.loc[i, '지원내용'])
    |('대납' in p_ent_new.loc[i, '지원내용'])|('융자' in p_ent_new.loc[i, '지원내용'])
    |('대여' in p_ent_new.loc[i, '지원내용'])|('감면' in p_ent_new.loc[i, '지원내용'])):
        p_ent_new.loc[i, '현금,현물'] = 1
for i in range(len(p_ent_new)):
    if ('정보제공' in p_ent_new.loc[i, '지원내용'])|('정보' in p_ent_new.loc[i, '지원내용']):
        p_ent_new.loc[i, '정보제공'] = 1
for i in range(len(p_ent_new)):
    if (('문화' in p_ent_new.loc[i, '지원내용'])|('서비스' in p_ent_new.loc[i, '지원내용'])
    |('이용권' in p_ent_new.loc[i, '지원내용'])|('의료' in p_ent_new.loc[i, '지원내용'])):
        p_ent_new.loc[i, '복지'] = 1
for i in range(len(p_ent_new)):
    if (('민원' in p_ent_new.loc[i, '지원내용'])|('상담' in p_ent_new.loc[i, '지원내용'])):
        p_ent_new.loc[i, '민원,상담,법률'] = 1
for i in range(len(p_ent_new)):
    if ('봉사' in p_ent_new.loc[i, '지원내용'])|('기부' in p_ent_new.loc[i, '지원내용']):
        p_ent_new.loc[i, '봉사,기부'] = 1
for i in range(len(p_ent_new)):
    if ('교육' in p_ent_new.loc[i, '지원내용'])|('용역' in p_ent_new.loc[i, '지원내용']):
        p_ent_new.loc[i, '교육,용역'] = 1
for i in range(len(p_ent_new)):
    if '시설이용' in p_ent_new.loc[i, '지원내용']:
        p_ent_new.loc[i, '시설이용'] = 1

In [ ]:
p_ent_new.columns

Index(['pid', '정책서비스ID', '서비스명', '서비스대상지역', '지원대상', '지원내용', '신청방법', '소관기관',
       '접수기관명', '문의처전화번호', '온라인신청사이트URL', '태그-지역(시도)', '태그-지역(시군구)',
       '태그-업종(대분류)', '태그-기업규모', '태그-업력(시작)', '태그-업력(종료)', '태그-신청절차', '현금,현물',
       '정보제공', '복지', '민원,상담,법률', '봉사,기부', '교육,용역'],
      dtype='object')

In [ ]:
p_ent_new['지원유형'] = ''
for i in range(len(p_ent_new)):
  values = p_ent_new.loc[i,'현금,현물':'교육,용역'].values
  if values.sum()==0 :
    p_ent_new.loc[i,'지원유형']= '기타'
  else:
    for j in range(len(values)):
      if j == 0:
        if values[j]==1:
          p_ent_new.loc[i,'지원유형']+='현금,현물'
      elif j == 1:
        if values[j]==1:
          if p_ent_new.loc[i,'지원유형']=='':
            p_ent_new.loc[i,'지원유형']+='정보제공'
          else:
            p_ent_new.loc[i,'지원유형']+='/'+'정보제공'
  
      elif j == 2:
        if values[j]==1:
          if p_ent_new.loc[i,'지원유형']=='':
            p_ent_new.loc[i,'지원유형']+='복지'
          else:
            p_ent_new.loc[i,'지원유형']+='/'+'복지'
      elif j == 3:
        if values[j]==1:
          if p_ent_new.loc[i,'지원유형']=='':
            p_ent_new.loc[i,'지원유형']+='민원,상담,법률'
          else:
            p_ent_new.loc[i,'지원유형']+='/'+'민원,상담,법률'
      elif j == 4:
        if values[j]==1:
          if p_ent_new.loc[i,'지원유형']=='':
            p_ent_new.loc[i,'지원유형']+='봉사,기부'
          else:
            p_ent_new.loc[i,'지원유형']+='/'+'봉사,기부'
      elif j == 5:
        if values[j]==1:
          if p_ent_new.loc[i,'지원유형']=='':
            p_ent_new.loc[i,'지원유형']+='교육,용역'
          else:
            p_ent_new.loc[i,'지원유형']+='/'+'교육,용역'

p_ent_new=p_ent_new.fillna('기타').drop(['현금,현물', '정보제공', '복지','민원,상담,법률', '봉사,기부', '교육,용역'],axis=1)

* 기존 정책 데이터 지원유형 바꾸기

In [ ]:
p_ent_original=pd.read_csv('/content/drive/MyDrive/Wello+project_신희택,윤다솜/Data/p_ent_ver3.csv')
p_ent_original

,정책서비스ID,서비스명,태그-신청절차,태그-지원유형,태그-지역(시도),태그-지역(시군구),태그-기업규모,태그-업종(대분류),태그-업력(시작),태그-업력(종료)
0,116010000027,‘힘내라 대한민국’특별운영자금 (산업은행),복합,현금(융자),전국,전체,소기업/중기업/중견기업,무관,NaN,NaN
1,119200000016,원양어업 경영자금 지원,오프라인,"현금,현금(융자)",전국,전체,무관,"농업, 임업 및 어업",NaN,NaN
2,119200000024,수산장비 구입 지원,오프라인,현금(융자),전국,전체,무관,"농업, 임업 및 어업/제조업",NaN,NaN
3,119200000027,산지 및 소비지 유통자금 융자 지원,오프라인,현금(융자),전국,전체,무관,도매 및 소매업,NaN,NaN
4,119200000044,수산경영인회생자금,오프라인,"현금,현금(융자)",전국,전체,무관,"농업, 임업 및 어업",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
16931,PTR000050134,장애인 직업생활상담원 양성교육,복합,정보제공,전국,전체,소기업/중기업/소상공인,교육 서비스업/보건업 및 사회복지 서비스업,NaN,NaN
16932,PTR000050137,장애인 고용종사자 교육연수,온라인,교육/용역,전국,전체,소기업/중기업/소상공인,"전문, 과학 및 기술 서비스업/교육 서비스업/보건업 및 사회복지 서비스업",NaN,NaN
16933,611000014088,농작물 병해충 예찰방제 지원,오프라인,정보제공,경기도,이천시,소기업/중기업/소상공인,"농업, 임업 및 어업/공공 행정, 국방 및 사회보장 행정",NaN,NaN
16934,369000000418,기본형 공익직접지불금 지원,오프라인,NaN,울산광역시,중구,소기업/중기업/소상공인,"농업, 임업 및 어업",NaN,NaN


In [ ]:
# 태그-지원유형 One-hot
# 태그-지원유형: 현금/현물, 정보제공, 복지, 민원/상담/법률, 봉사/기부, 교육/용역, 시설이용, 기타
p_ent_original['현금,현물'] = 0
p_ent_original['정보제공'] = 0
p_ent_original['복지'] = 0
p_ent_original['민원,상담,법률'] = 0
p_ent_original['봉사,기부'] = 0
p_ent_original['교육,용역'] = 0
p_ent_original['시설이용'] = 0
p_ent_original['기타'] = 0

p_ent_original['태그-지원유형'] = p_ent_original['태그-지원유형'].astype(str)

for i in range(len(p_ent_original)):
    if (('현금' in p_ent_original.loc[i, '태그-지원유형'])|('현물' in p_ent_original.loc[i, '태그-지원유형'])
    |('대납' in p_ent_original.loc[i, '태그-지원유형'])|('융자' in p_ent_original.loc[i, '태그-지원유형'])
    |('대여' in p_ent_original.loc[i, '태그-지원유형'])|('감면' in p_ent_original.loc[i, '태그-지원유형'])):
        p_ent_original.loc[i, '현금,현물'] = 1
for i in range(len(p_ent_original)):
    if '정보제공' in p_ent_original.loc[i, '태그-지원유형']:
        p_ent_original.loc[i, '정보제공'] = 1
for i in range(len(p_ent_original)):
    if (('문화' in p_ent_original.loc[i, '태그-지원유형'])|('서비스' in p_ent_original.loc[i, '태그-지원유형'])
    |('이용권' in p_ent_original.loc[i, '태그-지원유형'])|('의료' in p_ent_original.loc[i, '태그-지원유형'])):
        p_ent_original.loc[i, '복지'] = 1
for i in range(len(p_ent_original)):
    if (('민원' in p_ent_original.loc[i, '태그-지원유형'])|('상담' in p_ent_original.loc[i, '태그-지원유형'])):
        p_ent_original.loc[i, '민원,상담,법률'] = 1
for i in range(len(p_ent_original)):
    if '봉사/기부' in p_ent_original.loc[i, '태그-지원유형']:
        p_ent_original.loc[i, '봉사,기부'] = 1
for i in range(len(p_ent_original)):
    if '교육/용역' in p_ent_original.loc[i, '태그-지원유형']:
        p_ent_original.loc[i, '교육,용역'] = 1
for i in range(len(p_ent_original)):
    if '시설이용' in p_ent_original.loc[i, '태그-지원유형']:
        p_ent_original.loc[i, '시설이용'] = 1
for i in range(len(p_ent_original)):
    if ('기타,' in p_ent_original.loc[i, '태그-지원유형'])|('기타(교육)' in p_ent_original.loc[i, '태그-지원유형']):
        p_ent_original.loc[i, '기타'] = 1

In [ ]:
p_ent_original['지원유형'] = ''
for i in range(len(p_ent_original)):
  values = p_ent_original.loc[i,'현금,현물':'교육,용역'].values
  if values.sum()==0 :
    p_ent_original.loc[i,'지원유형']= '기타'
  else:
    for j in range(len(values)):
      if j == 0:
        if values[j]==1:
          p_ent_original.loc[i,'지원유형']+='현금,현물'
      elif j == 1:
        if values[j]==1:
          if p_ent_original.loc[i,'지원유형']=='':
            p_ent_original.loc[i,'지원유형']+='정보제공'
          else:
            p_ent_original.loc[i,'지원유형']+='/'+'정보제공'
  
      elif j == 2:
        if values[j]==1:
          if p_ent_original.loc[i,'지원유형']=='':
            p_ent_original.loc[i,'지원유형']+='복지'
          else:
            p_ent_original.loc[i,'지원유형']+='/'+'복지'
      elif j == 3:
        if values[j]==1:
          if p_ent_original.loc[i,'지원유형']=='':
            p_ent_original.loc[i,'지원유형']+='민원,상담,법률'
          else:
            p_ent_original.loc[i,'지원유형']+='/'+'민원,상담,법률'
      elif j == 4:
        if values[j]==1:
          if p_ent_original.loc[i,'지원유형']=='':
            p_ent_original.loc[i,'지원유형']+='봉사,기부'
          else:
            p_ent_original.loc[i,'지원유형']+='/'+'봉사,기부'
      elif j == 5:
        if values[j]==1:
          if p_ent_original.loc[i,'지원유형']=='':
            p_ent_original.loc[i,'지원유형']+='교육,용역'
          else:
            p_ent_original.loc[i,'지원유형']+='/'+'교육,용역'

p_ent_original=p_ent_original.drop(['현금,현물', '정보제공', '복지','민원,상담,법률', '봉사,기부', '교육,용역','태그-지원유형','기타'],axis=1).rename(columns={'지원유형':'태그-지원유형'})
p_ent_original=p_ent_original[['정책서비스ID','서비스명','태그-신청절차'	,'태그-지원유형','태그-지역(시도)',	'태그-지역(시군구)'	,'태그-기업규모','태그-업종(대분류)','태그-업력(시작)',	'태그-업력(종료)']]
p_ent_original['태그-업력(시작)']=p_ent_original['태그-업력(시작)'].fillna(0)
p_ent_original['태그-업력(종료)']=p_ent_original['태그-업력(종료)'].fillna(0)


## 합치기

In [ ]:
del p_ent_new['태그-지원유형']

In [ ]:
p_ent_new = p_ent_new.rename(columns={'서비스ID':'정책서비스ID','신청절차':'태그-신청절차','지원유형':'태그-지원유형','지역(시도)':'태그-지역(시도)','지역(시군구)':'태그-지역(시군구)','기업규모':'태그-기업규모','업종(대분류)':'태그-업종(대분류)','태그_업력(시작)':'태그-업력(시작)','태그_업력(종료)':'태그-업력(종료)'})
p_ent_new2=p_ent_new[['정책서비스ID','서비스명','태그-신청절차'	,'태그-지원유형','태그-지역(시도)',	'태그-지역(시군구)'	,'태그-기업규모','태그-업종(대분류)','태그-업력(시작)',	'태그-업력(종료)']]

In [ ]:
for i in range(len(p_ent_new2)):
    if p_ent_new2.loc[i,'태그-지원유형']!='':
        pass
    else:
        p_ent_new2.loc[i,'태그-지원유형']='기타'

In [ ]:
print(p_ent_new2.columns)
print(p_ent_original.columns)
print(p_ent_new2.shape)
print(p_ent_original.shape)

Index(['정책서비스ID', '서비스명', '태그-신청절차', '태그-지원유형', '태그-지역(시도)', '태그-지역(시군구)',
       '태그-기업규모', '태그-업종(대분류)', '태그-업력(시작)', '태그-업력(종료)'],
      dtype='object')
Index(['정책서비스ID', '서비스명', '태그-신청절차', '태그-지원유형', '태그-지역(시도)', '태그-지역(시군구)',
       '태그-기업규모', '태그-업종(대분류)', '태그-업력(시작)', '태그-업력(종료)'],
      dtype='object')
(1003, 10)
(16936, 10)


In [ ]:
p_ent_new2.to_csv('p_ent_train.csv',encoding='utf-8-sig',index=False)
p_ent_original.to_csv('p_ent_test.csv',encoding='utf-8-sig',index=False)
p_ent_final = pd.concat([p_ent_new2,p_ent_original],axis=0)
p_ent_final.to_csv('p_ent_final.csv',encoding='utf-8-sig',index=False)